__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x38_d03_reorganisieren.ipynb)__

# Reorganisieren und Reinigen
* speichert in data_03
* lex_test level 3
* 41 min

## ToDo
* löschliste am Ende nochmal anwenden
* Prüfen: ändert sich am score in translate_lex_full noch was, wenn echte Statistikzahlen da mit einfließen? Es geht um die Zuordnung lex --> lemma bei gegebenem tagZZ, also z.B. bei 'Alben' (--> Alb, Album, etc.).
* Symmetrie: Es gibt Lexeme für das Lemma "Ackermann/NN". Die gleichen Lexeme sollte es auch für das Lemma "Ackermann/NE" geben.
* Der gleiche Fall triff auch beim Lexem "ertragene" auf. Das sollte nicht nur ein Lexem des Verbs sein, sondern auch ein Lexem des Adjektivs.
* Noch besser: Lemmas zusammenfassen und beide Tag-Felder nutzen

## Spacy Nachschlagen 
* Kann das Lemma stimmen? >> Nachschlagen in wiktionary_merkmal (hier gibt es alle lex-lemma-Kombinationen, z.B. bei "Alben")
* Wenn nicht fündig geworden: Nachschlagen in translate_lex (hier gibt es nur 1 Lemma pro tagZZ)
* Resultat: Lemma

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 17:51:44


time: 409 ms


In [2]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, swifter

except ImportError as e:
    !pip install pandasklar swifter
    import pandasklar as pak, bpyth as bpy, swifter
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 4.2 s


## Einstellungen

In [5]:
# Einstellungen

# Was debuggen?
suche_debug = ['besonnte','überlegte',]   
suche_debug = ['besonne',]  # auch gut: besinnen
suche_debug = ['inner-','innere','inner']  
suche_debug = ['Citrone',]  
suche_debug = ['Dezember','December','Citrone',]  
suche_debug = ['Anna',]  
# 'einen','beigen','Montage',
# '24/7','Schriftwerk'
suche_debug = ['gefriergetrocknet','gefriertrocknen','missraten']  


gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True

# verbose
pak.Config.set('VERBOSE', True)

# Load
verzeichnis_load                       = 'data_02'
wiktionary_lemma_filename              = verzeichnis_load + '/wiktionary_lemma.pickle'  
translate_lex_filename                 = verzeichnis_load + '/wiktionary_nolemma.pickle'    # hieß vorher wiktionary_nolemma
wiktionary_merkmal_filename            = verzeichnis_load + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename       = verzeichnis_load + '/wiktionary_merkmal_text.pickle'  

# Steuertabellen
lex_test_filename                      = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                    = 'Steuertabellen/löschliste.xlsx'   
lexeme_manuell_filename                = 'Steuertabellen/lexeme_manuell.xlsx'                              # manuell eingepflegte Lexeme 
     
# Save
verzeichnis_save                       = 'data_03'
wiktionary_lemma_filename_save         = verzeichnis_save + '/wiktionary_lemma.pickle'  
translate_lex_filename_save            = verzeichnis_save + '/translate_lex.pickle'  
translate_lex_trash_filename_save      = verzeichnis_save + '/translate_lex_trash.pickle'  
translate_lemma_filename_save          = verzeichnis_save + '/translate_lemma.pickle'  
wiktionary_merkmal_filename_save       = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save  = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  



VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 64.8 ms


## Vorbereiten

In [4]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 44.4 ms


In [5]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

time: 38.9 ms


In [6]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

time: 43.8 ms


In [3]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

time: 3.37 s


from pandarallel import pandarallel
nb_workers = os.cpu_count()
if nb_workers > 2:
    nb_workers = int((nb_workers / 2) - 1)
pandarallel.initialize(use_memory_fs=True, nb_workers=nb_workers, progress_bar=True)  

## Daten laden
* Steuertabellen
* wiktionary_lemma
* translate_lex
* wiktionary_merkmal     
* wiktionary_merkmal_text 
* jeweils einen Trash generieren

In [6]:
#blab.help(plan_merkmal_erstellen)

time: 48.4 ms


In [7]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


time: 352 ms


In [8]:
a= pak.analyse_freqs(plan_merkmal,['plan','merkmal'])
grid(a, width=200)

3 rows


time: 203 ms


In [9]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3)
grid(lex_test)

79 rows loaded
Delete 3 rows from 79
76 rows


time: 471 ms


In [10]:
# lexeme_manuell laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag',    'lemma_tag' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag_lex','lex_tag' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'lex_tag','lemma_tag')
lexeme_manuell

1794 rows loaded


,lex,lemma,lemma_tag,lex_tag,member,genus,tab
0,abberufen,abberufen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
1,abgepaßt,abpassen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
2,achtend,achten,VVFIN,Ptz1,<NA>,<NA>,Partizipien
3,anbehalten,anbehalten,VVFIN,Ptz2,<NA>,<NA>,Partizipien
4,aneinandergepreßt,aneinanderpressen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
...,...,...,...,...,...,...,...
1789,überein,<NA>,ADV,PTKVZ,<NA>,<NA>,Partikel
1790,www,<NA>,TRUNC,<NA>,<NA>,<NA>,Partikel
1791,zurecht,<NA>,ADV,PTKVZ,<NA>,<NA>,Partikel
1792,Susannah,<NA>,NE,<NA>,Vorname,f,Namen


time: 508 ms


In [11]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
translate_lex            = pak.load_pickle(translate_lex_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

165346 rows loaded
644797 rows loaded
4650364 rows loaded
621260 rows loaded
time: 13.3 s


In [12]:
# rename_col
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'section_id2','lemma_id')
translate_lex           = pak.rename_col(translate_lex,'lex','data')

time: 711 ms


In [13]:
# trash erzeugen     
wiktionary_lemma_trash        = wiktionary_lemma.head(0)
wiktionary_lemma_trash        = pak.write_empty_col(wiktionary_lemma_trash, 'msg', 'string')

translate_lex_trash           = translate_lex.head(0)
translate_lex_trash           = pak.write_empty_col(translate_lex_trash, 'msg', 'string')

wiktionary_merkmal_trash      = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash      = pak.write_empty_col(wiktionary_merkmal_trash, 'msg', 'string')

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash = pak.write_empty_col(wiktionary_merkmal_text_trash, 'msg', 'string')

time: 50.6 ms


## suche_debug

In [14]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

7 rows


time: 276 ms


In [15]:
# wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma_trash[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag,msg


time: 59.7 ms


In [16]:
pak.analyse_freqs(wiktionary_lemma_trash,['msg','lemma_id'])

'No rows'

time: 38 ms


In [17]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

35 rows


time: 1.23 s


In [18]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

7 rows


time: 646 ms


In [19]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(suche_debug) #== 'gefriergetrocknet'
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

55 rows


time: 476 ms


In [20]:
# wiktionary_merkmal
# data_id ist bei allen gm-Merkmalen potentiell falsch!
mask1 = wiktionary_merkmal.data == 'besonnen'
mask2 = wiktionary_merkmal.merkmal == 'gm'
mask = mask1  &  mask2
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

29 rows


time: 1.21 s


In [21]:
# Genügend Datensätze?
assert wiktionary_merkmal.shape[0] > 3320 * 1000

time: 59 ms


In [22]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique
assert translate_lex.nolemma_id.is_unique

time: 216 ms


## wiktionary_merkmal
* is_lex

In [23]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 6.14 s


In [24]:
# korrigieren: Punkt hinten ergänzen
startet_nicht_mit = ( '.', 'Industrie', 'St.')
endet_nicht_mit   = ( '.', 'Tag', 'Ort','Länge','Sieg')
mask1 =  wiktionary_merkmal.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal.data.str.contains('[0123456789]', regex=True)  
mask3 = ~wiktionary_merkmal.data.str.startswith(startet_nicht_mit)
mask4 = ~wiktionary_merkmal.data.str.endswith(  endet_nicht_mit)
mask5 = (wiktionary_merkmal.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask6 =  wiktionary_merkmal.data.str.len() > 2
mask7 = ~wiktionary_merkmal.merkmal.isin(['ipa'])
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask( wiktionary_merkmal, mask, 233 )

wiktionary_merkmal.loc[mask,'data'] += '.'
grid(wiktionary_merkmal,mask)

check_mask:  233 rows
233 rows out of 4650364


time: 15.7 s


In [25]:
wiktionary_merkmal.head(1)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
0,Hallo,syn,Gejohle,Gejohle,,NN,1,,7.282582,False


time: 78.8 ms


In [26]:
# Merkmale, die eigentlich tags sind
mask = wiktionary_merkmal.merkmal.isin(alle_tags)
check_mask( wiktionary_merkmal, mask, 10 )

wiktionary_merkmal.loc[mask,'lemma_tag'] = ''
wiktionary_merkmal.loc[mask,'data_tag']  = wiktionary_merkmal[mask].merkmal
wiktionary_merkmal.loc[mask,'merkmal']   = 'lex'
wiktionary_merkmal.loc[mask,'is_lex']    = True
grid(wiktionary_merkmal,mask)

check_mask:  11 rows
11 rows out of 4650364


time: 904 ms


In [27]:
# Mehrfache Spaces >> Einfacher Space (eigentlich hätten die alle schon von Preprocess erwischt werden müssen)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

time: 2.84 s


In [28]:
# Mehrfachbindestrich
mask =  wiktionary_merkmal.data == '--'
wiktionary_merkmal.loc[mask,'data'] = '-'

time: 662 ms


In [29]:
# Hochkommas entfernen
mask1 = wiktionary_merkmal.data.str.count("\'") > 1
mask2 = wiktionary_merkmal.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace("'", '')

check_mask:  7 rows
time: 2.21 s


In [30]:
# Leerstring data Müll löschen
mask = wiktionary_merkmal['data'] == ''
check_mask(wiktionary_merkmal,mask,9)

msg = 'Leerstring data Müll'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
grid(wiktionary_merkmal_trash, msg)

check_mask:  9 rows
Move 9 rows from 4650364
9 rows


time: 4.99 s


In [31]:
# fillna
wiktionary_merkmal['meta']       = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']   = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']  = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']        = wiktionary_merkmal.num.fillna('')

time: 2.1 s


## translate_lex ==========
* hieß bislang wiktionary_nolemma
* Gerundivum als Merkmal speichern
* verwaiste löschen
* Spalten umbenennen
* pflege_translate_tabelle

In [32]:
assert not pak.any_nan(translate_lex)

time: 551 ms


In [33]:
# lemma_id vorhanden?
mask = translate_lex.lemma_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows
time: 126 ms


In [34]:
# nolemma_id vorhanden und nicht widersprüchlich?
mask = translate_lex.nolemma_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='nolemma_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 1.74 s


### Gerundivum als Merkmal speichern

In [35]:
# Gerundivum finden
mask1 =  translate_lex.data.str.contains('zu')
mask2 = ~translate_lex.data.str.endswith('zu')
mask3 =  translate_lex.tag.str.contains('LEX_A') 
mask4 =  translate_lex.member.str.contains('Dekliniertes_Gerundivum') 
mask = mask1  & mask2  &  mask3  &  mask4
check_mask(translate_lex,mask,16065)

df = translate_lex[mask].copy()
grid(df)

check_mask:  16065 rows
16065 rows


time: 1.2 s


In [36]:
# anfügen vorbereiten
df = pak.rename_col( df, 'lex',        'data' )
df = pak.rename_col( df, 'nolemma_id', 'data_id' )
df['merkmal']  = 'Gerundivum'
df['meta']     = ''
df['num']      = ''
df['is_lex']   = True
df['data_tag'] = 'ADJA'
df = pak.drop_cols( df, set(df.columns) - set(wiktionary_merkmal.columns) )
df = pak.update_col(df, wiktionary_lemma, left_on='lemma_id', right_on='lemma_id', col='tag', col_rename='lemma_tag')

update_col: 16065 cells written into new column
time: 409 ms


In [37]:
# Prüfen: Alle Gerundiva verweisen auf ein Verb
mask_L = wiktionary_lemma.tagZZ == 'V'
verben = wiktionary_lemma[mask_L].lemma_id

mask = ~df.lemma_id.isin(verben)
check_mask(df,mask,0)

check_mask:  0 rows
time: 73.4 ms


In [38]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df)

16065 rows added, now a total of 4666420
time: 2.14 s


In [39]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 5.47 s


In [40]:
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
grid(wiktionary_merkmal,mask)

16065 rows out of 4666420


time: 521 ms


### translate_lex pflegen

In [41]:
#grid(translate_lex,pak.sample)

time: 37.6 ms


In [42]:
# Verwaiste löschen
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,30)

msg = 'verwaist_lemma_id'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  7 rows
Move 7 rows from 644797
7 rows


time: 947 ms


In [43]:
# umformen
translate_lex = pak.rename_col( translate_lex, 'lex',        'data' )
translate_lex = pak.rename_col( translate_lex, 'nolemma_id', 'data_id' )
translate_lex['member'] = ('wiki ' + translate_lex['member'] ).str.strip()
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma)

update_col: 644790 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 644790 cells written into existing column
update_col: 644790 cells written into new column
renaming score --> data_score
time: 35.2 s


In [44]:
# Müll
mask = translate_lex.lemma_tag == 'TODO'
check_mask(translate_lex,mask,0,5)

msg = 'TODO'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )

check_mask:  5 rows
Move 5 rows from 644790
time: 1.23 s


In [45]:
# not any nan
assert not pak.any_nan(translate_lex)

time: 405 ms


In [46]:
pak.sample(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,2.382460,wiki Deklinierte_Form,lieb,lieb,ADJA,A,9.054811,lieb
1,Polen_1,Polen,,0.883673,wiki Deklinierte_Form,Pole,Pole,NN,N,9.124239,pole
43,mich,mich,,0.795918,wiki Deklinierte_Form,ich,ich,PPER,P,320.846527,ich
479,fax,fax,,0.528571,wiki Konjugierte_Form Verb,faxen,faxen,VVFIN,V,6.946407,faxen
1335,studierte,studierte,,1.702801,wiki Deklinierte_Form,studiert,studiert,ADJA,A,0.463265,studiert
188286,α-Zerfällen,α-Zerfällen,,0.434694,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,6.372716,α-zerfall
202911,-logien,-logien,,0.716327,wiki Deklinierte_Form,-logie,-logie,ZFIX,Z,9.939098,-logie
454818,umzuschlagen,umzuschlagen,,0.430612,wiki Erweiterter_Infinitiv Verb,umschlagen,umschlagen,VVFIN,V,11.375153,umschlagen
476083,böllernde,böllernde,,1.699147,wiki Deklinierte_Form,böllernd,böllernd,ADJA,A,0.330612,böllernd
524690,durchscheuer,durchscheuer,,0.751020,wiki Konjugierte_Form Verb,durchscheuern,durchscheuern,VVFIN,V,9.350792,durchscheuern


time: 6.64 s


In [47]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

35 rows


time: 1.31 s


In [48]:
# anschauen
#pak.analyse_freqs(translate_lex,['member','lemma_tag','lex'])

time: 52.8 ms


## Partizipien ==========
Für fast jedes Partizip gibt es zwei Datensätze: <br> 
1) in wiktionary_merkmal.data: Die Partizip-Flexion des Verbs. merkmal ist 'Ptz1' oder 'Ptz2'.<br> 
2) in wiktionary_lemma.lemma:  Als Adjektiv mit member 'Partizip'. Es gibt ca. 320 Ausnahmen, nicht alle Partizipien sind auch Adjektive.

In [49]:
# Falsche Partizipien löschen
falsche_partizipien = ['sehen','mögen','hören','sollen','können','lassen']
mask1 = wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask2 = wiktionary_merkmal.data.isin(falsche_partizipien)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,6)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'Falsche_Partizipien')
#wiktionary_merkmal[mask]

check_mask:  6 rows
Move 6 rows from 4666420
time: 4.11 s


In [50]:
# verben_ptz1: Alle Ptz1-Merkmale der Verben
spalten = ['lemma_id','data_id','data','score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz1'
verben_ptz1 = wiktionary_merkmal[mask][spalten]
verben_ptz1 = pak.update_col(df_to=verben_ptz1, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz1 = pak.move_cols(       verben_ptz1,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz1, suche_debug)
if len(a) == 0:
    a = verben_ptz1.sample(5)
a

,lemma_id,lemma,data_id,data,score,merkmal
1729462,missraten,missraten,missratend,missratend,4.472594,Ptz1
1751011,gefriertrocknen,gefriertrocknen,gefriertrocknend,gefriertrocknend,14.422655,Ptz1


time: 508 ms


In [51]:
# Kein leeres data_id
mask = verben_ptz1.data_id.isnull()  |  (verben_ptz1.data_id == '')
check_mask(verben_ptz1,mask,0)

check_mask:  0 rows
time: 33.5 ms


In [52]:
# alle enden mit 'nd'
mask = ~verben_ptz1.data.str.endswith('nd')
check_mask(verben_ptz1,mask,0)
#verben_ptz[mask]

check_mask:  0 rows
time: 39.7 ms


In [53]:
# verben_ptz2: Alle Ptz2-Merkmale der Verben
spalten = ['lemma_id','data_id','data','score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz2'
verben_ptz2 = wiktionary_merkmal[mask][spalten]
verben_ptz2 = pak.update_col(df_to=verben_ptz2, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz2 = pak.move_cols(       verben_ptz2,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz2, suche_debug)
if len(a) == 0:
    a = verben_ptz2.sample(5)
a

,lemma_id,lemma,data_id,data,score,merkmal
3000391,missraten,missraten,missraten_MADJ,missraten,4.472594,Ptz2
3000784,gefriertrocknen,gefriertrocknen,gefriergetrocknet,gefriergetrocknet,14.422655,Ptz2
3001274,missraten,missraten,mißraten_MADJ,mißraten,4.472594,Ptz2


time: 513 ms


In [54]:
# Kein leeres data_id
mask = verben_ptz2.data_id.isnull()  |  (verben_ptz2.data_id == '')
check_mask(verben_ptz2,mask,0)

check_mask:  0 rows
time: 37.5 ms


In [55]:
# adjektive_ptz1: Adjektive mit Ptz1-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz1')
mask = mask1  &  mask2 
adjektive_ptz1 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz1, suche_debug)
if len(a) == 0:
    a = adjektive_ptz1.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
127895,gefriertrocknend,gefriertrocknend,0.467347,ADJA,Ptz1,gefriertrocknen
140463,missratend,missratend,0.579592,ADJA,Ptz1,missraten


time: 119 ms


In [56]:
# Kein leeres data_id
# data_id verweist auf das Verb
mask = adjektive_ptz1.data_id.isnull()  |  (adjektive_ptz1.data_id == '')
check_mask(adjektive_ptz1,mask,0)

check_mask:  0 rows
time: 39.1 ms


In [57]:
# adjektive_ptz2: Adjektive mit Ptz2-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2 
adjektive_ptz2 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz2, suche_debug)
if len(a) == 0:
    a = adjektive_ptz2.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
2054,gefriergetrocknet,gefriergetrocknet,4.307552,ADJA,Ptz2 noSteig manuell,gefriertrocknen
140462,missraten_MADJ,missraten,0.500000,ADJA,Ptz2 manuell,missraten
140820,mißraten_MADJ,mißraten,0.500000,ADJA,Ptz2 manuell,missraten


time: 122 ms


In [58]:
# Kein leeres data_id
# data_id verweist auf das Verb
# Das sind Kandidaten für die Aufnahme in lexeme_manuell, Reiter Partizipien
nöö = ['befangen','betreten','durchwachsen','erfahren','erlesen','erwachsen','wollen']
mask1 = adjektive_ptz2.data_id.isnull()  |  (adjektive_ptz2.data_id == '')
mask2 = ~adjektive_ptz2.lemma.isin(nöö)
mask = mask1  &  mask2
#grid(adjektive_ptz2,mask)
check_mask(adjektive_ptz2,mask,0)

check_mask:  0 rows
time: 35.6 ms


In [59]:
#for p in adjektive_ptz2[mask].lemma:
#    print(p)

time: 33.3 ms


In [60]:
# adjektive_noptz: Adjektive ohne Ptz-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 =  wiktionary_lemma.tagZ == 'ADJ'
mask2 = ~wiktionary_lemma.member.str.contains('Ptz1')
mask3 = ~wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2  &  mask3  
adjektive_noptz = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_noptz, suche_debug)
if len(a) == 0:
    a = adjektive_noptz.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
131177,gleichaltrig,gleichaltrig,6.496955,ADJA,noSteig,
109403,akustisch,akustisch,7.816078,ADJA,noSteig,
147331,schau,schau,6.800243,ADJA,,
154235,vacat,vacat,4.143727,ADJA,noSteig,
149756,steiermärkisch,steiermärkisch,4.794154,ADJA,noSteig,


time: 195 ms


In [61]:
# Nur leeres data_id
mask = adjektive_noptz.data_id.isnull()  |  (adjektive_noptz.data_id == '')
#grid(adjektive_noptz,~mask)
check_mask(adjektive_noptz,~mask,0)

check_mask:  0 rows
time: 41.3 ms


In [62]:
# Das sind vermutlich eigentlich Partizipien 
mask1 = adjektive_noptz.lemma.str.endswith('nd')
mask2 = ~adjektive_noptz.lemma.str.contains(' ')
mask = mask1  &  mask2
kandidaten = adjektive_noptz[mask].copy()

# Kandidaten aufbereiten
nöö = ['dutzend','flektierend','frappierend','spannend']
kandidaten['verb'] = kandidaten.lemma.str[:-1]
mask =  wiktionary_lemma.tagZZ == 'V'
kandidaten = pak.update_col(kandidaten, 
                            wiktionary_lemma[mask],
                            left_on=['verb'],
                            right_on=['lemma'],
                            col='lemma_id',
                            col_rename='data_id',
                            func='min'
                           )

# und filtern
maskA = kandidaten.data_id.isnull()  |  (kandidaten.data_id == '')
maskB = kandidaten.lemma.isin(nöö)
mask = maskA  |  maskB
kandidaten = pak.drop_rows(kandidaten,mask)

spalten = ['lemma','verb']
grid(kandidaten[spalten])

update_col: func min applied, 380 records less!
update_col: 4 cells written into existing column
Delete 267 rows from 267
No rows
time: 1.42 s


### Gibt es Verben ohne Partizip-Lexeme?
* ggf. Partizipen manuell nachtragen

In [63]:
# Verben ohne Ptz1
spalten = ['lemma_id','lemma','score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz1.lemma)
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,26)
verben_ohne_ptz1 = wiktionary_lemma[mask][spalten]
#verben_ohne_ptz1

check_mask:  29 rows
time: 68.5 ms


In [64]:
# Kandidaten für die manuelle Aufnahme
# - entweder als Ptz1-Lexem in lexeme_manuell (Reiter Partizipien)
# - oder als lemma in die löschliste

nöö = ['angedeihen','erstinken','gegenanargumentieren','grundsanieren','interdependieren','mittagessen','umfragen','urformen','voranmelden','zusammenreiten']
mask1 = verben_ohne_ptz1.lemma.str.endswith('en')
mask2 = ~verben_ohne_ptz1.lemma.isin(nöö)
mask = mask1  &  mask2
kandidaten = verben_ohne_ptz1[mask][['lemma']]
kandidaten = kandidaten.drop_duplicates().sort_values('lemma')
kandidaten['lex'] = kandidaten.lemma + 'd'
kandidaten = pak.move_cols(kandidaten,'lex')
kandidaten

,lex,lemma
110667,anguckend,angucken
111223,anstaunend,anstaunen
127396,geborend,geboren


time: 51.1 ms


In [65]:
# Verben ohne Ptz2
nöö = ['Schlange stehen', 'gehn', 'stehn', 'umsehn','vergleiche']
spalten = ['lemma_id','lemma','score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz2.lemma)
mask3 = ~wiktionary_lemma.lemma.isin(nöö)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,0,11)
verben_ohne_ptz2 = wiktionary_lemma[mask][spalten]
verben_ohne_ptz2

check_mask:  5 rows


,lemma_id,lemma,score,tag,member
110667,angucken_M,angucken,0.45,VVFIN,manuell
111223,anstaunen_M,anstaunen,0.45,VVFIN,manuell
112557,aufrütteln_M,aufrütteln,0.45,VVFIN,manuell
127396,geboren_M,geboren,0.45,VVFIN,manuell
133187,herumpimpeln_M,herumpimpeln,0.45,VVFIN,manuell


time: 89 ms


In [66]:
del verben_ohne_ptz1
del verben_ohne_ptz2

time: 40 ms


In [67]:
#mask = wiktionary_merkmal.lemma_id == 'betreffen'
#wiktionary_merkmal[mask]

time: 40.7 ms


In [68]:
#mask = wiktionary_lemma.lemma == 'betreffend'
#grid(wiktionary_lemma[mask])

time: 34.2 ms


### Partizipien und Adjektive vergleichen

In [69]:
# merge für Ptz1
a = pd.merge(verben_ptz1, adjektive_ptz1, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))

time: 82.5 ms


In [70]:
# Übersicht
assert a.shape[0] > 13000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,13900,100.0,##############################################...
1,left_only,0,0.0,
2,right_only,0,0.0,


time: 48 ms


In [71]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,20)
a[mask][['lemma_a']]

check_mask:  0 rows


,lemma_a


time: 40.8 ms


In [72]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,1)
#a[mask] 

check_mask:  0 rows
time: 30.8 ms


In [73]:
# merge für Ptz2
a = pd.merge(verben_ptz2, adjektive_ptz2, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))
a = a.drop_duplicates()

time: 128 ms


In [74]:
# Übersicht
assert a.shape[0] > 14000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,13905,98.9,#################################################
1,left_only,159,1.1,
2,right_only,0,0.0,


time: 51.2 ms


In [75]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,30)
a[mask][['lemma_id_a','lemma_a','tag','member']]

check_mask:  0 rows


,lemma_id_a,lemma_a,tag,member


time: 41.2 ms


In [76]:
# entkennzeichnen
entkennzeichnen = a[mask].lemma_id_a
mask_L = wiktionary_lemma.lemma_id.isin(entkennzeichnen)
wiktionary_lemma.loc[mask_L,'member'] = pak.remove_str(wiktionary_lemma[mask_L].member, 'Ptz2')
wiktionary_lemma[mask_L]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag


time: 63.7 ms


In [77]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,350)
fehlende_adjektive = a[mask]
fehlende_adjektive = pak.update_col(df_to=fehlende_adjektive, df_from=adjektive_noptz, left_on='lemma_v', right_on='lemma',col='lemma_id',col_rename='adj_id')
#fehlende_adjektive

check_mask:  159 rows
update_col: 7 cells written into new column
time: 70.3 ms


In [78]:
# Das Partizipialadjektiv fehlt. Das kann zwei Ursachen haben:
# (1) Es gibt ein Adjektiv, das ist aber nicht als Partizipialadjektiv markiert
# (2) Es gibt kein Adjektiv.
# Zuerst kümmern wir uns um Fall (1).
# Die in adj_id genannten Adjektive müssen noch als Partizipialadjektiv markiert werden
mask_a = ~fehlende_adjektive.adj_id.isnull()
check_mask(fehlende_adjektive,mask_a,0,200)
fehlende_adjektive[mask_a]

check_mask:  7 rows


,lemma_id_v,lemma_v,data_id_v,data,score_v,merkmal,lemma_id_a,lemma_a,score_a,tag,member,data_id_a,_merge,adj_id
13849,durchwachsen_2,durchwachsen,durchwachsen,durchwachsen,1.479903,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,durchwachsen
13862,erfahren,erfahren,erfahren_1,erfahren,12.705237,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erfahren_1
13881,betreten_1,betreten,betreten,betreten,4.283800,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,betreten
13898,befangen_1,befangen,befangen,befangen,3.585871,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,befangen
13908,erwachsen_1,erwachsen,erwachsen,erwachsen,3.584653,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erwachsen
13942,beschlagen,beschlagen,beschlagen_1,beschlagen,9.160780,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,beschlagen_1
13974,erlesen_1,erlesen,erlesen,erlesen,2.080390,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erlesen


time: 64.7 ms


In [79]:
# markieren
mask = wiktionary_lemma.lemma_id.isin(fehlende_adjektive[mask_a].adj_id)
check_mask(wiktionary_lemma,mask,0,150)
wiktionary_lemma.loc[mask,'member'] += ' Ptz2'
#wiktionary_lemma[mask]

check_mask:  7 rows
time: 59.4 ms


In [80]:
# Fall (2)
# Dies sind Kandidaten dafür, manuell aufgenommen zu werden.
# Und zwar als Partizipien (Ptz2) in lexeme_manuell, dann werden die Adjektive automatisch erstellt und passend verlinkt.
# Das geschieht in d01_tag_und_member.
#

mask_a = fehlende_adjektive.adj_id.isnull()
kandidaten = fehlende_adjektive[mask_a][['data']].drop_duplicates().sort_values('data')

nöö = ['abbegeben','abbehalten','abbekommen','abgetan','anbekommen','angetan','aufbekommen','aufgetan','ausgetan','begeben','behalten','beibehalten','bekommen',
       'benachrangt','bestoßen','betan','betragen','dabehalten','dargetan','dazugetan','dickgetan','dortbehalten',
       'durchbekommen','durchhauen','durchmarschiert','entraten','entschlagen','erfangen','ermessen','erschlafen','erschollen',
       'ersehen','ertragen','fallenlassen','fortwollen','ge- -zt','gegenanargumentiert','geheimgetan',
       'gemecht','genuggetan','geraidet','getan','gleichgetan','großgetan','gutgetan','hartgetan','hervorgetan','hinbekommen','hindurchmarschieren',
       'hineinbegeben','hineingetan','hinterlaufen','hinunterbekommen','hinzugetan','hinüberrannte','hängen lassen','leichtgetan','leidgetan',
       'lieb behalten','missfallen','mitbekommen','mitwollen','nachbekommen','nachgeraten','nachgetan','notgetan','schwergetan','schöngetan','stehenlassen',
       'umeinandergehetzt','umeinandergelegen','umeinandergestanden','umgefragt','umgetan','umtreten','unterbekommen','unterfangen','unterwaschen',
       'verhebt','verladen','verschleißt','versehen','vertan','vertragen',
       'vorgetan','weggetan','wegwollen','weh getan','wehgetan','widerfahren','widerraten','wiederwollen','wohlgetan','zubekommen','zugetan',
       'zugutegetan','zurechtbekommen','zurückbegeben','zurückbekommen','zurückerhalten','zurückgetan','zurückwollen','zusammenbekommen','zusammengetan','zuvorgetan',
       'überlassen','überschlafen','betroffen','geschehen','umschlagen']
print('len nöö =',len(nöö))
mask1 = ~kandidaten.data.str.startswith('-')
mask2 = ~kandidaten.data.isin(nöö)
mask = mask1  &  mask2  
check_mask(kandidaten,mask, 0,200)
grid(kandidaten,mask)

len nöö = 110
check_mask:  0 rows
No rows, mask filters them all away
time: 35 ms


### Aufräumen

In [81]:
del verben_ptz1
del verben_ptz2
del adjektive_ptz1
del adjektive_ptz2
del adjektive_noptz

time: 32.9 ms


## wiktionary_lemma ==========

In [82]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows
time: 89.9 ms


In [83]:
# score page_part 
# -1 wird weiter unten behandelt
#  0 wird gepushed
#  1 bleibt gleich
#  2..n wird downgescored
mask = wiktionary_lemma.page_part == 0
wiktionary_lemma.loc[mask,'score'] += 0.3
wiktionary_lemma.loc[mask,'score'] *= 2.0

mask = wiktionary_lemma.page_part > 1
wiktionary_lemma.loc[mask,'score'] = wiktionary_lemma[mask].score / wiktionary_lemma[mask].page_part.astype(float)

time: 56.3 ms


In [84]:
wiktionary_lemma

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
0,24/7,583759,0,24/7,24/7,True,1,ADJA,ADJA,xa6xnLR8Bn,ADJ,A,manuell ergänzt Abkürzung,,7.112668,3,24 -- 7,CARD $( CARD
1,2D,4928,0,2D,2d,True,1,ADJA,ADJA,yNNcpEOlDA,ADJ,A,manuell ergänzt Abkürzung,,10.540195,1,2D,NE
2,3D,5222,0,3D,3d,True,1,ADJA,ADJA,i2PK7DDSi6,ADJ,A,manuell ergänzt Abkürzung,,14.274665,1,3D,NN
3,4D,403012,0,4D,4d,True,1,ADJA,ADJA,G648MEHkhG,ADJ,A,manuell ergänzt Abkürzung,,11.095615,1,4D,ADV
4,5G,974056,0,5G,5g,True,1,NN,NN,3JL8hbrOZS,NOM,N,Abkürzung manuell ergänzt,,6.308770,2,5 G,CARD NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165341,Œuvre,33691,0,Œuvre,œuvre,True,1,NN,NN,UFa8oxkzRs,NOM,N,,,41.131058,1,Œuvre,NE
165342,α-Zerfall,460277,0,α-Zerfall,α-zerfall,False,1,NN,NN,0bREezuiTj,NOM,N,,,13.345433,1,α-Zerfall,NN
165343,μC,191234,0,μC,μc,False,1,TODO,TODO,wVCpr3hvMC,X_todo,X,Abkürzung,,22.290621,1,μC,FM
165344,μm,239016,0,μm,μm,False,1,TODO,TODO,7XcQmdnlAC,X_todo,X,Abkürzung,,8.500122,1,μm,NE


time: 55.8 ms


In [85]:
# wiktionary_lemma score ergänzen
# Der score in translate_lex spiegelt die Länge des Wiktionary_Eintrags des Lexem wieder. Der längste wird zum Lemma draufaddiert.
wiktionary_lemma['score']     = wiktionary_lemma.score.fillna(0) 
translate_lex['data_score']   = translate_lex.data_score.fillna(0) 
wiktionary_lemma = pak.update_col(wiktionary_lemma, translate_lex, on=['lemma_id'], col='data_score', col_rename='score_add', func='max')
wiktionary_lemma['score_add'] = wiktionary_lemma.score_add.fillna(0) 

wiktionary_lemma['score']     = wiktionary_lemma.score  +  (wiktionary_lemma.score_add / 2.0)
wiktionary_lemma = pak.drop_cols(wiktionary_lemma, 'score_add')

update_col: func max applied, 508464 records less!
update_col: 136321 cells written into new column
time: 1.28 s


In [86]:
# Dups?
mask = wiktionary_lemma.duplicated(subset=['lemma_id'])
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows
time: 51.3 ms


In [87]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

time: 322 ms


In [88]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  287 rows
check_mask:  0 rows
time: 4.15 s


In [89]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
2054,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,V2Xzj6L3xb,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,10.341779,1,friergetrocknen,VVPP
2055,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,IUwOEm6Eis,VER,V,manuell,,31.451888,1,gefriertrockn,VVPP
2194,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,IbKmXDIZ46,VER,V,manuell,,11.045676,1,Missrat,VVFIN
127895,gefriertrocknend,1054319,0,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,ksSiX4o537,ADJ,A,Ptz1,gefriertrocknen,2.650406,1,gefriertrocknend,ADJD
140462,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,PEiiiSZpRn,ADJ,A,Ptz2 manuell,missraten,1.630329,1,Missrat,VVFIN
140463,missratend,446280,0,missratend,missratend,False,1,ADJA,ADJA,VzLpPPn6D5,ADJ,A,Ptz1,missraten,2.882204,1,Missratend,ADJD
140820,mißraten_MADJ,-1,-1,mißraten,mißraten,False,1,ADJA,ADJA,zOqB9HbDOi,ADJ,A,Ptz2 manuell,missraten,0.500000,1,mißraten,VVPP


time: 180 ms


## Abkürzungen ==========
* abspalten
* belöschen
* korrigieren
* variieren
* pflegen

In [90]:
geschütze_abkürzungen = ['H2O','DEU','deu','RAF','UNO','CDU','SPD','FDP',
                         'SED','UTC','ORF','RKI',
                         'Mo','Di','Mi','Do','Fr','Sa','So',  'MO','DI','MI','DO','FR','SA','SO',  'Mo.','Di.','Mi.','Do.','Fr.','Sa.','So.', 'NS','US','VR','IR','UV','9/11','OP','KI','AI','IT','IS',
                         'Dax','FDJ','MDR','USB','2D','3D','OEM','VW',
                         'u.','ca.','ev.','PKK','USK','dpi','SUV', 'Fa.','od.','u.','CO2','SO2','VR China','Meck-Pomm','Hbf',
]

time: 35.2 ms


In [91]:
bsp_abkürzungen_kurz = ['altisländ.', 'EU','Krh', 'Krh.', 'Krhs', 'Krhs.', 'Krkhs', 'Krkhs.']
bsp_abkürzungen_lang = ['altisländisch', 'Europäische Union','Krankenhaus']

time: 38.8 ms


In [92]:
# wiktionary_lemma kann sowohl die Langform als auch die Kurzform als eigenständiges lemma enthalten.
# Eine Kurzform ist dabei mit member 'Abkürzung' gekennzeichnet.
# TODO: Beide müssen nicht unbedingt den gleichen tag haben, siehe 'EU' vs. 'Europäische Union': NN / NE
# TODO: Nicht jede Kurzform hat einen solchen Datensatz.

search_str(wiktionary_lemma, bsp_abkürzungen_kurz + bsp_abkürzungen_lang)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
24630,EU,2574,0,EU,eu,True,1,NN,NN,GKtHDu6krk,NOM,N,Abkürzung noPl,,30.300365,1,EU,NE
28691,Europäische_Union,12472,0,Europäische Union,europäische union,True,2,NE,NE,Fan244HpYd,NAM,N,adjNN Wortverbindung adjNN,,19.637679,2,europäisch Union,ADJA NN
53107,Krankenhaus,724,0,Krankenhaus,krankenhaus,True,1,NN,NN,qCn17wdGxA,NOM,N,,,456.211774,1,Krankenhaus,NN
53449,Krh.,884635,0,Krh.,krh.,True,1,TODO,TODO,EAyhhORW0z,X_todo,X,Abkürzung,,4.860658,2,Krh --,NN $.
53450,Krhs.,884636,0,Krhs.,krhs.,True,1,TODO,TODO,ljMIqSSstz,X_todo,X,Abkürzung,,5.211449,2,Krhs --,NE $.
53748,Krkhs.,884637,0,Krkhs.,krkhs.,True,1,TODO,TODO,bfJ0sUGp3G,X_todo,X,Abkürzung,,4.882582,2,Krkhs --,NE $.
109677,altisländ.,21375,0,altisländ.,altisländ.,False,1,TODO,TODO,7OIQ5Fe8xf,X_todo,X,Abkürzung,,27.618271,2,altisländ --,VVFIN $.
109678,altisländisch,608920,0,altisländisch,altisländisch,False,1,ADJA,ADJA,FrCZAHiXia,ADJ,A,noSteig,,16.113399,1,altisländisch,ADJD


time: 189 ms


In [93]:
# wiktionary_merkmal enthält für Kurzformen ein Merkmal 'def', das die Langform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_kurz)
mask2 = wiktionary_merkmal.merkmal == 'def'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
51621,EU,def,Europäische_Union,Europäische Union,,NN,1,,14.850183,False
191494,altisländ.,def,altisländisch,altisländisch,,TODO,1,,13.509135,False
1432916,Krh.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.130329,False
1432917,Krhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.305725,False
1432918,Krkhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.141291,False


time: 687 ms


In [94]:
# wiktionary_merkmal enthält für Langformen Merkmale 'abk', das eine Kurzform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_lang)
mask2 = wiktionary_merkmal.merkmal == 'abk'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
18593,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,227.583435,True
18594,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,227.583435,True
18595,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,227.583435,True
1088835,altisländisch,abk,altisländisch~non,non,,ADJA,1,,7.213155,True
1088836,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,7.213155,True


time: 562 ms


### wiktionary_merkmal_abk abspalten

In [95]:
#suche_debug = ['au','ei','hm','qm','ui','mg','km','cm','mm','dm','ft','nm','pt','m²','m³','kg','lb','db']

time: 34.2 ms


In [96]:
# abk-Merkmale abspalten
mask = (wiktionary_merkmal.merkmal == 'abk')
wiktionary_merkmal, wiktionary_merkmal_abk = pak.move_rows( wiktionary_merkmal, mask, msg=None )
wiktionary_merkmal_abk = wiktionary_merkmal_abk.sort_values(['score','data'], ascending=[False,True])

Move 5051 rows from 4666414
time: 3.31 s


In [97]:
#TODO: Direkt durch die Kurzformen in wiktionary_lemma ergänzen?

time: 54.5 ms


In [98]:
# suche_debug wiktionary_merkmal_abk
print(bsp_abkürzungen_lang)
search_str(wiktionary_merkmal_abk, bsp_abkürzungen_lang)

['altisländisch', 'Europäische Union', 'Krankenhaus']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
18593,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,227.583435,True
18594,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,227.583435,True
18595,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,227.583435,True
1088836,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,7.213155,True
1088835,altisländisch,abk,altisländisch~non,non,,ADJA,1,,7.213155,True


time: 71 ms


In [99]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


time: 56.7 ms


### wiktionary_merkmal_abk belöschen

In [100]:
# belöschen: Manuelle Löschungen aus data oder lemma_id
löschmich =  ['Objekt','ECO-Codes','Bibo','Kilom','Land','Basche','DV-Vertrag','Igel','Dolle','Ampere','Baum',
              'Rat','Reibe','Heinrich_1','Diastema mediale','Divisionär','Soda-Zitron','Antillen-Gulden',
              "'H'B","'H'GW","'H'H","'H'L","'H'ST","'H'RO","'H'WI",'gel','gel.','Reg.','Imp.','Reg','Imp','AUR',
             ]

mask = wiktionary_merkmal_abk.data.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 55)

msg1 = 'löschmich_data'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg1)

mask = wiktionary_merkmal_abk.lemma_id.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 65)

msg2 = 'löschmich_id'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg2)

# anschauen
spalten = ['data', 'meta', 'lemma_id',  'lemma_tag',   'merkmal', 'msg']
mask = wiktionary_merkmal_trash.msg.isin([msg1, msg2])
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  48 rows
Move 48 rows from 5051
check_mask:  65 rows
Move 65 rows from 5003
113 rows out of 128


time: 108 ms


In [101]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 6                    # zu lang
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask5 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask6 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask7 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal_abk,mask, 331)

msg = 'zulang_single'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  331 rows
Move 331 rows from 4938
331 rows out of 459


time: 177 ms


In [102]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 15                   # zu lang
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 11)

msg = 'zulang'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  11 rows
Move 11 rows from 4607
11 rows out of 470


time: 159 ms


In [103]:
# belöschen: Namen
mask1 =  wiktionary_merkmal_abk.lemma_tag == 'NE'                  # ist Name
mask2 = ~wiktionary_merkmal_abk.data.str.isupper()                 # nicht nur groß
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal_abk,mask, 330)

msg = 'Namen'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  329 rows
Move 329 rows from 4596
329 rows out of 799


time: 173 ms


In [104]:
# belöschen: Nur Ziffern
mask1 = wiktionary_merkmal_abk.data.str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 20)

msg = 'nur_Ziffern'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  20 rows
Move 20 rows from 4267
20 rows out of 819


time: 116 ms


In [105]:
# belöschen: Endet mit Ziffer
mask1 =  wiktionary_merkmal_abk.data.str[-1].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(('²','³','₂')) 
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(wiktionary_merkmal_abk,mask, 162)

msg = 'Endet_mit_Ziffer'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  162 rows
Move 162 rows from 4247
162 rows out of 981


time: 116 ms


In [106]:
# belöschen: Bibel
mask1 =  wiktionary_merkmal_abk.lemma_id.str[0].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 11)

msg = 'Bibel'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  11 rows
Move 11 rows from 4085
11 rows out of 992


time: 100 ms


In [107]:
# belöschen: meta
löschmich_meta = ['Postleitzahl','Elementsymbol','seltener','selten','Ordenskürzel','Österreich','Makrosprache',
                  'Kirmanjki','Schweiz','ISO','Kfz','Internet','ISO-3166','Kfz-Kennzeichen','fachsprachlich','Dreibuchstabencode',
                  'international','Deutschland','ISO-4217-Code','ISO-639-3','chemisches','Zentral-Quiche','Komi',
                  'Kraftfahrzeugkennzeichen','englisch','veraltet','Dimli','Shimaore','bisweilen',
                  'Katalogbezeichnungen','ISO-639-1','inoffiziell','olympisch','chemische','Zusammensetzungen',
                  'Netzjargon','ISO-4217','bundesdeutsch','Währungscodes','UN LOCODE','römische','schweizerisch',
                  'Währungscode','Römische','bedeutungsgleiche','Summenformel',
                  'Rumänien','Lüftungstechnik','Lebensmittelfarbstoff','internationaler','volkstümlich',
                  'Maya-Sprachen','ISO-3166-1','ISO-3166-Code','üblich','wissenschaftlich','ehem',
                  'Polizeijargon','IOC-Ländercode','Moldawien','taktisches','postalisch','Abkürzung','Römisches',
                  'fachsprachlich schriftsprachlich','Schifffahrt','Codierung','SIL-Code']
mask1 =  wiktionary_merkmal_abk.meta.isin(löschmich_meta)
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal_abk,mask, 1030)

msg = 'meta'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = wiktionary_merkmal_trash[mask].sort_values(['meta','score'],ascending=[True,False])
grid(a)

check_mask:  1030 rows
Move 1030 rows from 4074
1030 rows


time: 146 ms


In [108]:
# Übriggebliebene nach meta
a = pak.analyse_freqs( wiktionary_merkmal_abk, ['meta','lemma_tag','data'] )
grid(a)

15 rows


time: 135 ms


In [109]:
# meta ist jetzt sinnlos, bis auf 'manuell'
mask =  wiktionary_merkmal_abk.meta != 'manuell'
wiktionary_merkmal_abk.loc[mask,'meta'] = ''

time: 48.4 ms


In [110]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


time: 69.8 ms


### Analysieren

In [111]:
# Gelöschte überprüfen, aus suche_debug?
print(suche_debug)
search_str(wiktionary_merkmal_trash, suche_debug)

['gefriergetrocknet', 'gefriertrocknen', 'missraten']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex,msg


time: 62.9 ms


In [112]:
# Gelöschte überprüfen, aus lexeme_manuell?
mask1 = wiktionary_merkmal_trash.lemma_id.str.endswith('_M') 
mask2 = ~wiktionary_merkmal_trash.msg.isin(['dup'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_trash, mask, 0)
#wiktionary_merkmal_trash[mask]

check_mask:  0 rows
time: 41.3 ms


In [113]:
# Gelöschte überprüfen, aus geschütze_abkürzungen?
mask = wiktionary_merkmal_trash.data.isin(geschütze_abkürzungen)
check_mask(wiktionary_merkmal_trash, mask, 0)

check_mask:  0 rows
time: 46.1 ms


In [114]:
# Gelöschte anschauen
a = pak.analyse_freqs( wiktionary_merkmal_trash, ['msg','lemma_tag','data'] )
grid(a)

11 rows


time: 157 ms


In [115]:
# Gelöschte mit dem höchsten score
mask = ~wiktionary_merkmal_trash.msg.isin(['dup'])
grid(wiktionary_merkmal_trash[mask].sort_values('score',ascending=False))

2022 rows


time: 134 ms


In [116]:
# Verbliebene nach tag
pak.analyse_freqs(wiktionary_merkmal_abk,['lemma_tag','meta','data'])

,lemma_tag,lemma_tag_count,lemma_tag_percent,meta,meta_count,data,data_count
0,NN,2051,67.4,"[, manuell]","[1962, 89]","[Str, Pfl, Konv, Ins, Gen, Adj, Ztr, Konj, Pat...","[16, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4,..."
1,ADJA,605,19.9,"[, manuell]","[597, 8]","[ind, ged, get, unv, afr, imp, geh, med, gebr,...","[7, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, ..."
2,NE,125,4.1,"[, manuell]","[124, 1]","[LUX, RUS, BAD, MKD, ABG, GKW, HAL, DEU, CHN, ...","[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,ADV,102,3.4,"[, manuell]","[98, 4]","[bes, pass, insb, insbes, ebd, östl, dto, sod,...","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,TODO,33,1.1,,33,"[Chemie, ad lib, s. A., s.A., sel. A., sel.A.,...","[4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,KOMBI,26,0.9,"[manuell, ]","[18, 8]","[i. Allg., u. a., u.a., i. A., z. Z., z. Zt., ...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,VVFIN,25,0.8,"[, manuell]","[23, 2]","[erk, inf, imp, gez, abgez, abn, rad, zit, her...","[4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,APPR,19,0.6,"[, manuell]","[18, 1]","[entg, innerh, bez, bezgl, bzgl, mod, betr, hi...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,,10,0.3,,10,"[dies, nen, inf, jdm, jmdm, jdn, jmdn, jds, jmds]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,KON,9,0.3,"[manuell, ]","[5, 4]","[&, +, u., bezw, bzw, bzw., resp, rsp, d.h.]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"


time: 146 ms


In [117]:
# suche_debug wiktionary_merkmal_abk
print(suche_debug)
search_str(wiktionary_merkmal_abk, suche_debug)

['gefriergetrocknet', 'gefriertrocknen', 'missraten']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


time: 66.8 ms


In [118]:
# suche_debug wiktionary_merkmal
#mask = wiktionary_merkmal.merkmal.isin(['gm'])
#a = search_str(wiktionary_merkmal[mask], suche_debug)
#grid(a.sort_values('lemma_id'))

time: 62.7 ms


### wiktionary_merkmal_abk korrigieren

In [119]:
# Einzelne Worte löschen
r = ['(grad)',]
wiktionary_merkmal_abk['data'] = pak.remove_str(wiktionary_merkmal_abk.data, r)

time: 70.5 ms


In [120]:
# korrigieren: Punkt innen ohne Punkt hinten
# Das ist eigentlich schon weiter oben passiert
endet_nicht_mit = ( '.', 'Tag', 'Ort',)
mask1 =  wiktionary_merkmal_abk.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(endet_nicht_mit)
mask3 = (wiktionary_merkmal_abk.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask4 =  wiktionary_merkmal_abk.data.str.len() > 2
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal_abk, mask, 0,136 )
wiktionary_merkmal_abk.loc[mask,'data'] += '.'
grid(wiktionary_merkmal_abk,mask)

check_mask:  2 rows
2 rows out of 3044


time: 164 ms


In [121]:
# korrigieren: Punkt hinten ergänzen
ausnahmen =  [ 'Admin','Schirm','Kappe','Symbol','Potenz','Schiri','Info','Bus','Kiga','Hof','Teen','Teenie','Teeny','Nitro','Steno','Perso','Navi',]
ausnahmen += [ 'Mayo','Hasch','Britta','Tusse','Tussi','Jäger','Reha','Latte','Pkw','Escape','Spasti','Proll','Prolo','Traufe','Qbit','Qubit',]
ausnahmen += [ 'Rose','Kino','Alster','Hafen','Psycho','Dame','Kelle','Kopie','FeWo','Chemie','Knobi','Makro','Majo','Tief']
mask1 = ~wiktionary_merkmal_abk.data.isin(ausnahmen)
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')                                                       # Kein Bindestrich
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')                                                       # kein Space
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)                                          # kein Punkt
mask5 = ~wiktionary_merkmal_abk.meta.isin(['ISO','kurz','umgangssprachlich','Netzjargon'])
mask6 = (wiktionary_merkmal_abk.data.str.len() > 2)   |   wiktionary_merkmal_abk.data.isin(['Dr','No'])
mask7 =  wiktionary_merkmal_abk.data.str.contains('^[A-Za-zöüäÖÄÜ]')                                         # startswith Buchstabe
mask8 =  wiktionary_merkmal_abk.data.str.count('[A-ZÜÖÄ]')  <=    1                                          # Nicht mehr als 1 Großbuchstabe
mask9 = ~wiktionary_merkmal_abk.data.str.contains('[^A-Za-zöüäÖÄÜß]')                                        # irgendwelche Sonderzeichen

mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7  &  mask8  &  mask9
check_mask( wiktionary_merkmal_abk, mask, 1799 )

wiktionary_merkmal_abk.loc[mask,'data'] += '.'  
grid(wiktionary_merkmal_abk,mask)

check_mask:  1795 rows
1795 rows out of 3044


time: 200 ms


In [122]:
# variieren: Punkt mit Space
#mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask = wiktionary_merkmal_abk.data.str.count('\.') > 1
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'. ',   '-':''}
df['data'] = pak.replace_str(df.data, r).str.strip()
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

159 rows added, now a total of 3203
time: 52.7 ms


In [123]:
# variieren: Bindestriche weglassen
mask1 =  wiktionary_merkmal_abk.data.str.contains('-')
mask2 = ~wiktionary_merkmal_abk.data.str.startswith('-')
mask3 = ~wiktionary_merkmal_abk.data.str.endswith('-')
mask = mask1  &  mask2  &  mask3
df = wiktionary_merkmal_abk[mask].copy()
r = {'-':' ', '  ':' '}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

117 rows added, now a total of 3320
time: 65.6 ms


In [124]:
# variieren: Punkte weglassen
mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'',}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

2220 rows added, now a total of 5540
time: 77.2 ms


In [125]:
# variieren: Spaces weglassen
mask = wiktionary_merkmal_abk.data.str.contains(' ')
df = wiktionary_merkmal_abk[mask].copy()
r = {' ':'',}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

676 rows added, now a total of 6216
time: 59.5 ms


In [126]:
# pflegen: Überfüssige Spaces streichen
r = {'  ':' ', '  ':' ',}
wiktionary_merkmal_abk['data'] = pak.replace_str(wiktionary_merkmal_abk.data, r)
wiktionary_merkmal_abk['data'] = wiktionary_merkmal_abk.data.str.strip()

time: 53.7 ms


In [127]:
grid(wiktionary_merkmal_abk.sort_values('data'))

6216 rows


time: 152 ms


In [128]:
# pflegen: Dups entfernen

wiktionary_merkmal_abk = pak.group_and_agg( wiktionary_merkmal_abk, 
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'score',   ],
              [ 'group',    'group',      'min',  'group',     'max',   'group',  'group',    'max',     ],
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'score',   ],        
             )


571 rows less, now 5645 rows
time: 1.34 s


In [129]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,score


time: 57.7 ms


In [130]:
# anschauen
grid(wiktionary_merkmal_abk.sort_values('score',ascending=False))

5645 rows


time: 142 ms


### wiktionary_lemma pflegen

In [131]:
# suche_debug 
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
2054,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,V2Xzj6L3xb,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,10.341779,1,friergetrocknen,VVPP
2055,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,IUwOEm6Eis,VER,V,manuell,,31.451888,1,gefriertrockn,VVPP
2194,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,IbKmXDIZ46,VER,V,manuell,,11.045676,1,Missrat,VVFIN
127895,gefriertrocknend,1054319,0,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,ksSiX4o537,ADJ,A,Ptz1,gefriertrocknen,2.650406,1,gefriertrocknend,ADJD
140462,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,PEiiiSZpRn,ADJ,A,Ptz2 manuell,missraten,1.630329,1,Missrat,VVFIN
140463,missratend,446280,0,missratend,missratend,False,1,ADJA,ADJA,VzLpPPn6D5,ADJ,A,Ptz1,missraten,2.882204,1,Missratend,ADJD
140820,mißraten_MADJ,-1,-1,mißraten,mißraten,False,1,ADJA,ADJA,zOqB9HbDOi,ADJ,A,Ptz2 manuell,missraten,0.500000,1,mißraten,VVPP


time: 205 ms


In [132]:
# Alle auflösbaren Abkürzungen mit eigenem Eintrag in wiktionary_lemma
mask_K = wiktionary_merkmal_abk.lemma_id != wiktionary_merkmal_abk.data_id
wiktionary_merkmal_abk[mask_K].tail(3)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,score
5642,üblich,ADJA,1,abk,,übl.,üblich~übl,12.390986
5643,üblicherweise,ADV,1,abk,,üblw,üblicherweise~üblw,10.589525
5644,üblicherweise,ADV,1,abk,,üblw.,üblicherweise~üblw,10.589525


time: 55.6 ms


In [133]:
# wiktionary_lemma.member nachtragen
# Nicht jede Abkürzung hat einen eigenen Eintrag in wiktionary_lemma.
# Aber wenn, dann ist dieser mit member 'Abkürzung' gekennzeichnet.

mask1 = ~wiktionary_lemma.member.str.contains('Abkürzung')                       # nicht als Abkürzung gekennzeichnet
mask2 = wiktionary_lemma.lemma_id.isin(wiktionary_merkmal_abk[mask_K].data_id)   # ist aber eine auflösbare Abkürzung
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 169 )

wiktionary_lemma.loc[mask,'member'] = (wiktionary_lemma[mask].member + ' Abkürzung').str.strip()
grid( wiktionary_lemma, mask )

check_mask:  171 rows
171 rows out of 165346


time: 222 ms


### translate_abk
* translate_abk1: Nimmt die in wiktionary_lemma gelisteten Abkürzungen und löst sie über ihr def-Merkmal auf
* translate_abk2: Basiert auf wiktionary_merkmal_abk

#### translate_abk1

In [134]:
# translate_abk1: Kurzformen aus wiktionary_lemma
mask_l = wiktionary_lemma.member.str.contains('Abkürzung')
spalten = ['lemma_id','lemma','tag','doc_tag','member','score',]
translate_abk1 = wiktionary_lemma[mask_l][spalten]
translate_abk1 = pak.rename_col(translate_abk1,'lemma',    'kurz')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_id', 'kurz_id')
translate_abk1 = pak.rename_col(translate_abk1,'tag',      'kurz_tag')
translate_abk1 = pak.rename_col(translate_abk1,'doc_tag',  'kurz_tag2')
translate_abk1 = pak.rename_col(translate_abk1,'member',   'kurz_member')
translate_abk1 = pak.rename_col(translate_abk1,'score',    'kurz_score')
translate_abk1 = pak.reset_index(translate_abk1)
pak.sample(translate_abk1)

,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score
0,24/7,24/7,ADJA,CARD $( CARD,manuell ergänzt Abkürzung,7.112668
47,&c.,&c.,TODO,KON NE,Abkürzung,4.341778
48,-sch,-sch,ZFIX,ADJD,Affix Suffix Suffix Abkürzung,7.864312
291,AdmOStArzt,AdmOStArzt,TODO,$(,Abkürzung,5.342996
321,Alt_4,Alt,TODO,ADJD,Abkürzung,0.152041
482,BH,BH,NN,NE,Abkürzung,19.050109
575,BPOLAK,BPOLAK,TODO,NE,Abkürzung,3.340560
760,Buch,Buch,NN,NN,Abkürzung,122.038642
4480,trans,trans,ADJA,NE,noSteig noFlex Abkürzung,7.286967
4623,°Re,°Re,TODO,_SP PTKVZ,Abkürzung,4.319854


time: 322 ms


In [135]:
# Langform verknüpfen
mask_m = (wiktionary_merkmal.merkmal == 'def') # alle Definitionen, nicht nur die der Abkürzungen

# lang_id
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data_id',
                                              col_rename = 'lang_id',
                                              func =       'first',
                                            )
# lang
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data',
                                              col_rename = 'lang',
                                              func =       'first',
                                            )

# lang_tag
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'lang_tag',
                                            )

# lang_tag2
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'lang_tag2',
                                            )

# lang_member
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )

# lang_score
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'score',
                                              col_rename = 'lang_score',
                                            )

translate_abk1['todo'] = 'lemma' 
translate_abk1['rank'] = 1.8 
print()
pak.sample(translate_abk1)

update_col: func first applied, 164448 records less!
update_col: 2790 cells written into new column
update_col: func first applied, 164448 records less!
update_col: 2790 cells written into new column
update_col: 2690 cells written into new column
update_col: 2690 cells written into new column
update_col: 2690 cells written into new column
update_col: 2690 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,todo,rank
0,24/7,24/7,ADJA,CARD $( CARD,manuell ergänzt Abkürzung,7.112668,Stunde,Stunde,NN,NN,Einheit manuell,74.508679,lemma,1.8
43,on,on,FM,FM,manuell ergänzt Abkürzung,5.702192,online,online,ADJA,ADV,noSteig noFlex,20.646286,lemma,1.8
47,&c.,&c.,TODO,KON NE,Abkürzung,4.341778,et_cetera,et cetera,ADV,NE FM,manuell Wortverbindung,5.975030,lemma,1.8
48,-sch,-sch,ZFIX,ADJD,Affix Suffix Suffix Abkürzung,7.864312,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,lemma,1.8
53,1_Joh,1 Joh,TODO,CARD NE,Abkürzung,3.628015,1._Johannesbrief,1. Johannesbrief,NN,ADV NN,adjNN Wortverbindung adjNN noPl,6.527298,lemma,1.8
80,8.,8.,TODO,ADJA,Abkürzung,7.520828,achter,achter,ADJA,ADJA,,16.371621,lemma,1.8
169,AKH,AKH,NN,NE,Abkürzung,8.108799,Krankenhaus,Krankenhaus,NN,NN,,456.211774,lemma,1.8
291,AdmOStArzt,AdmOStArzt,TODO,$(,Abkürzung,5.342996,Dienstgrad,Dienstgrad,NN,NE,,17.381010,lemma,1.8
321,Alt_4,Alt,TODO,ADJD,Abkürzung,0.152041,Beschriftung,Beschriftung,NN,NN,,17.397667,lemma,1.8
482,BH,BH,NN,NE,Abkürzung,19.050109,Büstenhalter,Büstenhalter,NN,NN,manuell,37.166863,lemma,1.8


time: 2.63 s


#### translate_abk2

In [136]:
# translate_abk2: aus wiktionary_merkmal_abk
translate_abk2 = wiktionary_merkmal_abk.copy()
translate_abk2 = pak.drop_cols(translate_abk2,['num','merkmal'])
translate_abk2 = pak.rename_col(translate_abk2,'lemma_id',  'lang_id')
translate_abk2 = pak.rename_col(translate_abk2,'lemma_tag', 'lang_tag')
translate_abk2 = pak.rename_col(translate_abk2,'score',     'lang_score')
translate_abk2 = pak.rename_col(translate_abk2,'data',      'kurz')
translate_abk2 = pak.rename_col(translate_abk2,'data_id',   'kurz_id')
translate_abk2 = pak.rename_col(translate_abk2,'meta',      'kurz_member')
translate_abk2 = pak.reset_index(translate_abk2)
translate_abk2 = pak.move_cols(translate_abk2,['kurz_id','kurz'])
pak.sample(translate_abk2)

,kurz_id,kurz,lang_id,lang_tag,kurz_member,lang_score
0,-prozentig~-%ig,-%ig,-prozentig,ZFIX,,4.341047
196,Anmerkung_der_Redaktion_M~Anm. d. Red.,Anm d Red,Anmerkung_der_Redaktion_M,KOMBI,manuell,0.450000
1031,Euro~€,€,Euro,NN,manuell,17.539585
1313,Gen,Gen,General,NN,,22.538368
1639,Ingenieur~Dipl.-Ing,Dipl Ing,Ingenieur,NN,,28.680878
1950,Krankenhaus~Krh,Krh,Krankenhaus,NN,,227.583435
2220,μm,μm,Mikrometer,NN,manuell,11.360536
2690,Prozent~%,%,Prozent,NN,manuell,17.524969
3187,Strafe~Str,Str,Strafe,NN,,23.846529
4008,dies,dies,dieselbe,,,0.844898


time: 238 ms


In [137]:
# keine eigene kurz_id
mask = translate_abk2.kurz_id == translate_abk2.lang_id
check_mask(translate_abk2,mask, 0) 
translate_abk2.loc[mask,'kurz_id'] = ''

check_mask:  0 rows
time: 54.4 ms


In [138]:
# kurz_id ist nicht in wiktionary_lemma
mask1 = ~translate_abk2.kurz_id.isin(wiktionary_lemma.lemma_id)
mask2 =  translate_abk2.kurz_id != ''
mask = mask1  &  mask2 
check_mask(translate_abk2,mask,4503) 
translate_abk2.loc[mask,'kurz_id'] = ''

check_mask:  4493 rows
time: 67.3 ms


In [139]:
# War das richtig?
bgc('orange')

time: 45.1 ms


In [140]:
# kurz_tag
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'kurz_tag',
                                              func =       'first',
                                            )

# kurz_tag2
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'kurz_tag2',
                                            )

# kurz_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'kurz_member',
                                              cond =       'null'
                                            )

# kurz_score
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'score',
                                              col_rename = 'kurz_score',
                                            )

# lang
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma',
                                              col_rename = 'lang',
                                            )
# lang_tag2
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'lang_tag2',
                                            )


# lang_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )


translate_abk2 = pak.move_cols(translate_abk2,['kurz_tag','kurz_tag2','kurz_member','kurz_score'],'kurz')
translate_abk2 = pak.move_cols(translate_abk2,['lang','lang_tag','lang_tag2','lang_member'],'lang_id')
translate_abk2['rank'] = 1.0 
translate_abk2['todo'] = 'merkmal' 
print()
pak.sample(translate_abk2)

update_col: func first applied, but it was pointless!
update_col: 1152 cells written into new column
update_col: 1152 cells written into new column
update_col: 983 cells written into existing column
update_col: 1152 cells written into new column
update_col: 5625 cells written into new column
update_col: 5625 cells written into new column
update_col: 5625 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,rank,todo
0,,-%ig,<NA>,<NA>,,NaN,-prozentig,-prozentig,ZFIX,NN,Gebundenes_Lexem,4.341047,1.0,merkmal
1,-sch,-sch,ZFIX,ADJD,Affix Suffix Suffix Abkürzung,7.864312,-schaft,-schaft,ZFIX,NN,Substantiv Substantiv Affix Suffix Suffix,13.516443,1.0,merkmal
196,,Anm d Red,<NA>,<NA>,manuell,NaN,Anmerkung_der_Redaktion_M,Anmerkung der Redaktion,KOMBI,NN ART NN,Wortverbindung manuell,0.450000,1.0,merkmal
578,Buch,Buch,NN,NN,Abkürzung,122.038642,Buchung,Buchung,NN,NN,,9.054811,1.0,merkmal
606,BVGer,BVGer,TODO,ADJA,Abkürzung,4.582948,Bundesverwaltungsgericht,Bundesverwaltungsgericht,NN,NN,,9.409257,1.0,merkmal
833,DZ,DZ,TODO,XY,manuell,11.021438,Doppelzimmer,Doppelzimmer,NN,ADJA,manuell,10.560292,1.0,merkmal
1031,,€,<NA>,<NA>,manuell,NaN,Euro,Euro,NN,NN,Einheit manuell,17.539585,1.0,merkmal
1080,FschJg,FschJg,TODO,_SP,Abkürzung,7.067722,Fallschirmjäger,Fallschirmjäger,NN,NN,,12.175396,1.0,merkmal
1313,Gen,Gen,NN,NN,Abkürzung,59.548377,General,General,NN,NN,,22.538368,1.0,merkmal
1639,,Dipl Ing,<NA>,<NA>,,NaN,Ingenieur,Ingenieur,NN,NN,,28.680878,1.0,merkmal


time: 1.6 s


#### translate_abk

In [141]:
# translate_abk erstellen
translate_abk = pd.concat([translate_abk1,translate_abk2], ignore_index=True)
translate_abk = pak.change_datatype(translate_abk)
translate_abk_trash = translate_abk.head(0)
translate_abk_trash['msg'] = None

change_datatype 
kurz_score           --> np.float32
lang_score           --> np.float32
todo                 --> pd.string 
rank                 --> np.float32
change_datatype before: 6.9 MB after: 6.8 MB

time: 172 ms


In [142]:
# fillna

translate_abk['kurz_id']     = translate_abk.kurz_id.fillna('')
translate_abk['lang_id']     = translate_abk.lang_id.fillna('')

translate_abk['kurz']        = translate_abk.kurz.fillna('')
translate_abk['lang']        = translate_abk.lang.fillna('')

translate_abk['kurz_tag']    = translate_abk.kurz_tag.fillna('').str.replace('TODO','').str.strip()
translate_abk['lang_tag']    = translate_abk.lang_tag.fillna('').str.replace('TODO','').str.strip()

translate_abk['kurz_tag2']    = translate_abk.kurz_tag2.fillna('')
translate_abk['lang_tag2']    = translate_abk.lang_tag2.fillna('')

translate_abk['kurz_member'] = translate_abk.kurz_member.fillna('')
translate_abk['lang_member'] = translate_abk.lang_member.fillna('')

translate_abk['kurz_score']  = translate_abk.kurz_score.fillna(0)
translate_abk['lang_score']  = translate_abk.lang_score.fillna(0)

time: 74.8 ms


#### Belöschen und ranken

In [143]:
# konkurrenz: Wie viele konkurrierende lang_id gibt es?
gruppiert = translate_abk.groupby(['kurz'])
translate_abk['konkurrenz'] = gruppiert.lang_id.transform( 'nunique' ) 

time: 57.5 ms


In [144]:
#grid(translate_abk)

time: 30.9 ms


In [145]:
# ID fehlt
maskA = translate_abk.kurz_id == ''
maskB = translate_abk.lang_id == ''
mask = maskA | maskB
check_mask(translate_abk,mask,6213)

translate_abk.loc[mask,'rank'] *= 0.5
translate_abk[mask]

check_mask:  6369 rows


,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,todo,rank,konkurrenz
4,5G,5G,NN,CARD NE,Abkürzung manuell ergänzt,6.308770,,,,,,0.000000,lemma,0.9,1
13,GmbH,GmbH,NN,NN,manuell ergänzt Abkürzung,20.605112,,,,,,0.000000,lemma,0.9,1
14,HMM,HMM,ITJ,NE,Abkürzung manuell ergänzt,5.387941,,,,,,0.000000,lemma,0.9,1
20,Max,Max,NE,NE,Person_vorname Abkürzung,17.480053,,,,,,0.000000,lemma,0.9,2
23,Opa_1,Opa,NN,FM,Person_pre manuell ergänzt Abkürzung,1.188776,,,,,,0.000000,lemma,0.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10306,,überw.,,,,0.000000,überwiegend,überwiegend,ADJA,ADJD,Ptz1 noSteig manuell,5.408039,merkmal,0.5,2
10307,,übl,,,,0.000000,üblich,üblich,ADJA,ADJD,,12.390986,merkmal,0.5,1
10308,,übl.,,,,0.000000,üblich,üblich,ADJA,ADJD,,12.390986,merkmal,0.5,1
10309,,üblw,,,,0.000000,üblicherweise,üblicherweise,ADV,ADV,,10.589525,merkmal,0.5,1


time: 75.1 ms


In [146]:
# Anfangsbuchstabe ungleich
mask1 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask2 = ~translate_abk.kurz.str[0].str.isdigit() 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk,mask,992)

translate_abk.loc[mask,'rank'] *= 0.8
translate_abk[mask]

check_mask:  978 rows


,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,todo,rank,konkurrenz
7,Ann,Ann,NE,NE,Person_vorname Abkürzung,15.046822,Vorname,Vorname,NN,NN,,36.444092,lemma,1.44,2
8,Ari_1,Ari,NE,FM,Person_vorname Abkürzung,3.072032,Vorname,Vorname,NN,NN,,36.444092,lemma,1.44,3
9,BDSM,BDSM,NN,NE,manuell ergänzt Abkürzung,15.816687,Sammelbezeichnung,Sammelbezeichnung,NN,NN,,11.641731,lemma,1.44,2
12,Franz,Franz,NE,NE,Person_vorname Abkürzung,18.639338,Vorname,Vorname,NN,NN,,36.444092,lemma,1.44,2
15,ISBN,ISBN,NN,NE,Abkürzung manuell ergänzt,12.128364,Nummer,Nummer,NN,NN,Einheit manuell,68.075066,lemma,1.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10254,Hafen,Hafen,NN,NN,Abkürzung,71.411949,Ölhafen,Ölhafen,NN,NN,,5.645554,merkmal,0.80,2
10255,,AUT,,,,0.000000,Österreich,Österreich,NE,NE,Geo Geo_land,27.548113,merkmal,0.40,1
10257,,OTH Radar,,,,0.000000,Überhorizontradar,Überhorizontradar,NN,NE,,5.302071,merkmal,0.40,1
10258,,OTH-Radar,,,,0.000000,Überhorizontradar,Überhorizontradar,NN,NE,,5.302071,merkmal,0.40,1


time: 84.6 ms


In [147]:
# nichtssagende_abk1
mask1 =  translate_abk.kurz.str.len() == 3
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(translate_abk,mask,20)

msg = 'nichtssagende_abk1'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  20 rows
Move 20 rows from 10311
20 rows


time: 148 ms


In [148]:
# nichtssagende_abk2
mask1 =  translate_abk.kurz.str.len() == 4
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 =  translate_abk.kurz.str.endswith('.')
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,34)

msg = 'nichtssagende_abk2'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  34 rows
Move 34 rows from 10291
34 rows out of 54


time: 223 ms


In [149]:
# Abk_kürzt_nicht
mask1 =  translate_abk.lang.str.len() <= translate_abk.kurz.str.len() 
mask2 =  translate_abk.lang_id != ''
mask3 =  translate_abk.kurz_id != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,161)

msg = 'Abk_kürzt_nicht'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  162 rows
Move 162 rows from 10257
162 rows out of 216


time: 233 ms


In [150]:
# zu_kurz_end_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.lower().str.endswith(('a.','e.','i.','o.','u.','ä.','ö.','ü.','y.'))
mask3 =  translate_abk.kurz_tag == ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,20)

msg = 'zu_kurz_end_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  20 rows
Move 20 rows from 10095
20 rows out of 236


time: 201 ms


In [151]:
# zu_kurz_start_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.endswith('.')
mask3 =  translate_abk.kurz.str.lower().str.startswith(('a','e','i','o','u','ä','ö','ü','y'))
mask4 =  translate_abk.kurz_tag == ''
mask5 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,18)

msg = 'zu_kurz_start_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  18 rows
Move 18 rows from 10075
18 rows out of 254


time: 264 ms


In [152]:
# Ranking: Ist die Abkürzung in lexeme_manuell?
mask = translate_abk.kurz_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

mask = translate_abk.lang_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

time: 44.4 ms


In [153]:
# Ranking: Ist die Abkürzung beidseitig verknüpft?
gruppiert = translate_abk.groupby(['kurz_id'])
translate_abk['rank2'] = gruppiert.todo.transform( 'nunique' ) 

mask = translate_abk.rank2 > 1
translate_abk.loc[mask,'rank'] *= 5.0
translate_abk = pak.drop_cols(translate_abk,'rank2')

time: 62.3 ms


In [154]:
# Abkürzung hat eigenen Eintrag
mask = translate_abk.kurz_id != ''
translate_abk.loc[mask,'rank'] *= 2.0

time: 60.9 ms


In [155]:
# Ranking: lang_score
translate_abk['rank'] *= (translate_abk.lang_score + 0.2)

time: 52.9 ms


In [156]:
# Ranking: Kein tag
mask = translate_abk.lang_tag == ''
translate_abk.loc[mask,'rank'] *= 0.2
translate_abk[mask]

,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,todo,rank,konkurrenz
4,5G,5G,NN,CARD NE,Abkürzung manuell ergänzt,6.308770,,,,,,0.000000,lemma,0.144000,1
5,9/11,9/11,NN,CARD $( CARD,manuell ergänzt Abkürzung,27.385506,Terroranschläge,Terroranschläge,,,,0.000000,lemma,1.440000,2
13,GmbH,GmbH,NN,NN,manuell ergänzt Abkürzung,20.605112,,,,,,0.000000,lemma,0.144000,1
14,HMM,HMM,ITJ,NE,Abkürzung manuell ergänzt,5.387941,,,,,,0.000000,lemma,0.144000,1
16,Israel,Israel,NE,NE,Geo Geo_land Abkürzung,48.746639,Nahen_Osten,Nahen Osten,,,,0.000000,lemma,0.576000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9828,,st,,,,0.000000,sine_tempore,sine tempore,,NE NN,Wortverbindung,1.213155,merkmal,0.141315,1
10097,,u s,,,,0.000000,ut_supra,ut supra,,XY FM,Wortverbindung,0.863265,merkmal,0.106327,1
10098,,u. s.,,,,0.000000,ut_supra,ut supra,,XY FM,Wortverbindung,0.863265,merkmal,0.106327,1
10099,,u.s.,,,,0.000000,ut_supra,ut supra,,XY FM,Wortverbindung,0.863265,merkmal,0.106327,1


time: 68.6 ms


In [157]:
# sicherheitshalber einmal sortieren
translate_abk = translate_abk.sort_values(['kurz','lang_score'],ascending=[True,False]).reset_index(drop=True)

time: 73.3 ms


In [158]:
# Ranking betrachten
grid(translate_abk.sort_values(['konkurrenz','kurz','rank',],ascending=[False,True,False]))

10057 rows


time: 229 ms


In [159]:
# rank!
translate_abk = pak.rank(translate_abk, col_score='rank', cols_group='kurz', on_conflict='first')

rank: 2351 rows less, now 7706 rows
time: 60.8 ms


In [160]:
# zu häufig und Anfangsbuchstabe ungleich

erlaubt = ['Ingenieur','okay','per Adresse','Professor','Kalorie','insbesondere','insbesonders']

a = pak.analyse_freqs(translate_abk,'lang')
mask_a = a.lang_count >= 5 
mask1 = translate_abk.lang.isin(a[mask_a].lang)
mask2 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask3 = ~translate_abk.kurz.str[0].str.isdigit() 
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5A = translate_abk.kurz_member == 'Abkürzung'  # genau gleich 'Abkürzung', also nichts zusätzliches 
mask5B = translate_abk.kurz_member == ''           # oder ganz leer 
mask6 = ~translate_abk.lang.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B)  &  mask6  
check_mask(translate_abk,mask,183)

msg = 'zu_häufig'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  184 rows
Move 184 rows from 7706
184 rows out of 438


time: 234 ms


In [161]:
a = pak.analyse_freqs(translate_abk,['lang','kurz'])
grid(a)

3083 rows


time: 577 ms


In [162]:
# erlaubt_und_verboten

verboten     = ['Bundesministerium','Brandmeister','Braumeister','Rundfunk','Ortsnamengrundwort','Kurzwort','Diminutiv','diminuendo']
doch_erlaubt = []

mask1 =  translate_abk.lang.isin(verboten) 
mask2 = ~translate_abk.lang.isin(doch_erlaubt) 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4 
#grid(translate_abk,mask)
check_mask(translate_abk,mask,66)

msg = 'erlaubt_und_verboten'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  67 rows
Move 67 rows from 7522
67 rows out of 505


time: 206 ms


In [163]:
# lang_tag_fehlt
mask1 = translate_abk.lang_tag == ''
mask2 = translate_abk.kurz_tag != ''
mask3 = translate_abk.lang_id  != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(translate_abk,mask)
check_mask(translate_abk,mask,6)

msg = 'lang_tag_fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  6 rows
Move 6 rows from 7455
6 rows out of 511


time: 206 ms


In [164]:
# tag von lang nach kurz übertragen
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_tag != ''
mask = mask1  &  mask2
check_mask(translate_abk,mask,5068)

translate_abk.loc[mask,'kurz_tag'] = translate_abk[mask].lang_tag
grid(translate_abk,mask)

check_mask:  5064 rows
5064 rows out of 7449


time: 228 ms


In [165]:
# Weiterer_Schrott
mask1 =  translate_abk.kurz_tag == ''
mask2 = ~translate_abk.lang_member.isin(['','Wortverbindung','Redewendung'])
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  
#grid(translate_abk,mask)
check_mask(translate_abk,mask,2)

msg = 'Weiterer_Schrott'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  2 rows
Move 2 rows from 7449
2 rows out of 513


time: 207 ms


In [166]:
# KOMBI
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member.isin(['Wortverbindung','Redewendung'])
mask = mask1  &  mask2 
check_mask(translate_abk,mask,88)

translate_abk.loc[mask,'kurz_tag'] = 'KOMBI'
grid(translate_abk,mask)

check_mask:  88 rows
88 rows out of 7447


time: 174 ms


In [167]:
# lang_member ist jetzt ausgeschöpft
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member != ''
mask = mask1  &  mask2 
check_mask(translate_abk,mask,0)

check_mask:  0 rows
time: 68.9 ms


In [168]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str.len() > 2
mask5A = translate_abk.lang_tag2.str.contains('NN|NE', regex=True)
mask5B = translate_abk.kurz.str.len() >= 4 
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B) 
check_mask(translate_abk, mask, 945)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  945 rows
945 rows out of 7447


time: 158 ms


In [169]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str[0] ==  translate_abk.lang.str[0]
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(translate_abk, mask, 4)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  4 rows
4 rows out of 7447


time: 147 ms


In [170]:
# Aufzählungen => CARD
# nicht nur für kurz_tag == ''
mask1 = translate_abk.kurz.str[:-1].str.isnumeric()
mask2 = translate_abk.kurz.str.endswith('.')  
mask3 = translate_abk.kurz.str.len() <= 3 
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk, mask, 11)
translate_abk.loc[mask,'kurz_tag'] = 'CARD'
grid(translate_abk,mask)

check_mask:  11 rows
11 rows out of 7447


time: 165 ms


In [171]:
# Rest
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.lang_tag2 == ''
mask = mask1  &  mask2  
check_mask(translate_abk, mask, 719)

grid(translate_abk,mask)
msg = 'Rest'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  727 rows
727 rows out of 7447
Move 727 rows from 7447
727 rows out of 1240


time: 323 ms


In [172]:
# Mehr gibt es nicht
mask = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 0)

#grid(translate_abk,mask)
#msg = 'Rest'
#translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
time: 50.1 ms


In [173]:
# Neue kurz_id vergeben
mask = translate_abk.kurz_id == ''
translate_abk.loc[mask,'kurz_id'] = translate_abk[mask].kurz.str.replace(' ','_') + '_A'
check_mask(translate_abk,mask,2983)
grid(translate_abk,mask)

check_mask:  2967 rows
2967 rows out of 6720


time: 194 ms


In [174]:
# member Abkürzung nachtragen
mask = ~translate_abk.kurz_member.str.contains('Abkürzung')
translate_abk.loc[mask,'kurz_member'] = (translate_abk[mask].kurz_member + ' Abkürzung').str.strip()

time: 60 ms


#### rückübertragen

In [175]:
# Alle Kurzformen aus wiktionary_lemma rauslöschen, die nicht in translate_abk erwähnt werden
# und keinen echten tag haben
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.member.str.contains('manuell')
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask4 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask5 =  wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(wiktionary_lemma,mask)
check_mask(wiktionary_lemma,mask,1136)

msg = 'translate_abk1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( wiktionary_lemma, wiktionary_lemma_trash, mask, msg)

# anschauen
grid( wiktionary_lemma_trash, msg )

check_mask:  1131 rows
Move 1131 rows from 165346
1131 rows


time: 583 ms


In [176]:
# Einige Datensätze sind in translate_abk gelöscht, haben aber in wiktionary_lemma einen richtigen tag.
# Die bleiben drin.
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask4 =  wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma,mask,58)

# anschauen
#grid( wiktionary_lemma, mask )

check_mask:  58 rows
time: 136 ms


In [177]:
# kurz_tag => tag_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'kurz_tag',
                                   col_rename = 'tag_new',
                                   func =       'first',
                                  )
wiktionary_lemma = pak.move_cols(wiktionary_lemma,'tag_new','lemma_len')

update_col: func first applied, 276 records less!
update_col: 3477 cells written into new column
time: 877 ms


In [178]:
# tag_new => tag
mask = (wiktionary_lemma.tag != wiktionary_lemma.tag_new)  
check_mask(wiktionary_lemma,mask,3014)
wiktionary_lemma.loc[mask,'tag'] = wiktionary_lemma[mask].tag_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'tag_new')

check_mask:  3009 rows
time: 198 ms


In [179]:
# pflege_tag_felder für die veränderten tags
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1

time: 4.51 s


In [180]:
# lang_id => data_id_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'lang_id',
                                   col_rename = 'data_id_new',
                                   func =       'first',
                                  )

update_col: func first applied, 276 records less!
update_col: 3477 cells written into new column
time: 834 ms


In [181]:
# data_id_new => data_id
mask = (wiktionary_lemma.data_id != wiktionary_lemma.data_id_new)  
check_mask(wiktionary_lemma,mask, 2397)
wiktionary_lemma.loc[mask,'data_id'] = wiktionary_lemma[mask].data_id_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'data_id_new')

check_mask:  2417 rows
time: 164 ms


In [182]:
# Neue Datensätze für wiktionary_lemma
mask1 = translate_abk.kurz_id.str.endswith('_A')
mask2 = ~translate_abk.lang_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  
check_mask(translate_abk,mask,2934)

df = translate_abk[mask]

df = pak.rename_col(df, 'kurz_id',     'lemma_id')
df = pak.rename_col(df, 'kurz',        'lemma')
df = pak.rename_col(df, 'kurz_tag',    'tag')
df = pak.rename_col(df, 'kurz_member', 'member')
df = pak.rename_col(df, 'kurz_score',  'score')
df = pak.rename_col(df, 'lang_id',     'data_id')
df['page_id']   = -1
df['page_part'] = -1
df['doc_len']   = ''
df['doc_lemma'] = ''
df['doc_tag']   = ''
df = pak.move_cols(df,['page_id','page_part'],'lemma_id')
df = pak.drop_cols(df,['konkurrenz','rank','todo','lang','lang_tag','lang_tag2','lang_member','lang_score','kurz_tag2',])

df = ergänze_felder(df)
df = pflege_tag_felder(df, translate_tagZ)  
assert set(wiktionary_lemma.columns) ==  set(df.columns) 
grid(df)

check_mask:  2967 rows
check_mask:  8 rows
2967 rows


time: 500 ms


In [183]:
# wiktionary_lemma anfügen!
wiktionary_lemma = pak.add_rows(wiktionary_lemma, df)

2967 rows added, now a total of 167182
time: 195 ms


In [184]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows
time: 99.7 ms


In [185]:
# Keine neuen Datensätze für translate_lex
mask1 = translate_abk.kurz_id.str.endswith('_A')
mask2 = translate_abk.lang_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  
check_mask(translate_abk,mask,0)

check_mask:  0 rows
time: 150 ms


#### nochmal belöschen

In [186]:
# lang_id fehlt >> löschen
mask = translate_abk.lang_id == ''
check_mask(translate_abk, mask, 1062)

msg = 'lang_id fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  1062 rows
Move 1062 rows from 6720
1062 rows out of 2302


time: 212 ms


In [187]:
# Bedeutungsloses ggf. löschen (dürfte eigentlich nichts mehr dabei sein)
maskA = translate_abk.kurz_id == translate_abk.lang_id
maskB = translate_abk.kurz    == translate_abk.lang
mask = maskA | maskB
check_mask(translate_abk,mask,0)

msg = 'bedeutungslos'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
No rows moved
time: 58 ms


### wiktionary_merkmal belöschen
* alle abk-Merkmale sind schon gelöscht

In [188]:
# alle kurzformen
mask_abk =  wiktionary_lemma.member.str.contains('Abkürzung')
kurzformen = wiktionary_lemma[mask_abk].lemma_id

time: 98 ms


In [189]:
# Alle def-Merkmale aller Kurzformen löschen
mask1 = wiktionary_merkmal.merkmal == 'def'
mask2 = wiktionary_merkmal.lemma_id.isin(kurzformen)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,4659)

msg = 'def einer Abkürzung'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)

mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash, mask )

check_mask:  4687 rows
Move 4687 rows from 4661363
4687 rows out of 6709


time: 4.47 s


### Abkürzungen überprüfen

In [190]:
search_str(translate_abk,'5G')

,kurz_id,kurz,kurz_tag,kurz_tag2,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_tag2,lang_member,lang_score,todo,rank,konkurrenz


time: 95.8 ms


In [191]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0)

check_mask:  0 rows
time: 105 ms


In [192]:
mask = wiktionary_lemma.lemma_id == 'H2O_A'
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
164220,H2O_A,-1,-1,H2O,h2o,True,1,NN,NN,2kWP3EntQJ,NOM,N,Abkürzung,Wasser,0.0,,,


time: 72 ms


## wiktionary_lemma ==========

### pflegen

In [193]:
# manuelle_kürzel erstellen: Welche Sorten Sonder-IDs gibt es?
# alt: ('_V', '_M', '_N', '_Z', '_O', '_NV')
# ('_V', '_A', '_M', '_N', '_MADJ', '_Z', '_T')

wiktionary_lemma['kürzel'] = wiktionary_lemma.lemma_id.str.split('_').str[-1]

mask = wiktionary_lemma.lemma_id.str.contains('_')
a = pak.analyse_freqs(wiktionary_lemma[mask], ['kürzel','lemma_id'])

mask1 = a.kürzel_count > 30
mask2 = a.kürzel.str.isalpha() 
mask3 = a.kürzel.str.len() < 5 
mask  = mask1  &  mask2  &  mask3
manuelle_kürzel = tuple(['_'+kürzel for kürzel in list(a[mask].kürzel)])  # um manuelle lemma_id erkennen zu können 
print(manuelle_kürzel)
assert len(manuelle_kürzel) == 7
grid(a,mask)

wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'kürzel')

('_V', '_A', '_M', '_N', '_MADJ', '_Z', '_T')
7 rows out of 2537
time: 1.01 s


In [194]:
# score page_part -1 analysieren
mask = wiktionary_lemma.page_part == -1
pak.analyse_freqs(wiktionary_lemma[mask],'member',splits=' ')

,member,member_count,member_percent,graph
0,Person_vorname,3061,38.6,###################
1,Abkürzung,2973,37.5,##################
2,manuell,860,10.8,#####
3,Zahlwort,291,3.7,#
4,ergänzt,231,2.9,#
5,Person_nachname,195,2.5,#
6,Ptz2,129,1.6,
7,Ptz1,54,0.7,
8,Geo,50,0.6,
9,Geo_stadt,47,0.6,


time: 105 ms


In [195]:
# score page_part -1 pushen
mask = wiktionary_lemma.page_part == -1
check_mask(wiktionary_lemma,mask,7047)

wiktionary_lemma.loc[mask,'score'] += 0.2
wiktionary_lemma.loc[mask,'score'] *= 1.5

check_mask:  7039 rows
time: 65 ms


### tag

In [196]:
# sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,Redewendung,263,64.0,"[A-a_machen, Aa_machen, Abbitte_leisten, Abbit...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Wortverbindung,146,35.5,"[Anno_Domini, Artes_liberales, Besorgnis_erreg...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,,2,0.5,"[Tempi_passati, variatio_delectat]","[1, 1]"


time: 110 ms


In [197]:
# NE-Datensätze löschen, die mit hoher Wahrscheinlichkeit was anderes als einen Namen bezeichnen
geschützte_namen = ['Martin','Baum','Junge','Klein','Keller','Mann','Peter','Schwarz','Simon',
                    'Thomas','Weiß','Winter','Sommer','Wolf','Stein','Henry','Frank','Müller',
                    'Meier','Huber','Schneider','Bauer','Conrad','Ackermann','Kramer','Grimm',
                    'Heinrich','Wagner','Pfeifer','Zimmer','Bach','Krämer','Fischer','Arnold',
                    'Lorenz','Fiedler','Weber','Walther','Busch','Schäfer','Michel','Ritter',
                    'Beck',
                   ]
mask1 =  wiktionary_lemma.member.str.contains('Person_nachname')
mask2 =  wiktionary_lemma.tag == 'NE'
mask3 =  wiktionary_lemma.page_part > 0
mask4 =  wiktionary_lemma.score < 2.6
mask5 = ~wiktionary_lemma.lemma.isin(geschützte_namen)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask( wiktionary_lemma, mask, 343 )

msg = 'NE1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  372 rows
Move 372 rows from 167182
372 rows out of 1503


time: 498 ms


In [198]:
# Prüfen: Alle Person_nachname sind NE
mask1 = wiktionary_lemma.member.str.contains('Person_nachname')
mask2 = wiktionary_lemma.tag == 'NE'
mask = mask1  &  ~mask2
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows
time: 128 ms


In [199]:
# anschauen
mask = wiktionary_lemma.member.str.contains('Person_nachname')
a = pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma'] )
grid(a)

10 rows


time: 175 ms


In [200]:
#grid(pak.analyse_cols(wiktionary_lemma))

time: 51.8 ms


In [201]:
# KOMBI: Datensätze umtaggen, die eine Wortverbindung oder eine Redewendung beinhalten
mask1  = wiktionary_lemma.tag == 'TODO'
mask2a = wiktionary_lemma.member.str.contains('Redewendung')
mask2b = wiktionary_lemma.member.str.contains('Wortverbindung')
mask = mask1  &  (mask2a | mask2b)
check_mask( wiktionary_lemma, mask, 407 )

change_tag( wiktionary_lemma, mask, 'KOMBI', manuelle_kürzel )

#wiktionary_lemma.loc[mask,'tag']   = 'KOMBI'
#wiktionary_lemma.loc[mask,'tag_0'] = 'KOMBI'
#wiktionary_lemma.loc[mask,'tagZ']  = 'X_kombi'
#wiktionary_lemma.loc[mask,'tagZZ'] = 'X'
#grid(wiktionary_lemma,mask)

check_mask:  407 rows
407 Datensätze
time: 268 ms


In [202]:
# Rest sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,,2,100.0,"[Tempi_passati, variatio_delectat]","[1, 1]"


time: 97.4 ms


In [203]:
#mask = wiktionary_lemma.member.str.contains('Abkürzung')
#pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'tag'],  )

time: 39.9 ms


In [204]:
# Rest löschen
mask = wiktionary_lemma.tag == 'TODO'
check_mask( wiktionary_lemma, mask, 0, 10 )

msg = 'TODO-Rest'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  2 rows
Move 2 rows from 166810
2 rows out of 1505


time: 359 ms


In [205]:
# ADJD >> ADJA
wiktionary_lemma['tag'] = wiktionary_lemma.tag.str.replace('ADJD','ADJA')

time: 169 ms


In [206]:
# Einzelkorrekturen        
einzelkorrekturen = { 'sämtlich'       : 'PIAT ADJA',  
                      'besitzergreifen': 'VVFIN',    
                    }
for i,t in einzelkorrekturen.items():
    mask = (wiktionary_lemma.lemma_id == i)
    wiktionary_lemma.loc[mask,'tag'] = t

time: 106 ms


In [207]:
# Hilvsverben auf VAFIN
hilfsverben = [ 'haben', 'sein', 'werden', 'tun' ]
mask1 = wiktionary_lemma.tagZZ == 'V'
mask2 = wiktionary_lemma.lemma.isin(hilfsverben)
mask3 = wiktionary_lemma.tag != 'VAFIN'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 1)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VAFIN', manuelle_kürzel )

check_mask:  1 rows
1 Datensätze
time: 144 ms


In [208]:
# Modalverben auf VMFIN
modalverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 =  wiktionary_lemma.lemma.isin(modalverben)
mask3 =  wiktionary_lemma.tag != 'VMFIN'
mask4 =  wiktionary_lemma.page_part == 0
mask  = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_lemma, mask, 7)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VMFIN', manuelle_kürzel )

check_mask:  7 rows
7 Datensätze
time: 158 ms


In [209]:
# Alle anderen auf VVFIN 
#TODO vgl. und vergleiche müssten hier verschwinden
sonderverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen', 'haben', 'sein', 'werden', 'tun' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(sonderverben)
mask3 =  wiktionary_lemma.tag != 'VVFIN'
mask  = mask1  &  mask2  &  mask3  
check_mask( wiktionary_lemma, mask, 0,2)

change_tag( wiktionary_lemma, mask, 'VVFIN', manuelle_kürzel, manuelle_quellen_schützen=False )
grid(wiktionary_lemma,mask)

check_mask:  0 rows
0 Datensätze
No rows, mask filters them all away
time: 165 ms


In [210]:
# Verben analysieren
mask = wiktionary_lemma.tagZZ == 'V'
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma_id'])

,tag,tag_count,tag_percent,lemma_id,lemma_id_count
0,VVFIN,13593,99.9,"[Schlange_stehen, abbeißen, abberufen, abfahre...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,VMFIN,7,0.1,"[brauchen, dürfen, können, mögen, müssen, soll...","[1, 1, 1, 1, 1, 1, 1]"
2,VAFIN,5,0.0,"[haben, sein, tun, werden, tun_1]","[1, 1, 1, 1, 1]"


time: 168 ms


### Affix und Suffix

In [211]:
# AFIX verschieben
mask1  = wiktionary_lemma.lemma.str.endswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 554)

msg = 'AFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  547 rows
Move 547 rows from 166808
547 rows


time: 394 ms


In [212]:
# ZFIX verschieben
mask1 = wiktionary_lemma.lemma.str.startswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 389)

msg = 'ZFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  386 rows
Move 386 rows from 166261
386 rows out of 933


time: 395 ms


In [213]:
# Sonstige Fixe
maskA = wiktionary_lemma.lemma.str.startswith('-')
maskB = wiktionary_lemma.lemma.str.endswith('-')
maskC = wiktionary_lemma.tag.str.contains('AFIX')
maskD = wiktionary_lemma.tag.str.contains('ZFIX')
maskE = wiktionary_lemma.member.str.contains('Affix')
maskF = wiktionary_lemma.member.str.contains('Suffix')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = (maskA | maskB | maskC | maskD | maskE | maskF)  &  mask2
check_mask( wiktionary_lemma, mask, 2)

msg = 'DIVFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  2 rows
Move 2 rows from 165875
2 rows out of 935


time: 718 ms


### member

In [214]:
# member Aufzählung bei CARD
mask1 = wiktionary_lemma.lemma.str[:-1].str.isnumeric()
mask2 = wiktionary_lemma.lemma.str.endswith('.')  
mask3 = wiktionary_lemma.lemma.str.len() <= 3 
mask4 = wiktionary_lemma.tag == 'CARD'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma, mask, 11)

wiktionary_lemma.loc[mask,'member'] += ' Aufzählung'
grid(wiktionary_lemma,mask)

check_mask:  11 rows
11 rows out of 165873


time: 297 ms


In [215]:
# member noSg und noPl nur bei Nomen
mask1a = wiktionary_lemma.member.str.contains('noPl')
mask1b = wiktionary_lemma.member.str.contains('noSg')
mask2  = wiktionary_lemma.tagZZ != 'N'
mask  = (mask1a | mask1b)  &  mask2
check_mask( wiktionary_lemma, mask, 0, 17 )

wiktionary_lemma.loc[mask,'member'] =  pak.remove_words( wiktionary_lemma[mask].member, ['noPl', 'noSg'] )
#grid(wiktionary_lemma,mask)

check_mask:  16 rows
time: 235 ms


### Allgemein

In [216]:
# Redundante Spalten
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0) # ein paar können wir tolerieren

check_mask:  283 rows
check_mask:  0 rows
time: 4.85 s


In [217]:
# Keine Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
check_mask( wiktionary_lemma, mask, 0)

check_mask:  0 rows
time: 75.3 ms


In [218]:
# not any_nan
assert not pak.any_nan(wiktionary_lemma)
#pak.analyse_cols(wiktionary_lemma)

time: 144 ms


In [219]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag
2048,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,PQFyiOn9vW,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,10.341779,1,friergetrocknen,VVPP
2049,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,XkCBLk3Wge,VER,V,manuell,,31.451888,1,gefriertrockn,VVPP
2188,missraten,242150,0,missraten,missraten,False,1,VVFIN,VVFIN,hpVjAWx54m,VER,V,manuell,,11.045676,1,Missrat,VVFIN
125994,gefriertrocknend,1054319,0,gefriertrocknend,gefriertrocknend,False,1,ADJA,ADJA,vEF3ZkJd4C,ADJ,A,Ptz1,gefriertrocknen,2.650406,1,gefriertrocknend,ADJD
138347,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,Glxk4ASQg2,ADJ,A,Ptz2 manuell,missraten,2.745493,1,Missrat,VVFIN
138348,missratend,446280,0,missratend,missratend,False,1,ADJA,ADJA,9n5laNb4dL,ADJ,A,Ptz1,missraten,2.882204,1,Missratend,ADJD
138701,mißraten_MADJ,-1,-1,mißraten,mißraten,False,1,ADJA,ADJA,eHfLf44Icy,ADJ,A,Ptz2 manuell,missraten,1.050000,1,mißraten,VVPP


time: 183 ms


In [220]:
a = pak.analyse_freqs(wiktionary_lemma,['tag','member'], splits=[None,' '])
grid(a)

58 rows


time: 1.1 s


## wiktionary_merkmal ==========

### pflegen

In [221]:
# merkmal_order: Sichert die Reihenfolge der gleichnamigen Merkmale, z.B. mehrere Defs
wiktionary_merkmal = wiktionary_merkmal.reset_index()
wiktionary_merkmal = pak.rank( wiktionary_merkmal, 
                               cols_group = ['lemma_id','merkmal'], 
                               col_score = 'index',
                               find = 'min',
                               col_target = 'merkmal_order')
wiktionary_merkmal = pak.rename_col(wiktionary_merkmal,'index','i')
wiktionary_merkmal = pak.force_index(wiktionary_merkmal,'i')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'merkmal_order','lemma_id')

rank: 0 rows less, now 4656676 rows
time: 13.2 s


In [222]:
wiktionary_merkmal.head(1)

,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
i,,,,,,,,,,,
0,Hallo,1,syn,Gejohle,Gejohle,,NN,1,,7.282582,False


time: 44.1 ms


### Schrott löschen 1
* ohne die Felder lemma und lemma_tag

In [223]:
# zu viele Sonderzeichen
mask1 =  wiktionary_merkmal.data.str.count('[A-ZÖÄÜa-zöäüß0-9]') < wiktionary_merkmal.data.str.count('[^A-ZÖÄÜa-zöäüß0-9]')-1
mask2 =  wiktionary_merkmal.data.str.len() > 2
mask3 = ~wiktionary_merkmal.merkmal.isin(['ipa','rhymes'])
mask4 = ~wiktionary_merkmal.data.isin(['u. s. w.','V. I. P.'])
mask  = mask1  & mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal, mask, 19)
#grid(wiktionary_merkmal, mask)

msg = 'zu viele Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  17 rows
Move 17 rows from 4656676
17 rows out of 6726


time: 16.3 s


In [224]:
# fehlende Klammern
mask1 = wiktionary_merkmal.data.str.count('\(') != wiktionary_merkmal.data.str.count('\)')
mask2 = wiktionary_merkmal.data.str.len() > 1
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0, 10)

msg = 'fehlende_Klammern'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 4.19 s


In [225]:
# Fugenelemente
mask1 =  wiktionary_merkmal.data != '-'
mask2 =  wiktionary_merkmal.data.str.startswith('-')
mask3 =  wiktionary_merkmal.data.str.endswith('-')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

msg = 'Fugenelemente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 3.02 s


### aufsplitten
wiktionary_merkmal wird in drei Teile aufgesplittet:
* wiktionary_merkmal wurzelt in wiktionary_lemma
* wiktionary_lexmerkmal wurzelt in translate_lex.<br>
  translate_lex enthält ja Lexeme (z.B. die Deklinierten Formen). <br>
  wiktionary_lexmerkmal enthält die Merkmale dieser Lexeme.<br>
* translate_gm enthält alle gm-Merkmale, also die Verweise von einer deklinierten Form auf das Lemma.<br>

In [226]:
# Spalten löschen
wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, ['lemma_lower'])

time: 1.42 s


In [227]:
# anschauen: verwaistes und verschollenes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_trash.lemma_id)
mask3 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_fix.lemma_id)
mask4 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
grid(wiktionary_merkmal,mask)

1604 rows out of 4656659


time: 2.63 s


In [228]:
# verwaistes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2

#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','lemma_id'])
grid(a)

43 rows


time: 2.64 s


In [229]:
# verwaistes lemma_id löschen
check_mask(wiktionary_merkmal,mask,14420) 

msg = 'verwaist_lemma_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
#grid( wiktionary_merkmal_trash, msg )

check_mask:  14538 rows
Move 14538 rows from 4656659
time: 3.23 s


In [230]:
# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
grid(a)

43 rows


time: 164 ms


In [231]:
# verwaiste data_id (früher wurden die gelöscht) 
mask1 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask3 =  wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 51123)
grid(wiktionary_merkmal,mask)

check_mask:  51127 rows
51127 rows out of 4642121


time: 2.66 s


In [232]:
# translate_gm abspalten
mask = wiktionary_merkmal.merkmal == 'gm'
check_mask(wiktionary_merkmal, mask, 671313)
wiktionary_merkmal, translate_gm = pak.move_rows(wiktionary_merkmal, mask, msg=None)

check_mask:  671293 rows
Move 671293 rows from 4642121
time: 4 s


In [233]:
# wiktionary_lexmerkmal abspalten
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 =  wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal, mask, 1658317)

wiktionary_merkmal, wiktionary_lexmerkmal = pak.move_rows(wiktionary_merkmal, mask, msg=None)
wiktionary_lexmerkmal = pak.rename_col(wiktionary_lexmerkmal, 'lemma_id','lex_id')

check_mask:  1658238 rows
Move 1658238 rows from 3970828
time: 5.6 s


### pflegen

In [234]:
# suche_debug wiktionary_merkmal
a = search_str(wiktionary_merkmal, suche_debug)
grid(a)

72 rows


time: 2.37 s


In [235]:
# lemma nach wiktionary_merkmal übertragen
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')

update_col: 2312590 cells written into new column
time: 8.64 s


In [236]:
# lemma_tag nach wiktionary_merkmal übertragen
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2312590 cells written into existing column
time: 12.1 s


In [237]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 339 ms


### Schrott löschen 2
* diesmal mit den Feldern lemma und lemma_tag

In [238]:
# beginnt mit Sonderzeichen
mask1 =  wiktionary_merkmal.data.str.len() > 1
mask2 = ~wiktionary_merkmal.data.str[0].str.isalpha()  
mask3 = ~wiktionary_merkmal.data.str[0].str.isnumeric()    
mask4 = ~wiktionary_merkmal.lemma_tag.isin([ '$(', '$.', '$,', 'ZFIX NE', 'ZFIX', 'CARD' ])
mask5 =  wiktionary_merkmal.is_lex == True
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5    
check_mask( wiktionary_merkmal, mask, 23)

msg = 'beginnt mit Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg ) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  24 rows
Move 24 rows from 2312590
24 rows out of 21288


time: 4.68 s


In [239]:
# nicht_erlaubte_sonderzeichen 
mask1 = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen)   
mask2 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2 
check_mask( wiktionary_merkmal, mask, 2)

msg = 'nicht_erlaubte_sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  2 rows
Move 2 rows from 2312566
2 rows out of 21290


time: 3.37 s


In [240]:
# zu viele Wörter 
mask1 =  wiktionary_merkmal.data.str.split().str.len() > 3
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  &  mask3 
check_mask( wiktionary_merkmal, mask, 0)

msg = 'zu viele Wörter'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 4.36 s


In [241]:
# Verbotene Fragmente in data, die nicht im lemma auftreten
verbotene_fragmente = ['Konjugation','Genitiv','Dativ','(',')',':','»','<','!','#',',',';','!','?']
wiktionary_merkmal['error'] = None
for verboten in verbotene_fragmente:
    mask1 =  wiktionary_merkmal.data.str.contains(verboten, regex=False)    
    mask2 = ~wiktionary_merkmal.lemma.str.contains(verboten, regex=False)  
    mask3 =  wiktionary_merkmal.data.str.len() > 1       
    mask4 =  wiktionary_merkmal.is_lex == True   
    mask5 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
    mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
    wiktionary_merkmal.loc[ mask, 'error'] = 'Verbotenes Fragment ' + verboten

time: 28.8 s


In [242]:
mask = ~wiktionary_merkmal.error.isnull()  
check_mask( wiktionary_merkmal, mask, 0, 5)
#grid( wiktionary_merkmal, mask)

wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, 'error' )
msg = 'Verbotene Fragmente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 925 ms


In [243]:
# Überflüssige lemmagleiche Lexeme löschen
mask1  = wiktionary_merkmal.is_lex == True
mask2  = wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3A = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask3B = wiktionary_merkmal.merkmal.isin( ['lex'] )
mask  = mask1  &  mask2  &  (mask3A | mask3B) 
check_mask( wiktionary_merkmal, mask, 205)

msg = 'lemmagleiche Lexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  205 rows
Move 205 rows from 2312564
205 rows out of 21495


time: 3.12 s


In [244]:
# Überflüssige lemmagleiche Nichtlexeme löschen
mask1 =  wiktionary_merkmal.is_lex == False
mask2 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3 = ~wiktionary_merkmal.merkmal.isin( ['syllables','Hilfsverb'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 1014)

msg = 'lemmagleiche Nichtlexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  989 rows
Move 989 rows from 2312359
989 rows out of 22484


time: 2.83 s


In [245]:
# Benannte Lexeme
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal != 'lex'
mask  = mask1  &  mask2
benannte_lexeme = wiktionary_merkmal[mask]

time: 720 ms


In [246]:
# Überflüssige lex: Die gibt es schon unter einem konkreteren Merkmal
# Das sind nicht so viele, weil die alle in translate_lex sind
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.merkmal == 'lex'
mask3 = pak.isin( wiktionary_merkmal, benannte_lexeme, on=['lemma','data'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 21)

del benannte_lexeme
msg = 'Überflüssige lex'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  21 rows
Move 21 rows from 2311370
21 rows out of 22505


time: 15.4 s


In [247]:
# TRUNC löschen
mask =  wiktionary_merkmal.lemma_tag == 'TRUNC'
check_mask( wiktionary_merkmal, mask, 0, 4)

msg = 'TRUNC'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  2 rows
Move 2 rows from 2311349
6 rows out of 22507


time: 2.56 s


In [248]:
#search_str(wiktionary_lemma, 'http')

time: 44.8 ms


In [249]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.lemma.isin(suche_debug)
mask = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

No rows, mask filters them all away
time: 63.2 ms


In [250]:
# suche_debug
#mask = wiktionary_merkmal.data.isin(suche_debug)
mask = wiktionary_merkmal.lemma.isin(suche_debug)
#mask = maskA | maskB
grid(wiktionary_merkmal,mask)

58 rows out of 2311347


time: 337 ms


In [251]:
# Nochmal sicherstellen: Alle Datensätze in wiktionary_merkmal haben einen passenden DS in wiktionary_lemma
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 599 ms


In [252]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, ['lemma_id','merkmal','is_lex','data_id','data','data_tag','lemma','lemma_tag','num','meta',])
wiktionary_merkmal.head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score
i,,,,,,,,,,,,
0,Hallo,syn,False,Gejohle,Gejohle,,Hallo,NN,1,,1,7.282582


time: 83.8 ms


### Grundformen anfügen

In [253]:
# Grundformen anfügen (merkmal 'lemma')
spalten = ['lemma_id', 'lemma', 'tag_0', 'score' ]
grundformen = wiktionary_lemma[spalten].copy()
grundformen = pak.rename_col(grundformen, 'tag_0', 'lemma_tag')
grundformen['merkmal']  = 'lemma'
grundformen['is_lex']   = True
grundformen['data']     = grundformen.lemma.copy()
grundformen['data_id']  = grundformen.lemma_id.copy()
grundformen['data_tag'] = grundformen.lemma_tag.copy()
grundformen['meta']     = ''
grundformen['num']      = ''


# Spaltenvergleich
bpy.minivenn( grundformen.columns, wiktionary_merkmal.columns, format='diff')

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, grundformen)
#grid(grundformen)

165873 rows added, now a total of 2477220
time: 1.12 s


In [254]:
# Sonderfall Mag.a löschen
mask = wiktionary_merkmal.data_id == 'Mag.a'
check_mask(wiktionary_merkmal,mask,0,2)

msg = 'Sonderfall Mag.a' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  2 rows
Move 2 rows from 2477220
2 rows out of 22509


time: 2.58 s


In [255]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 3.98 s


In [256]:
# Verwaiste Merkmale: Gibt es nicht, weil die abgetrennt oder gelöscht wurden
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#grid( wiktionary_merkmal, mask, )

# löschen
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'verwaist4')
grid(wiktionary_merkmal_trash, color='green')

check_mask:  0 rows
No rows moved
22509 rows


time: 798 ms


In [257]:
# fillna
wiktionary_merkmal['meta']          = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']      = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']     = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']           = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['merkmal_order'] = wiktionary_merkmal.merkmal_order.fillna(0)

time: 1.17 s


In [258]:
wiktionary_merkmal

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score
0,Hallo,syn,False,Gejohle,Gejohle,,Hallo,NN,1,,1.0,7.282582
1,Subfamilia,def,False,hierarchisch,hierarchisch,,Subfamilia,NN,1,Biologie,1.0,5.028015
2,Subfamilia,def,False,Systematik,Systematik,,Subfamilia,NN,1,Biologie,2.0,5.028015
3,Subfamilia,def,False,Familie,Familie,,Subfamilia,NN,1,Biologie,3.0,5.028015
4,Subfamilia,syn,False,Unterfamilie,Unterfamilie,,Subfamilia,NN,1,,1.0,5.028015
...,...,...,...,...,...,...,...,...,...,...,...,...
2477215,sstr_A,lemma,True,sstr_A,sstr,KOMBI,sstr,KOMBI,,,0.0,0.300000
2477216,u_s_A,lemma,True,u_s_A,u s,KOMBI,u s,KOMBI,,,0.0,0.300000
2477217,u._s._A,lemma,True,u._s._A,u. s.,KOMBI,u. s.,KOMBI,,,0.0,0.300000
2477218,u.s._A,lemma,True,u.s._A,u.s.,KOMBI,u.s.,KOMBI,,,0.0,0.300000


time: 61.8 ms


In [259]:
# update_col aus wiktionary_lemma
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag_0', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2477218 cells written into existing column
update_col: 2477218 cells written into existing column
time: 25.4 s


In [260]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 354 ms


In [261]:
#pak.analyse_cols(wiktionary_merkmal)

time: 30.1 ms


### Leer oder ultrakurz

In [262]:
# Prüfen: Keine Leerstrings
spalten = ['merkmal','data','lemma','lemma_tag']
for s in spalten:
    print(s)
    mask = wiktionary_merkmal[s] == ''
    check_mask( wiktionary_merkmal, mask, 0)

merkmal
check_mask:  0 rows
data
check_mask:  0 rows
lemma
check_mask:  0 rows
lemma_tag
check_mask:  0 rows
time: 1.05 s


In [263]:
# noSteig ermitteln und löschen
# data ist nur '-'
mask1 = wiktionary_merkmal.data.str.len() <= 1
mask2 = wiktionary_merkmal.lemma.str.len() > 1
mask3 = wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask4 = wiktionary_merkmal.is_lex == True
mask  =  mask1  &  mask2  &  mask3  &  mask4 
noSteig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 15369)

msg = 'noSteig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  15367 rows
Move 15367 rows from 2477218
15367 rows out of 37876


time: 3.05 s


In [264]:
# noSteig in wiktionary_lemma kennzeichnen
mask1 =  wiktionary_lemma.lemma_id.isin(noSteig)
mask2 = ~wiktionary_lemma.member.str.contains('noSteig')
mask3 =  wiktionary_lemma.tagZ == 'ADJ'
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 146)

wiktionary_lemma.loc[mask,'member'] += ' noSteig' 
grid( wiktionary_lemma, mask )

check_mask:  149 rows
149 rows out of 165873


time: 267 ms


In [265]:
# Genus korrigieren
genus_m = ['Stromboli','Micky_Maus','Jesus_Christus','Jehova','Daniel_Düsentrieb','Dagobert_Duck','Tacken','Donald_Duck']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_m)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_m))

wiktionary_merkmal.loc[mask,'data'] = 'm'
wiktionary_merkmal[mask]

check_mask:  8 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score
1013745,Jesus_Christus,Genus,False,,m,,Jesus Christus,NE,,,1.0,5.756395
1038153,Stromboli,Genus,False,,m,,Stromboli,NE,,,1.0,12.511571
1040767,Tacken,Genus,False,,m,,Tacken,NN,,,1.0,15.317905
1058216,Dagobert_Duck,Genus,False,,m,,Dagobert Duck,NE,,,1.0,6.115713
1058222,Daniel_Düsentrieb,Genus,False,,m,,Daniel Düsentrieb,NE,,,1.0,3.136419
1058336,Micky_Maus,Genus,False,,m,,Micky Maus,NE,,,1.0,5.801462
1058337,Donald_Duck,Genus,False,,m,,Donald Duck,NE,,,1.0,3.399513
1072796,Jehova,Genus,False,,m,,Jehova,NE,,,1.0,13.885506


time: 726 ms


In [266]:
# Genus korrigieren
genus_f = ['Arminia','Trauerwoche']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_f)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_f))

wiktionary_merkmal.loc[mask,'data'] = 'f'
wiktionary_merkmal[mask]

check_mask:  2 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score
1027520,Trauerwoche,Genus,False,,f,,Trauerwoche,NN,,,1.0,5.550548
1083628,Arminia,Genus,False,,f,,Arminia,NE,,,1.0,4.154689


time: 796 ms


In [267]:
# Genus n falsch
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 =  wiktionary_merkmal.data == 'n'
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['NN','NE'])
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0, 3)

msg = 'Genus n falsch' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  1 rows
Move 1 rows from 2461851
1 rows out of 37877


time: 2.82 s


In [268]:
# Ungültige Genus löschen
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 = ~wiktionary_merkmal.data.isin(['m','f','n'])
mask  =  mask1  &  mask2 
genus_ungültig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 700)

msg = 'Genus ungültig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  697 rows
Move 697 rows from 2461850
697 rows out of 38574


time: 2.64 s


In [269]:
# genus_ungültig in wiktionary_lemma betrachten
# Sind da Kandidaten für manuelle Löschung oder Genuss setzen dabei?
mask1 =  wiktionary_lemma.lemma_id.isin(genus_ungültig)
mask2 = ~wiktionary_lemma.member.str.contains('noSg')
mask3 = ~wiktionary_lemma.member.str.contains('noPl')
mask4 = ~wiktionary_lemma.lemma_id.str.endswith('_V')
mask5 = ~wiktionary_lemma.member.str.contains('Person_vorname')
mask6 = ~wiktionary_lemma.member.str.contains('Geo')
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask( wiktionary_lemma, mask, 7)

spalten = ['lemma_id','tag','score','member']
a = wiktionary_lemma[mask][spalten].sort_values(['tag'])
grid( a )

check_mask:  7 rows
7 rows


time: 402 ms


In [270]:
# data ist nur '-'
mask1 =  wiktionary_merkmal.data == '-'
mask2 =  wiktionary_merkmal.lemma_id != '-_M'
#mask3 = ~wiktionary_merkmal.lemma_tag.isin(['VVFIN'])
mask = mask1  &  mask2 # &  mask3
check_mask( wiktionary_merkmal, mask, 919)

msg = 'data ist nur -' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  1012 rows
Move 1012 rows from 2461153
1012 rows out of 39586


time: 2.9 s


In [271]:
# superkurz >> alles bleibt wie es ist
mask1 =  wiktionary_merkmal.data.str.len() <= 1
mask2 =  wiktionary_merkmal.lemma.str.len() > 1   
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2  &  mask3     

#check_mask( wiktionary_merkmal, mask, 1292)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='superkurz' ) 

pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data','lemma_id','lemma_tag'])
#grid( wiktionary_merkmal, mask )

,merkmal,merkmal_count,merkmal_percent,data,data_count,lemma_id,lemma_id_count,lemma_tag,lemma_tag_count
0,Genus,96681,100.0,"[f, m, n]","[41313, 35974, 19394]","[Dingsda, Release, Mockumentary, Dingsbums, Qu...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[NN, NE, CARD, PTKANT]","[91141, 5322, 217, 1]"
1,rhymes,33,0.0,"[a, ɛ, ɔ]","[23, 5, 5]","[Abasa, Ucha, Isba, voila, Sobriquet, Yue, Nia...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[NN, ITJ, NE, ADV]","[17, 9, 6, 1]"
2,lex,10,0.0,"[3, 1, 7, 2, 8, 4, 0, 9, 5, 6]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[drei, eins, sieben, zwei, acht, vier, null_1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",CARD,10


time: 1.6 s


### Allgemein

In [272]:
# Explizit gesetzte Tags werden Merkmale
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data_tag != ''
mask3 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask4 = ~((wiktionary_merkmal.data_tag == 'ADJA')  &   (wiktionary_merkmal.lemma_tag == 'VVFIN')) # Partizipien
mask5 = wiktionary_merkmal.merkmal == 'lex'
mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask,1,50)

# Merkmal erstellen
wiktionary_merkmal.loc[mask,'merkmal'] = wiktionary_merkmal[mask].data_tag

grid(wiktionary_merkmal,mask)
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
#grid(a)

check_mask:  8 rows
8 rows out of 2460141


time: 1.45 s


In [273]:
# Überblick wiktionary_merkmal
a = pak.analyse_freqs(wiktionary_merkmal, ['merkmal', 'data'] )
grid(a)

74 rows


time: 6.9 s


In [274]:
# alle Merkmale sind in plan_merkmal erfasst
mask = ~a.merkmal.isin(plan_merkmal.merkmal)
check_mask(a,mask,0)

check_mask:  0 rows
time: 54.7 ms


In [275]:
grid(a,mask, color='green')

No rows, mask filters them all away
time: 56.4 ms


In [276]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
search_str(wiktionary_merkmal[mask], suche_debug)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score
533958,missraten,alt,True,mißraten_MADJ,mißraten,,missraten,VVFIN,,,1.0,4.472594
903969,missraten,Ptz1,True,missratend,missratend,,missraten,VVFIN,,,1.0,4.472594
925517,gefriertrocknen,Ptz1,True,gefriertrocknend,gefriertrocknend,,gefriertrocknen,VVFIN,,,1.0,14.422655
1507704,gefriertrocknen,Imp Sg,True,gefriertrockne,gefriertrockne,,gefriertrocknen,VVFIN,,,1.0,14.422655
1508528,missraten,Imp Sg,True,missrate,missrate,,missraten,VVFIN,,,1.0,4.472594
1854589,gefriertrocknen,Imp Pl,True,gefriertrocknet,gefriertrocknet,,gefriertrocknen,VVFIN,,,1.0,14.422655
1862182,missraten,Imp Pl,True,missratet,missratet,,missraten,VVFIN,,,1.0,4.472594
1866785,gefriertrocknen,Präsens du,True,gefriertrocknest,gefriertrocknest,,gefriertrocknen,VVFIN,,,1.0,14.422655
1873654,missraten,Präsens du,True,missrätst,missrätst,,missraten,VVFIN,,,1.0,4.472594
1882679,missraten,Präsens ersiees,True,missrät,missrät,,missraten,VVFIN,,,1.0,4.472594


time: 2.32 s


In [277]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

time: 1.1 s


### Wortarten -------

#### Partizipien 

In [278]:
# Alle Adjektive
mask = wiktionary_lemma.tagZZ == 'A'
adj = wiktionary_lemma[mask].lemma
#adj

time: 72.1 ms


In [279]:
# Alle Partizipien
mask = wiktionary_merkmal.merkmal == 'Ptz1'
ptz1 = wiktionary_merkmal[mask]
mask = wiktionary_merkmal.merkmal == 'Ptz2'
ptz2 = wiktionary_merkmal[mask].copy()
ptz_all = pak.add_rows(ptz1, ptz2)
#ptz_all

13923 rows added, now a total of 27281
time: 431 ms


In [280]:
# Alle Ptz1 sind auch als Adjektiv registriert
mask = ~ptz1.data.isin(adj)
check_mask(ptz1,mask,0)

check_mask:  0 rows
time: 36.4 ms


In [281]:
# Die meisten Ptz2 sind auch als Adjektiv registriert
mask = ~ptz2.data.isin(adj)
check_mask(ptz2,mask,0,120)
grid(ptz2,mask)

check_mask:  104 rows
104 rows out of 13923


time: 85.8 ms


#### Verben

In [282]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows
time: 2.64 s


In [283]:
# Übersicht: Alle V-Tags in wiktionary_lemma
mask = wiktionary_lemma.tag.str.startswith('V')
a = pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])
grid(a)

3 rows


time: 176 ms


In [284]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

6 rows


time: 1.95 s


In [285]:
# Infinitiv
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'lemma'
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 13605)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'INF'
grid(wiktionary_merkmal,mask)

check_mask:  13605 rows
13605 rows out of 2460141


time: 3.14 s


In [286]:
# Imperativ
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal.str.contains('Imp')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 36672)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'IMP'
grid(wiktionary_merkmal,mask)

check_mask:  36672 rows
36672 rows out of 2460141


time: 3.77 s


In [287]:
# Imperativ: VMIMP gibt es gar nicht
mask = wiktionary_merkmal.data_tag == 'VMIMP'
check_mask(wiktionary_merkmal,mask, 3)
wiktionary_merkmal.loc[mask,'data_tag'] = 'VVIMP'
#grid(wiktionary_merkmal,mask)

check_mask:  3 rows
time: 330 ms


In [288]:
# Partizip2
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz2' 
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,13885)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'PP'
grid(wiktionary_merkmal,mask)

check_mask:  13923 rows
13923 rows out of 2460141


time: 2.98 s


In [289]:
# Partizip1 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz1'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 26716)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
grid(wiktionary_merkmal,mask)

check_mask:  13358 rows
13358 rows out of 2460141


time: 1.91 s


In [290]:
# (1) Gerundivum: Schon fertig
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
grid(wiktionary_merkmal,mask)

16065 rows out of 2460141


time: 486 ms


In [291]:
# Adjektivformen der Verben: ADJA ergänzen 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask3 = ~wiktionary_merkmal.data_tag.str.contains('ADJA')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal, mask, 27243)

adjaformen = wiktionary_merkmal[mask].copy()
adjaformen['data_tag'] = 'ADJA'
adjaformen['merkmal_order'] += 10

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,adjaformen)
grid(adjaformen)

check_mask:  27281 rows
27281 rows added, now a total of 2487422
27281 rows


time: 2.3 s


In [292]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 4.05 s


In [293]:
# Übersicht: Adjektivformen der Verben
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2','Gerundivum'])
mask  = mask1  &  mask2
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag','data'] )
a

,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count,data,data_count
0,Ptz2,27846,39.4,"[ADJA, VVPP, VMPP, VAPP]","[13923, 13909, 8, 6]","[gestrippt, geschert, erschrocken, gewahrt, ge...","[12, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,..."
1,Ptz1,26716,37.8,"[ADJA, VVFIN, VMFIN, VAFIN]","[13358, 13347, 7, 4]","[bedeutend, belastend, nachtragend, rezidivier...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
2,Gerundivum,16065,22.7,ADJA,16065,"[aufzuwärmende, aufzuwärmendes, aufzuwärmender...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


time: 423 ms


In [294]:
# VVIZU in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal != 'lemma'
mask4 = wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask5 = wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask, 8)

#wiktionary_merkmal.loc[mask,'data_tag']     = 'VVIZU'
#wiktionary_merkmal.loc[mask,'meta']    = '' 
#wiktionary_merkmal.loc[mask,'merkmal'] = 'Erweiterter_Infinitiv' 
grid(wiktionary_merkmal,mask)

check_mask:  8 rows
8 rows out of 2487422


time: 5.03 s


In [295]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

35 rows


time: 858 ms


In [296]:
# TODO_VVIZU
# VVIZU in translate_lex
mask1 = translate_lex.lemma_tag.str.startswith('V')
mask2 = translate_lex.data.str.replace('zu','') == translate_lex.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3 = translate_lex.data.str.len() == translate_lex.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask, 5473)

vvizuformen = translate_lex[mask].copy()

check_mask:  5489 rows
time: 1.26 s


In [297]:
# vvizuformen anpassen und anfügen
vvizuformen = pak.drop_cols(vvizuformen,['lemma_tagZZ','lemma_lower','lemma_score'])
vvizuformen = pak.rename_col(vvizuformen,'member',    'meta'     )
vvizuformen = pak.rename_col(vvizuformen,'data_score', 'score'     )
vvizuformen['merkmal']       = 'VVIZU'
vvizuformen['is_lex']        = True
vvizuformen['data_tag']      = 'VVIZU'
vvizuformen['num']           = ''
vvizuformen['merkmal_order'] = 0

# Spaltenvergleich
bpy.minivenn( vvizuformen.columns, wiktionary_merkmal.columns, format='diff')

[{'data_tagZZ'}, set()]

time: 57.2 ms


In [298]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, vvizuformen)
grid(vvizuformen)

5489 rows added, now a total of 2492911
5489 rows


time: 1.13 s


In [299]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 4.35 s


In [300]:
# Kontrolle: Keine anderen Infinitiv-like
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask4 = ~wiktionary_merkmal.merkmal.isin( ['lemma','Ptz2'] )
mask  = mask1  &  mask2  &  mask3  &  mask4

check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='Infinitiv-like' ) 
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows
time: 1.49 s


In [301]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows
time: 2.49 s


In [302]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3

#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag

time: 1.21 s


In [303]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

13 rows


time: 2.03 s


#### Nomen

In [304]:
# Übersicht über alle explizit gesetzten tags (Tagquelle: lexeme_manuell)
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = wiktionary_merkmal.data_tag != ''
mask  = mask1  &  mask2  &  mask3
#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
(a)

,lemma_tag,lemma_tag_count,lemma_tag_percent,data_tag,data_tag_count
0,NN,95377,88.9,NN,95377
1,NE,11884,11.1,NE,11884


time: 1.51 s


In [305]:
# Keine Doppeltags
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 =  wiktionary_merkmal.lemma_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows
time: 1.85 s


In [306]:
# Irgendwelche unbekannten tags?
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = ~wiktionary_merkmal.data_tag.isin( ['NN','NE',''] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows
time: 959 ms


In [307]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 761247)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#grid(wiktionary_merkmal,mask)

check_mask:  773981 rows
time: 2.19 s


In [308]:
# Nomen analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask  = mask1  &  mask2  
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

27 rows


time: 4.09 s


#### Rest

In [309]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.is_lex    == True
mask2 = wiktionary_merkmal.data_tag  == ''
mask3 = ~wiktionary_merkmal.lemma_tag.str[0].isin(['V','N'])
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 30194)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#a = pak.analyse_freqs( wiktionary_merkmal[mask], ['lemma_tag','data_tag'] )
#grid(a)

check_mask:  30225 rows
time: 1.78 s


#### zweittag

In [310]:
spalten = ['lemma_id','tag_1']
mask = wiktionary_lemma.tag.str.contains(' ')
check_mask( wiktionary_lemma, mask, 150)

zweitags = wiktionary_lemma[mask][spalten]

check_mask:  152 rows
time: 78.7 ms


In [311]:
# Merkmale anfügen mit zweittag
mask1 = wiktionary_merkmal.lemma_id.isin(zweitags.lemma_id)
mask2 = wiktionary_merkmal.lemma_tag == wiktionary_merkmal.data_tag
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 513)

zweitlexeme = wiktionary_merkmal[mask].copy()
zweitlexeme = pak.update_col(zweitlexeme, zweitags, on='lemma_id', col='tag_1', col_rename='data_tag')
zweitlexeme.meta = (zweitlexeme.meta + ' zweittag').str.strip()
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,zweitlexeme)
grid(zweitlexeme)

check_mask:  518 rows
update_col: 518 cells written into existing column
518 rows added, now a total of 2493429
518 rows


time: 1.92 s


In [312]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 4.21 s


#### Adjektive

In [313]:
# Merkmal Positiv ist sinnlos
mask1 = wiktionary_merkmal.merkmal == 'Positiv'
mask2 = wiktionary_merkmal.data != wiktionary_merkmal.lemma
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0,10)
wiktionary_merkmal[mask]

check_mask:  1 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,score,data_tagZZ
1946979,Wiesbadner,Positiv,True,Wiesbadener,Wiesbadener,ADJA,Wiesbadner,ADJA,,,1.0,2.718636,<NA>


time: 752 ms


In [314]:
# Merkmal Positiv löschen
mask = wiktionary_merkmal.merkmal == 'Positiv'
check_mask( wiktionary_merkmal, mask, 13301)

msg = 'Merkmal Positiv ist sinnlos' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  13303 rows
Move 13303 rows from 2493429
13303 rows out of 52889


time: 2.95 s


In [315]:
# Ist kein Adjektiv
mask1 =  wiktionary_merkmal.data_tag.isin(['ADJA','ADJD'])
mask2 = ~wiktionary_merkmal.merkmal.isin(['lemma','Ptz1','Ptz2','Gerundivum','Positiv','Komparativ','Superlativ','alt','alt_1','alt_2','lexAlt','lex','alt_abk'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 16)

msg = 'Ist kein Adjektiv' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  20 rows
Move 20 rows from 2480126
20 rows out of 52909


time: 2.9 s


In [316]:
# Adjektiv-only Merkmale
mask1 = ~wiktionary_merkmal.data_tag.isin(['ADJA','ADJD','ADV'])
mask2 =  wiktionary_merkmal.merkmal.isin(['Gerundivum','Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, )

#a = wiktionary_merkmal[mask].sort_values(['data_tag','lemma_id'])
#grid(a)

msg = 'Adjektiv-only Merkmale' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  4 rows
Move 4 rows from 2480106
4 rows out of 52913


time: 2.99 s


In [317]:
# Steigerbare Adverbien #_TODO_ADJ_STEIGERBAR
mask1 =  wiktionary_merkmal.data_tag == 'ADV'
mask2 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask4 = ~wiktionary_merkmal.lemma.isin(['sehr','gern','gerne','oft', 'bald','wohl'])
    
mask  = mask1  &  mask2  &  mask3  &  mask4 
check_mask( wiktionary_merkmal, mask, )

a = wiktionary_merkmal[mask].sort_values(['merkmal','data_tag','lemma_id'])
grid(a)

# 'sehr', 'mehr','am meisten'
# 'gern', 'lieber','am liebsten'
# 'bald', 'eher','am ehesten'
## 'oft',  'öfter','am öftesten'  # drin
## 'wohl', 'besser','am besten'   # drin

#msg = 'Irregeleitete Adjektive ' 
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
#grid( wiktionary_merkmal_trash, msg )

check_mask:  18 rows
18 rows


time: 919 ms


#### analysieren

In [318]:
# Merkmale analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask  = mask1   
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


time: 4.09 s


In [319]:
# Keine Verwaisten lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='verwaist' ) 
#grid( wiktionary_merkmal, mask)

check_mask:  0 rows
time: 601 ms


### Felder -------

#### data

In [320]:
# Prüfen: Keine mehrfache Spaces
mask = wiktionary_merkmal.data.str.contains('  ')
check_mask(wiktionary_merkmal,mask,0)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

check_mask:  0 rows
time: 2.19 s


In [321]:
# Prüfen: nicht leer
mask = wiktionary_merkmal.data.str.len() == 0
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='leer3' ) 

check_mask:  0 rows
time: 317 ms


In [322]:
# '⇀' ist nur in syllables
mask1 =  wiktionary_merkmal.data.str.contains('⇀')
mask2 = ~wiktionary_merkmal.merkmal.isin(['syllables'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 803 ms


In [323]:
# Apostroph anschauen
mask =  wiktionary_merkmal.data.str.contains("'")
check_mask(wiktionary_merkmal,mask,40)
grid(wiktionary_merkmal,mask)

check_mask:  32 rows
32 rows out of 2480102


time: 831 ms


In [324]:
# Komma anschauen
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,33)
#grid(wiktionary_merkmal,mask)

check_mask:  33 rows
time: 1.91 s


#### num

In [325]:
# num ist bei den meisten Lexemen sinnlos
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.num.str.len() > 0
#mask3 = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  #&  mask3

check_mask( wiktionary_merkmal, mask, 18770)
wiktionary_merkmal.loc[mask,'num'] = ''   
#grid(wiktionary_merkmal,mask, color='green')

check_mask:  18776 rows
time: 408 ms


In [326]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['num','data',])

time: 34.9 ms


#### meta

In [327]:
# remove_words
d = [ '(', ')', 'auch','nicht','bis','Adjektive','Adjektiv','Kombination','Substantive','Substantiv','allgemein','Adverbien','Verb','Adverbien','Adverb','andere','über']       
wiktionary_merkmal['meta'] = pak.remove_words( wiktionary_merkmal.meta, d )

time: 1.92 s


In [328]:
# replace_str
r = {       
    'seltener':'selten',
    'Schweiz':'schweizerisch',    
    'Österreich':'österreichisch', 
    'Süddeutschland':'süddeutsch',    
    'bairisch':'bayrisch',        
    'bairisch':'bayrisch',       
     
    }
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, r )

time: 3.27 s


In [329]:
# meta Überblick Lexeme
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
grid(a)

163 rows


time: 8.57 s


In [330]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

time: 1.63 s


In [331]:
# meta Überblick Nichtlexeme
mask = wiktionary_merkmal.is_lex == False
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
grid(a)

15717 rows


time: 12.7 s


In [332]:
# Seltene meta löschen
mask = a.meta_count < 2
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == False
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

time: 6.47 s


#### tag-Felder

In [333]:
# data_tagZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZ', col_rename='data_tagZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZ', col_rename='data_tagZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZ','data_tag')

update_col: 1139833 cells written into new column
update_col: 1340269 cells written into existing column
time: 22.5 s


In [334]:
# data_tagZZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZZ', col_rename='data_tagZZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='data_tagZZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZZ','data_tagZ')

update_col: 1139833 cells written into existing column
update_col: 1340269 cells written into existing column
time: 23.2 s


In [335]:
# Wieder löschen wenn kein Lexem
mask = wiktionary_merkmal.is_lex != True
wiktionary_merkmal.loc[mask,'data_tagZ']  = ''
wiktionary_merkmal.loc[mask,'data_tagZZ'] = ''

time: 150 ms


In [336]:
# data_tagZ und data_tagZZ überall gesetzt
mask1  = wiktionary_merkmal.is_lex == True
mask2A = wiktionary_merkmal.data_tagZ  == ''
mask2B = wiktionary_merkmal.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask, 0)

check_mask:  0 rows
time: 272 ms


In [337]:
# alle verwendeten tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['data_tag','merkmal'], splits=[' ', None] )
grid(a)

45 rows


time: 6.72 s


In [338]:
# Gibt es unregistrierte tags?
alle_realen_tags = set(alle_tags) - {'AFIX','ZFIX','TODO','LEX_A','LEX_N', 'APPRART','MFIX','NNE','XY','_SP',}
seltsame_tags = set(a.data_tag) - alle_realen_tags
assert len(seltsame_tags) == 0

#mask = wiktionary_merkmal.tag.isin(seltsame_tags)
#wiktionary_merkmal[mask]

time: 66.6 ms


In [339]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'TRUNC'
mask  =  mask1  &  mask2

grid(wiktionary_merkmal,mask, color='green')

3 rows out of 2480102


time: 352 ms


In [340]:
# Alle merkmale und ihre tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


time: 4.03 s


In [341]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'ADV'
mask3 =  wiktionary_merkmal.merkmal == 'Superlativ'
mask  =  mask1  &  mask2  &  mask3

grid(wiktionary_merkmal,mask, color='green')

15 rows out of 2480102


time: 610 ms


In [342]:
# lemma_tagZZ schreiben
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tagZZ','lemma_tag')
wiktionary_merkmal['lemma_tagZZ'] = wiktionary_merkmal.lemma_tagZZ.fillna('')

time: 11.3 s


In [343]:
# check
mask1 = wiktionary_merkmal.lemma_tag != ''
mask2 = wiktionary_merkmal.lemma_tagZZ == ''
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 516 ms


In [344]:
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask].head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,score
46,Subspezies,alt_2,True,Subspecies,Subspecies,NN,NOM,N,Subspezies,NN,N,,,1.0,7.563946


time: 590 ms


#### hunspell

In [345]:
# Hunspell instance
from hunspell import Hunspell
hun = Hunspell('de_DE', hunspell_data_dir='/usr/share/hunspell') 

time: 140 ms


In [346]:
# verify_hunspell
# liefert 1=verifiziert, 0=nicht verifiziert, -1=Besonderes Unicode-Zeichen
wiktionary_merkmal['hunspell'] = -2
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask] = wiktionary_merkmal[mask].apply(verify_hunspell, col_check='data', col_result='hunspell', hunspellinstance=hun, axis=1)
#wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal, verbose=False)  

time: 2min 16s


In [347]:
# prüfen: alle erfasst?
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell < 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 60.5 ms


In [348]:
# abwerten
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell == 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'score'] *= 0.1
grid(wiktionary_merkmal,mask)

check_mask:  504494 rows
504494 rows out of 2480102


time: 7.42 s


## translate_lex aufbauen ==========
* translate_lex enthält derzeit alle Wiktionary-Seitentitel, die keine Lemmas sind.
* Diverse Listen werden hier zusätzlich angefügt.
* Nach jeder Anfügung werden Dups entfernt.

In [349]:
# set_translate_lex_score 
#from x32_Funktionen import *
translate_lex['score'] = 0
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

644785 Datensätze werden neu bewertet
time: 10.9 s


In [350]:
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_lex)

rank: 0 rows less, now 644785 rows
update_col: 644785 cells written into existing column
update_col: 644693 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into existing column
update_col: 644693 cells written into new column
update_col: 644693 cells written into existing column
time: 33 s


In [351]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

35 rows


time: 873 ms


In [352]:
# data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

assert translate_lex.data_id.is_unique

#a = pak.same_but_different(translate_lex,same='data_id',different='data')
#assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 333 ms


### translate_gm
* gm-Merkmale sind die Merkmale auf den Wiktionary-Seiten, die keine Lemmas sind. Also diejenigen, die unter wiktionary_nolemma bzw. translate_lex gespeichert sind.
* Daher weisen ihre lemma_id nicht auf ein Lemma, sondern auf ein Lexem. lemma_id und data_id müssen also hier vertauscht werden.

In [353]:
# translate_lex 
# Dies sind die Seiten, deren Merkmale translate_gm enthält.
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
(search_str(translate_lex, ['besonne']))

,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,score,lemma_lower
532988,besonne,T,besonne,,1.146163,wiki Konjugierte_Form Verb,besonnen_2,L,besonnen,VVFIN,V,2.454933,2.700163,besonnen


time: 804 ms


In [354]:
# lemma und data vertauschen
translate_gm = pak.rename_col(translate_gm, 'lemma_id',    'data_id_neu')
translate_gm = pak.rename_col(translate_gm, 'data_id',     'lemma_id')
translate_gm = pak.rename_col(translate_gm, 'data_id_neu', 'data_id')
translate_gm = pak.rename_col(translate_gm, 'data',        'lemma')

time: 1.1 s


In [355]:
# lemma_id nachschlagen (und dabei korrigieren!)
translate_gm = pak.update_col(translate_gm, translate_lex, on='data_id', col='lemma_id' ) 

update_col: 644714 cells written into existing column
time: 3.99 s


In [356]:
# translate_gm umformen
translate_gm = pak.rename_col(translate_gm, 'merkmal', 'member')
translate_gm['member'] = (translate_gm.member + ' ' + translate_gm.meta).str.strip()

translate_gm = pak.drop_cols(translate_gm, ['is_lex','data_tag','num','meta',])

time: 883 ms


In [357]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

29 rows


time: 130 ms


In [358]:
from x32_Funktionen import *
translate_gm = pflege_translate_tabelle(translate_gm, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_gm)

rank: 0 rows less, now 644785 rows
update_col: 644714 cells written into new column
update_col: 671200 cells written into existing column
update_col: 26579 cells written into existing column
update_col: 26579 cells written into new column
update_col: 671200 cells written into new column
update_col: 671200 cells written into new column
renaming score --> data_score
time: 40.1 s


In [359]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

29 rows


time: 283 ms


In [360]:
assert translate_gm.lemma_home.nunique() == 2 # stand mal auf 1
assert translate_gm.data_home.nunique() == 2

time: 109 ms


In [361]:
pak.analyse_freqs(translate_gm, ['data_home','lemma_home'])

,data_home,data_home_count,data_home_percent,lemma_home,lemma_home_count
0,T,644714,96.0,"[L, ]","[644622, 92]"
1,L,26579,4.0,"[L, ]","[26578, 1]"


time: 1.41 s


In [362]:
# translate_lex_trash pflegen
translate_lex_trash = pak.drop_cols(translate_lex_trash,['tag','tag_0','tag_1','tagZ','tagZZ'])
translate_lex_trash = pak.move_cols( translate_lex_trash, translate_lex.columns)
#translate_lex_trash

time: 63.6 ms


In [363]:
pak.nnan(translate_gm)

lemma_tag      93
lemma_score    93
dtype: int64

time: 648 ms


In [364]:
mask = translate_gm.lemma_tag.isnull()
check_mask(translate_gm,mask,0,100)

msg = 'verwaist_gm'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  93 rows
Move 93 rows from 671293
93 rows out of 105


time: 1.2 s


In [365]:
# Seltene Kombination (git's nicht mehr)
mask1 = translate_gm.lemma_home == 'T'
mask2 = translate_gm.data_home == 'T'
mask = mask1  &  mask2
check_mask(translate_gm,mask,0,51)

#grid(translate_gm,mask)
msg = 'Kombi TT'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 180 ms


In [366]:
# Bedeutungsloses löschen
mask1 = translate_gm.data == translate_gm.lemma
mask2A = translate_gm.data_tagZZ == translate_gm.lemma_tagZZ
mask2B = translate_gm.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_gm,mask,0,260)

translate_gm, df = pak.move_rows(translate_gm,mask)
grid(df)

check_mask:  239 rows
Move 239 rows from 671200
239 rows


time: 1.32 s


In [367]:
# Schrott rauslöschen
löschmich = ['Nominativ','Genitiv','Dativ','Akkusativ','Maskulinum','Femininum','Neutrum','Singular','Plural',
            'Hauptsatzkonjugation','Nebensatzkonjugation','Artikel','unbestimmt','Fugenelement','Allomorph',
             'Deklinationsendung','Komposita','Astronomischer Kalender','Konjugationsendung','Wortbildung',
             'am Lateinischen angelehnte Deklination','Hilfsverb','backen (regelmäßig)','Hauptsatz','Endung',
             'lateinisch','unbestimmter','Maskulinums','vernetzt','Derivatem','Ableitungsmorphem','griechisch',
             'Hapaxlegomenon','Interfix','Vollverb','unbestimmt','Artikel',
            ]
mask1A = translate_gm.lemma.isin(löschmich)
mask1B = translate_gm.data.isin(löschmich)
mask1C = translate_gm.lemma_id.isin(löschmich)
mask1D = translate_gm.data_id.isin(löschmich)
mask2 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask = (mask1A | mask1B | mask1C | mask1D)  &  mask2
check_mask(translate_gm,mask,10)

msg = 'Schrott1'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  10 rows
Move 10 rows from 670961
10 rows out of 115


time: 2.22 s


In [368]:
# Weiteren Schrott finden
mask1 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask2 = translate_gm.lemma_tagZZ != 'V'    # keine Verben
mask3 = ~translate_gm.lemma.str.endswith(('en','ern','eln','sein','tun')) # und sieht auch nicht aus wie ein Verb
mask = mask1  &  mask2  &  mask3
a = pak.analyse_freqs(translate_gm[mask],['lemma','data',])
grid(a)

49 rows


time: 1.56 s


In [369]:
# suche_debug translate_gm
search_str(translate_gm, suche_debug)

,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,merkmal_order,lemma_lower
66259,gefriergetrocknetem,T,gefriergetrocknetem,,0.822449,gm,gefriergetrocknet,L,gefriergetrocknet,ADJA,A,10.341779,1,gefriergetrocknet
66260,gefriergetrockneten,T,gefriergetrockneten,,2.253350,gm,gefriergetrocknet,L,gefriergetrocknet,ADJA,A,10.341779,1,gefriergetrocknet
66261,gefriergetrockneter,T,gefriergetrockneter,,1.298417,gm,gefriergetrocknet,L,gefriergetrocknet,ADJA,A,10.341779,1,gefriergetrocknet
66262,gefriergetrocknetes,T,gefriergetrocknetes,,1.166870,gm,gefriergetrocknet,L,gefriergetrocknet,ADJA,A,10.341779,1,gefriergetrocknet
66263,gefriergetrocknete,T,gefriergetrocknete,,1.848965,gm,gefriergetrocknet,L,gefriergetrocknet,ADJA,A,10.341779,1,gefriergetrocknet
182457,missratend,L,missratend,A,0.579592,gm,missraten_MADJ,L,missraten,ADJA,A,2.745493,1,missraten
182458,missrietet,T,missrietet,,0.963265,gm,missraten,L,missraten,VVFIN,V,11.045676,1,missraten
182459,missrieten,T,missrieten,,1.144945,gm,missraten,L,missraten,VVFIN,V,11.045676,1,missraten
182460,missrietst,T,missrietst,,0.812245,gm,missraten,L,missraten,VVFIN,V,11.045676,1,missraten
182461,missrietest,T,missrietest,,0.993878,gm,missraten,L,missraten,VVFIN,V,11.045676,1,missraten


time: 1.26 s


In [370]:
# not any_nan
assert not pak.any_nan(translate_gm)

time: 569 ms


In [371]:
# Spalten anpassen
translate_gm = pak.drop_cols(translate_gm,['merkmal_order'])
translate_gm['score'] = 0

time: 368 ms


In [372]:
# Spaltenvergleich
bpy.minivenn( translate_gm.columns, translate_lex.columns, format='diff')

[]

time: 40.1 ms


In [373]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_gm)

670951 rows added, now a total of 1315736
time: 1.79 s


In [374]:
# set_translate_lex_score
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

670951 Datensätze werden neu bewertet
time: 15.9 s


In [375]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

72 rows


time: 1.72 s


In [376]:
# nan entfernen
translate_lex['lemma_tag'] = translate_lex.lemma_tag.fillna('')

time: 430 ms


In [377]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'score', ]
col_funcs   =  [ '',        '',     'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'sum',   ]   
col_names   =  [ '',        '',     'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'score', ] 

time: 49.3 ms


In [378]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

27019 unique rows and 1288717 rows with duplicates
644398 rows less, now 671338 rows
time: 6min 1s


In [379]:
# data_id vorhanden und nicht widerspüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 1.68 s


In [380]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

37 rows


time: 967 ms


### translate_m
* Alle Lexeme aus wiktionary_merkmal

In [381]:
# Bei lemma: lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 845 ms


In [382]:
# Alle Lexeme aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate_m = wiktionary_merkmal[mask].copy()
#translate_m

time: 1.56 s


In [383]:
# translate_m umformen
translate_m = pak.rename_col(translate_m, 'merkmal', 'member')
translate_m['member'] = ('merkmal ' + translate_m.member + ' ' + translate_m.meta).str.strip()

translate_m = pak.drop_cols(translate_m, ['is_lex','num','meta'])

time: 2.24 s


In [384]:
# lemma_tagZZ schreiben
translate_m = pak.update_col( translate_m, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
translate_m = pak.move_cols(  translate_m,'lemma_tagZZ','lemma_tag')
translate_m['lemma_tagZZ'] =  translate_m.lemma_tagZZ.fillna('')

time: 6.06 s


In [385]:
# temp
mask1 = translate_m.data  == 'ausgerutscht'
mask2 = translate_m.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_m[mask1]

,lemma_id,member,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,merkmal_order,score,hunspell
904567,ausrutschen,merkmal Ptz2,ausgerutscht,ausgerutscht,VVPP,ADJ,A,ausrutschen,VVFIN,V,1.0,9.354445,1
2405708,ausgerutscht,merkmal lemma,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausgerutscht,ADJA,A,0.0,2.529797,1
2465006,ausrutschen,merkmal Ptz2,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausrutschen,VVFIN,V,11.0,9.354445,1


time: 359 ms


In [386]:
# data_id löschen, wenn nötig
mask1 = translate_m.data_id != translate_m.data
mask2 = translate_m.data_id == translate_m.lemma_id
mask = mask1  &  mask2
check_mask(translate_m,mask,25579)

translate_m.loc[mask,'data_id'] = ''
grid(translate_m,mask)
#a = pak.analyse_freqs(translate_m[mask],['member','data'])
#grid(a)

check_mask:  25558 rows
25558 rows out of 1139833


time: 1.11 s


In [387]:
# wiktionary_lemma suche_debug
grid(search_str(translate_lex, suche_debug))

37 rows


time: 1.11 s


In [388]:
# data_id: Wenn leer, neu schreiben
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,25579)
translate_m[mask] = pak.update_col( translate_m[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='score') 
translate_m[mask] = pak.update_col( translate_m[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',                        col_score='score') 

check_mask:  25558 rows
update_col: col_score score applied, 3747 records less!
update_col: 25558 cells written into existing column
update_col: col_score score applied, 20158 records less!
update_col: 0 cells written into existing column
time: 5.54 s


In [389]:
# data_id neu erzeugen
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,0)

translate_m.loc[mask,'data_id'] = translate_m[mask].lemma + '~' + translate_m[mask].data + '~N'

check_mask:  0 rows
time: 357 ms


In [390]:
search_str(translate_m, suche_debug)

,lemma_id,member,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,merkmal_order,score,hunspell
533957,missraten,merkmal alt,mißraten_MADJ,mißraten,VVFIN,VER,V,missraten,VVFIN,V,1.0,0.447259,0
903968,missraten,merkmal Ptz1,missratend,missratend,VVFIN,VER,V,missraten,VVFIN,V,1.0,0.447259,0
925516,gefriertrocknen,merkmal Ptz1,gefriertrocknend,gefriertrocknend,VVFIN,VER,V,gefriertrocknen,VVFIN,V,1.0,1.442266,0
1506993,gefriertrocknen,merkmal Imp Sg,gefriertrockne,gefriertrockne,VVIMP,VER,V,gefriertrocknen,VVFIN,V,1.0,1.442266,0
1507817,missraten,merkmal Imp Sg,missrate,missrate,VVIMP,VER,V,missraten,VVFIN,V,1.0,0.447259,0
1853878,gefriertrocknen,merkmal Imp Pl,gefriertrocknet,gefriertrocknet,VVIMP,VER,V,gefriertrocknen,VVFIN,V,1.0,1.442266,0
1861471,missraten,merkmal Imp Pl,missratet,missratet,VVIMP,VER,V,missraten,VVFIN,V,1.0,0.447259,0
1865997,gefriertrocknen,merkmal Präsens du,gefriertrocknest,gefriertrocknest,VVFIN,VER,V,gefriertrocknen,VVFIN,V,1.0,1.442266,0
1872680,missraten,merkmal Präsens du,missrätst,missrätst,VVFIN,VER,V,missraten,VVFIN,V,1.0,0.447259,0
1881578,missraten,merkmal Präsens ersiees,missrät,missrät,VVFIN,VER,V,missraten,VVFIN,V,1.0,0.447259,0


time: 1.96 s


In [391]:
#translate_m_bak = translate_m.copy()
#translate_m = translate_m_bak.copy()

time: 36.6 ms


In [392]:
from x32_Funktionen import *
translate_m = pflege_translate_tabelle(translate_m, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_m)

rank: 5 rows less, now 671333 rows
update_col: 586753 cells written into existing column
update_col: 1139833 cells written into existing column
update_col: 611634 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1139833 cells written into new column
update_col: 1139833 cells written into new column
renaming score --> data_score
time: 1min 29s


In [393]:
# Bedeutungsloses löschen
mask1 = ~translate_m.member.str.contains('lemma')
mask2 =  translate_m.data == translate_m.lemma
mask3A = translate_m.data_tagZZ == translate_m.lemma_tagZZ
mask3B = translate_m.data_tagZZ == ''
mask = mask1  &  mask2  &  (mask3A | mask3B)
#grid(translate_m,mask)
check_mask(translate_m,mask,357911)

translate_m, df = pak.move_rows(translate_m,mask)
grid(df)

check_mask:  357574 rows
Move 357574 rows from 1139833
357574 rows


time: 8.76 s


In [394]:
pak.analyse_freqs(translate_m, ['lemma_home','data_home'])

,lemma_home,lemma_home_count,lemma_home_percent,data_home,data_home_count
0,L,754822,96.5,"[T, L, B, ]","[499300, 165851, 61428, 28243]"
1,B,27437,3.5,"[B, L, , T]","[25857, 964, 526, 90]"


time: 2.69 s


In [395]:
# Neuzugänge
mask = translate_m.data_home == ''
grid(translate_m,mask)

28769 rows out of 782259


time: 690 ms


In [396]:
assert not pak.any_nan(translate_m)

time: 1.03 s


In [397]:
# Spalten anpassen
translate_m = pak.drop_cols(translate_m,['data_tagZ', 'hunspell', 'merkmal_order'])
translate_m['score'] = 0

time: 796 ms


In [398]:
# Spaltenvergleich
bpy.minivenn( translate_m.columns, translate_lex.columns, format='diff')

[{'data_tag', 'data_tagZZ'}, set()]

time: 43.6 ms


In [399]:
search_str(translate_m, suche_debug)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,score
18315,mißraten_MADJ,L,mißraten,VVFIN,V,0.447259,merkmal alt,missraten,L,missraten,VVFIN,V,11.045676,missraten,0
43694,missratend,B,missratend,VVFIN,V,0.447259,merkmal Ptz1,missraten,L,missraten,VVFIN,V,11.045676,missraten,0
65242,gefriertrocknend,B,gefriertrocknend,VVFIN,V,1.442266,merkmal Ptz1,gefriertrocknen,L,gefriertrocknen,VVFIN,V,31.451888,gefriertrocknen,0
488244,gefriertrockne,T,gefriertrockne,VVIMP,V,1.442266,merkmal Imp Sg,gefriertrocknen,L,gefriertrocknen,VVFIN,V,31.451888,gefriertrocknen,0
489068,missrate,T,missrate,VVIMP,V,0.447259,merkmal Imp Sg,missraten,L,missraten,VVFIN,V,11.045676,missraten,0
835129,gefriertrocknet,T,gefriertrocknet,VVIMP,V,1.442266,merkmal Imp Pl,gefriertrocknen,L,gefriertrocknen,VVFIN,V,31.451888,gefriertrocknen,0
842722,missratet,T,missratet,VVIMP,V,0.447259,merkmal Imp Pl,missraten,L,missraten,VVFIN,V,11.045676,missraten,0
847248,gefriertrocknest,T,gefriertrocknest,VVFIN,V,1.442266,merkmal Präsens du,gefriertrocknen,L,gefriertrocknen,VVFIN,V,31.451888,gefriertrocknen,0
853931,missrätst,T,missrätst,VVFIN,V,0.447259,merkmal Präsens du,missraten,L,missraten,VVFIN,V,11.045676,missraten,0
862829,missrät,T,missrät,VVFIN,V,0.447259,merkmal Präsens ersiees,missraten,L,missraten,VVFIN,V,11.045676,missraten,0


time: 2.11 s


In [400]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_m)

782259 rows added, now a total of 1453597
time: 2.43 s


In [401]:
# set_translate_lex_score
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

782259 Datensätze werden neu bewertet
time: 28.2 s


In [402]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home,score,data_tag,data_tagZZ
4939,Alben,Alben,1.485714,wiki Deklinierte_Form gm,Album,Album,NN,N,album,58.288780,T,L,174.409199,<NA>,<NA>
4940,Alben_1,Alben,1.444898,wiki Deklinierte_Form gm,Alb,Alb,NN,N,alb,36.840843,T,L,27.511245,<NA>,<NA>
4941,Alben_2,Alben,1.465306,wiki Deklinierte_Form gm,Alba,Alba,NN,N,alba,24.635221,T,L,12.234277,<NA>,<NA>
4942,Alben_3,Alben,1.465306,wiki Deklinierte_Form gm,Albe,Albe,NN,N,albe,24.997210,T,L,9.315535,<NA>,<NA>
884393,Alben_4,Alben,14.086480,merkmal Nom Pl,Album,Album,NN,N,album,29.144390,L,L,13.066405,NN,N
910088,Alben_4,Alben,1.428745,merkmal Nom Pl,Alba_1,Alba,NN,N,alba,1.428745,L,L,0.178593,NN,N
919240,Alben_4,Alben,5.766139,merkmal Nom Pl,Albe,Albe,NN,N,albe,12.498605,L,L,5.576056,NN,N
919425,Alben_4,Alben,8.729598,merkmal Nom Pl,Alb,Alb,NN,N,alb,18.420422,L,L,8.241129,NN,N
933799,Alben_4,Alben,1.428745,merkmal Gen Pl,Alba_1,Alba,NN,N,alba,1.428745,L,L,0.178593,NN,N
938843,Alben_4,Alben,5.766139,merkmal Gen Pl,Albe,Albe,NN,N,albe,12.498605,L,L,5.576056,NN,N


time: 304 ms


In [403]:
# nan entfernen
translate_lex['data_tag']   = translate_lex.data_tag.fillna('')
translate_lex['data_tagZZ'] = translate_lex.data_tagZZ.fillna('')

time: 432 ms


In [404]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'score', ]
col_funcs   =  [ '',        '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'sum',   ]   
col_names   =  [ '',        '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'score', ] 

time: 35.2 ms


In [405]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1149829 unique rows and 303768 rows with duplicates
219549 rows less, now 1234048 rows
time: 59.3 s


In [406]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0

check_mask:  0 rows
time: 2.63 s


In [407]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

71 rows


time: 2.02 s


In [408]:
#pak.sample(translate_lex)

time: 52.9 ms


In [409]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home,score
1936,Alben_4,Alben,NN,N,34.918392,merkmal Nom Pl Gen Dat Akk,Alb,Alb,NN,N,alb,73.681686,L,L,32.964514
1937,Alben_4,Alben,NN,N,5.714982,merkmal Nom Pl Gen Dat Akk,Alba_1,Alba,NN,N,alba,5.714982,L,L,0.714373
1938,Alben_4,Alben,NN,N,23.064556,merkmal Nom Pl Gen Dat Akk,Albe,Albe,NN,N,albe,49.994419,L,L,22.304224
1939,Alben_4,Alben,NN,N,56.345921,merkmal Nom Pl Gen Dat Akk,Album,Album,NN,N,album,116.577560,L,L,52.265619
89158,Alben,Alben,,,1.485714,wiki Deklinierte_Form gm,Album,Album,NN,N,album,58.288780,T,L,174.409199
89159,Alben_1,Alben,,,1.444898,wiki Deklinierte_Form gm,Alb,Alb,NN,N,alb,36.840843,T,L,27.511245
89160,Alben_2,Alben,,,1.465306,wiki Deklinierte_Form gm,Alba,Alba,NN,N,alba,24.635221,T,L,12.234277
89161,Alben_3,Alben,,,1.465306,wiki Deklinierte_Form gm,Albe,Albe,NN,N,albe,24.997210,T,L,9.315535
1040836,Alben_4,Alben,NN,N,0.519488,merkmal lemma,Alben_4,Alben,NN,N,alben,0.519488,L,L,0.025974


time: 327 ms


In [410]:
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

check_mask:  562710 rows
time: 584 ms


### translate_lma und translate_lmb
* aus wiktionary_lexmerkmal

In [411]:
# zunächst wiktionary_lexmerkmal: Nichtlexeme löschen. Das sind vor allem syllables, ipa und rhymes.
mask = wiktionary_lexmerkmal.is_lex == False
check_mask( wiktionary_lexmerkmal, mask, 1523133) # 1523133

msg = 'Nichtlexeme'
wiktionary_lexmerkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_lexmerkmal, wiktionary_merkmal_trash, mask, msg )
#grid(wiktionary_merkmal_trash, msg)

check_mask:  1524420 rows
Move 1524420 rows from 1658238
time: 4.94 s


In [412]:
#pak.analyse_freqs(wiktionary_lexmerkmal,['merkmal','data'])

time: 41.9 ms


In [413]:
# wiktionary_lexmerkmal
wiktionary_lexmerkmal.head(1)

,lex_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
i,,,,,,,,,,,
1735,lieb_1,1,alt_1,liebe_1,liebe,,,,,0.95102,True


time: 64 ms


In [414]:
# lemma_id für data_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, on='data_id',  col='lemma_id')

update_col: 198023 cells written into existing column
update_col: WARNING: df_from identifier not unique. I call this again with func="max"
update_col: func max applied, 401492 records less!
update_col: 112165 cells written into existing column
time: 1min 26s


In [415]:
# translate_lma erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lma = wiktionary_lexmerkmal[mask].copy()

translate_lma = pak.rename_col(translate_lma, 'merkmal', 'member')
translate_lma['member'] = ('lma ' + translate_lma.member + ' ' + translate_lma.meta ).str.strip()

translate_lma = pak.drop_cols(translate_lma, ['lex_id','is_lex','data_tag','num','meta','data'])

time: 369 ms


In [416]:
# lemma_id für lex_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, left_on='lex_id', right_on='data_id',  col='lemma_id', col_score='score')

update_col: col_score score applied, 401492 records less!
update_col: 133818 cells written into existing column
time: 3.88 s


In [417]:
# translate_lmb erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lmb = wiktionary_lexmerkmal[mask].copy()

translate_lmb = pak.rename_col(translate_lmb, 'merkmal', 'member')
translate_lmb['member'] = ('lmb ' + translate_lmb.member + ' ' + translate_lmb.meta ).str.strip()

translate_lmb = pak.drop_cols(translate_lmb, ['data_id','data','data_tag','is_lex','num','meta','data'])

translate_lmb = pak.rename_col(translate_lmb, 'lex_id', 'data_id')

time: 419 ms


In [418]:
# pflege_translate_tabelle
from x32_Funktionen import *
translate_lma = pflege_translate_tabelle(translate_lma, translate_tagZ, wiktionary_lemma, translate_lex )
translate_lmb = pflege_translate_tabelle(translate_lmb, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 401492 rows less, now 832556 rows
update_col: 112165 cells written into new column
update_col: 112163 cells written into new column
update_col: 16724 cells written into existing column
update_col: 16724 cells written into new column
update_col: 112163 cells written into new column
update_col: 112163 cells written into new column
renaming score --> data_score
rank: 401492 rows less, now 832556 rows
update_col: 133818 cells written into new column
update_col: 133814 cells written into new column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 133814 cells written into new column
update_col: 133814 cells written into new column
renaming score --> data_score
time: 24.4 s


In [419]:
# Bedeutungsloses löschen
mask1  = translate_lma.data       == translate_lma.lemma
mask2A = translate_lma.data_tagZZ == translate_lma.lemma_tagZZ
mask2B = translate_lma.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lma,mask,15885)

translate_lma, df = pak.move_rows(translate_lma,mask)
grid(df)

check_mask:  15968 rows
Move 15968 rows from 112165
15968 rows


time: 522 ms


In [420]:
# Bedeutungsloses löschen
mask1  = translate_lmb.data       == translate_lmb.lemma
mask2A = translate_lmb.data_tagZZ == translate_lmb.lemma_tagZZ
mask2B = translate_lmb.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lmb,mask,12)

translate_lmb, df = pak.move_rows(translate_lmb,mask)
grid(df)

check_mask:  9 rows
Move 9 rows from 133818
9 rows


time: 367 ms


In [421]:
# Spalten anpassen
translate_lma = pak.drop_cols(translate_lma,['merkmal_order'])
translate_lma['score'] = 0

time: 112 ms


In [422]:
# Spaltenvergleich
bpy.minivenn( translate_lma.columns, translate_lex.columns, format='diff')

[set(), {'data_tag'}]

time: 34.7 ms


In [423]:
# Spalten anpassen
translate_lmb = pak.drop_cols(translate_lmb,['merkmal_order'])
translate_lmb['score'] = 0

time: 113 ms


In [424]:
# Spaltenvergleich
bpy.minivenn( translate_lmb.columns, translate_lex.columns, format='diff')

[set(), {'data_tag'}]

time: 48.1 ms


In [425]:
# Scores reduzieren
translate_lma['data_score'] /= 3
translate_lma['lemma_score'] /= 3
translate_lmb['data_score'] /= 3
translate_lmb['lemma_score'] /= 3

time: 44.8 ms


In [426]:
#pak.sample(translate_lma)

time: 35.9 ms


In [427]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_lma)
translate_lex = pak.add_rows(translate_lex,translate_lmb)

96197 rows added, now a total of 1330245
133809 rows added, now a total of 1464054
time: 2.33 s


### pflegen

In [428]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 2.98 s


In [429]:
# verwaiste lemma
mask = translate_lex.lemma.isnull()
check_mask(translate_lex,mask,0,10)

msg = 'verwaist_lemma_10'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  6 rows
Move 6 rows from 1464054
6 rows out of 121


time: 1.68 s


In [430]:
# verwaiste lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,100)

msg = 'verwaist_lemma_id_100'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  92 rows
Move 92 rows from 1464048
92 rows out of 213


time: 2.24 s


In [431]:
#data_tag fillna
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')

time: 333 ms


In [432]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

time: 1.19 s


In [433]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 631492 rows less, now 832464 rows
update_col: 1463956 cells written into existing column
update_col: 1463956 cells written into existing column
update_col: 278445 cells written into existing column
update_col: 26553 cells written into existing column
update_col: 1463956 cells written into new column
update_col: 1463956 cells written into existing column
time: 1min 39s


### analyse_stringdiff

In [434]:
from strsimpy.qgram import QGram 

qgram = QGram(2)
a = 'herbeispringen'
b = 'spring herbei'
c = 'herbeispringendes'
d = 'xfalsche Anfänger'
print(qgram.distance(a,b))
print(qgram.distance(a,c))
print(qgram.distance(a,d))

5
3
21
time: 65.6 ms


In [435]:
from strsimpy.metric_lcs import MetricLCS
metric_lcs = MetricLCS()

print(metric_lcs.distance(a,b))
print(metric_lcs.distance(a,c))
print(metric_lcs.distance(a,d))

0.5
0.17647058823529416
0.7058823529411764
time: 43.1 ms


In [436]:
# data_swaped: dächtest um >> umdächtest
translate_lex['data_swaped'] = translate_lex.data.str.split()
mask = translate_lex.data_swaped.str.len() == 2
translate_lex.loc[ mask, 'data_swaped'] = translate_lex[mask].data_swaped.str[-1] + translate_lex[mask].data_swaped.str[0]
translate_lex.loc[~mask, 'data_swaped'] = ''

time: 3.11 s


In [437]:
# Ermittelt die Differenz zwischen lex und lemma

def analyse_stringdiff(zeile):
    if len(zeile.data_swaped) > 0:
        result_a = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        result_b = metric_lcs.distance( zeile.data_swaped.lower(), zeile.lemma.lower() )     
        zeile['same_lexlemma'] = 1.0 - min(result_a, result_b)
    else:
        result   = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        zeile['same_lexlemma'] = 1.0 - result      
    return zeile

time: 30.1 ms


In [438]:
# apply analyse_stringdiff 
translate_lex['same_lexlemma']   = 0.0
translate_lex = translate_lex.swifter.apply( analyse_stringdiff, axis=1 )
#translate_lex = pak.change_datatype(translate_lex, verbose=False)  

time: 3min 24s


In [439]:
# data_swaped wieder löschen
translate_lex = pak.drop_cols(translate_lex,'data_swaped')

time: 812 ms


In [440]:
# same_lexlemma auf scores übertragen
translate_lex['data_score']  = translate_lex.data_score  * ((translate_lex.same_lexlemma + 0.1)**0.5)
translate_lex['lemma_score'] = translate_lex.lemma_score * ((translate_lex.same_lexlemma + 0.1)**0.5)

time: 90 ms


### data_id eindeutig

In [441]:
# score komplett neu berechnen
translate_lex['score'] = 0
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

1463956 Datensätze werden neu bewertet
time: 17.1 s


In [442]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

time: 41.7 ms


In [443]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,score,lemma_lower,same_lexlemma
1936,Alben_4,B,Alben,NN,N,29.214823,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,15.411630,9.086134,alb,0.6
1937,Alben_4,B,Alben,NN,N,4.781497,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,1.195374,0.239075,alba,0.6
1938,Alben_4,B,Alben,NN,N,21.880959,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,11.857218,6.930983,albe,0.8
1939,Alben_4,B,Alben,NN,N,47.142379,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,24.383946,14.467816,album,0.6
89158,Alben,T,Alben,,,1.243038,wiki Deklinierte_Form gm,Album,B,Album,NN,N,24.383946,98.778823,album,0.6
89159,Alben_1,T,Alben,,,1.208888,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,15.411630,15.713853,alb,0.6
89160,Alben_2,T,Alben,,,1.225963,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,10.305653,7.074762,alba,0.6
89161,Alben_3,T,Alben,,,1.390111,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,11.857218,6.102373,albe,0.8
1040744,Alben_4,B,Alben,NN,N,0.544844,merkmal lemma,Alben_4,B,Alben,NN,N,0.544844,0.027242,alben,1.0


time: 140 ms


In [444]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ '',        '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'first',          'sum',   ]   
col_names   =  [ '',        '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'same_lexlemma',  'score', ] 

time: 32.2 ms


In [445]:
# Gruppieren!
lookup_lex_tag = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1115309 unique rows and 348647 rows with duplicates
228034 rows less, now 1235922 rows
time: 1min 8s


In [446]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,score,lemma_lower,same_lexlemma
1936,Alben_4,B,Alben,NN,N,29.214823,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,15.411630,9.086134,alb,0.6
1937,Alben_4,B,Alben,NN,N,4.781497,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,1.195374,0.239075,alba,0.6
1938,Alben_4,B,Alben,NN,N,21.880959,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,11.857218,6.930983,albe,0.8
1939,Alben_4,B,Alben,NN,N,47.142379,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,24.383946,14.467816,album,0.6
89158,Alben,T,Alben,,,1.243038,wiki Deklinierte_Form gm,Album,B,Album,NN,N,24.383946,98.778823,album,0.6
89159,Alben_1,T,Alben,,,1.208888,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,15.411630,15.713853,alb,0.6
89160,Alben_2,T,Alben,,,1.225963,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,10.305653,7.074762,alba,0.6
89161,Alben_3,T,Alben,,,1.390111,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,11.857218,6.102373,albe,0.8
1040744,Alben_4,B,Alben,NN,N,0.544844,merkmal lemma,Alben_4,B,Alben,NN,N,0.544844,0.027242,alben,1.0


time: 154 ms


### weniger data_ids 

In [447]:
translate_lex['data_id_bak'] = '' # Sicherheitskopie von data_id

time: 45.5 ms


In [448]:
# Eine halbe Million data_ids, die nirgendwohin verweisen. 
# Sichern und Leeren!
mask1 =  translate_lex.data != ''
mask2 = ~translate_lex.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~translate_lex.data_id.isin(wiktionary_merkmal.data_id)
mask = mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask,455800)

# kennzeichnen
translate_lex.loc[mask,'data_id_bak'] = translate_lex[mask].data_id  # sichern
translate_lex.loc[mask,'data_id']     = ''                           # leeren

#grid(translate_lex[mask],pak.sample )

check_mask:  455698 rows
time: 2.09 s


In [449]:
# Sicherheitskopie
#translate_lex_bak = translate_lex.copy()
#translate_lex = translate_lex_bak.copy()

time: 36.1 ms


In [450]:
# Zweites data_id-Feld
translate_lex['data_id2'] = translate_lex.data_id.copy()
translate_lex = pak.move_cols(translate_lex,'data_id2','data_id')
#translate_lex

time: 1.19 s


In [451]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

time: 47.4 ms


In [452]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1088852 unique rows and 375104 rows with duplicates
242661 rows less, now 1221295 rows
time: 1min 47s


In [453]:
# Beide data_id leer? Dann data_id_bak verwenden
mask1 = translate_lex.data_id == ''
mask2 = translate_lex.data_id2 == ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,354478)
translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id_bak
#grid(translate_lex,mask)

check_mask:  364907 rows
time: 387 ms


In [454]:
# Prüfen: data_id immer belegt
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows
time: 120 ms


In [455]:
# data_ids sind verschieden, aber nahezu gleich
# z.B. abstoßend_1 und abstoßend
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask3 = pak.remove_str( translate_lex.data_id, ['_1','_2','_3','_4','_MADJ',], safemode=False )  ==  pak.remove_str( translate_lex.data_id2, ['_1','_2','_3','_4','_MADJ',], safemode=False )
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask,393)

translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id2 # das Minimum, also die kürzere id
translate_lex.loc[mask,'data_id2'] = ''
#grid(translate_lex,mask)

check_mask:  557 rows
time: 2.03 s


In [456]:
# Jetzt gibt es keine widersprüchliche data_id mehr
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask,)

check_mask:  0 rows
time: 202 ms


In [457]:
# Hilfsspalten löschen
translate_lex = pak.drop_cols(translate_lex,['data_id2','data_id_bak'])

time: 883 ms


In [458]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

71 rows


time: 1.66 s


In [459]:
# pflege_translate_tabelle
translate_lex_ranked  = pak.rank(translate_lex, col_score='score', cols_group='data_id', on_conflict='first')
translate_lex         = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex_ranked )

rank: 402987 rows less, now 818308 rows
rank: 0 rows less, now 818308 rows
update_col: 1221295 cells written into existing column
update_col: 1221295 cells written into existing column
update_col: 277826 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1221295 cells written into new column
update_col: 1221295 cells written into existing column
time: 1min 13s


In [460]:
# translate_lex: data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 1.75 s


In [461]:
assert not pak.any_nan(translate_lex)

time: 1.6 s


In [462]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
134379,Alben_4,B,Alben,NN,N,29.214823,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,18.420421,0.6,9.086134,alb
134380,Alben_4,B,Alben,NN,N,4.781497,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,1.428745,0.6,0.239075,alba
134381,Alben_4,B,Alben,NN,N,21.880959,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,12.498605,0.8,6.930983,albe
134382,Alben_4,B,Alben,NN,N,47.142379,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,29.144389,0.6,14.467816,album
219854,Alben,T,Alben,,,1.243038,wiki Deklinierte_Form gm,Album,B,Album,NN,N,29.144389,0.6,98.778823,album
219855,Alben_1,T,Alben,,,1.208888,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,18.420421,0.6,15.713853,alb
219856,Alben_2,T,Alben,,,1.225963,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,12.317611,0.6,7.074762,alba
219857,Alben_3,T,Alben,,,1.390111,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,12.498605,0.8,6.102373,albe
1026964,Alben_4,B,Alben,NN,N,0.544844,merkmal lemma,Alben_4,B,Alben,NN,N,0.519488,1.0,0.027242,alben


time: 168 ms


In [463]:
# Beispiel anzeigen
mask = translate_lex.data == 'ausführt'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
59273,ausführt_1,T,ausführt,,,1.296715,wiki Konjugierte_Form Verb gm lmb alt_1,ausfahren,B,ausfahren,VVFIN,V,83.063455,0.666667,145.784074,ausfahren
59274,ausführt,T,ausführt,,,1.707449,wiki Konjugierte_Form Verb gm lma alt_1,ausführen,B,ausführen,VVFIN,V,35.611569,0.777778,268.622694,ausführen


time: 184 ms


In [464]:
# Beispiel anzeigen
#finde = ['eine','einer','eines','einen','ein','einem']
#mask = translate_lex.data.isin(finde)
#translate_lex[mask]

time: 39.4 ms


In [465]:
#pak.analyse_freqs(translate_lex,['data_home','lemma_home'])

time: 33.2 ms


## wiktionary_merkmal

In [466]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 410 ms


In [467]:
# Sicherheitskopie
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

time: 42 ms


In [468]:
wiktionary_merkmal['data_id_bak'] = '' # Sicherheitskopie von data_id

time: 58.3 ms


In [469]:
translate_lex

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
0,"1,5-Grad-Zieles",T,"1,5-Grad-Zieles",,,1.495522,wiki Deklinierte_Form gm lma alt_1 lmb,"1,5-Grad-Ziel",B,"1,5-Grad-Ziel",NN,N,68.230416,0.866667,806.498770,"1,5-grad-ziel"
1,"1,5-Grad-Ziels",T,"1,5-Grad-Ziels",,,1.526107,wiki Deklinierte_Form gm lma alt_1 lmb,"1,5-Grad-Ziel",B,"1,5-Grad-Ziel",NN,N,68.230416,0.928571,831.905359,"1,5-grad-ziel"
2,1-Cent-Münzen,T,1-Cent-Münzen,,,3.335997,wiki Deklinierte_Form gm lma alt_2 lmb,1-Cent-Münze,B,1-Cent-Münze,NN,N,7.302753,0.923077,151.066685,1-cent-münze
3,1-Cent-Stücke,T,1-Cent-Stücke,,,3.971149,wiki Deklinierte_Form gm Dativ-e lma alt_2 lmb...,1-Cent-Stück,B,1-Cent-Stück,NN,N,7.804628,0.923077,193.431185,1-cent-stück
4,1-Cent-Stücken,T,1-Cent-Stücken,,,2.149679,wiki Deklinierte_Form gm lma alt_2 lmb,1-Cent-Stück,B,1-Cent-Stück,NN,N,7.804628,0.857143,154.860779,1-cent-stück
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221290,führe_auf,T,führe auf,,,0.357185,lmb alt_1,auffahren,B,auffahren,VVFIN,V,59.319242,0.777778,111.330743,auffahren
1221291,dränge_zu,T,dränge zu,,,0.357566,lmb alt_1,zudringen,B,zudringen,VVFIN,V,18.573203,0.777778,34.981177,zudringen
1221292,äste_ab,T,äste ab,,,0.361255,lmb alt_1,abäsen,B,abäsen,VVFIN,V,9.726676,0.833333,18.974351,abäsen
1221293,beschreit,T,beschreit,,,0.286541,lmb alt_1,beschreien,B,beschreien,VVFIN,V,26.774055,0.800000,1028.869381,beschreien


time: 166 ms


In [470]:
# verwaiste data_id:
# leeren und neu belegen
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(wiktionary_merkmal,mask, 0, 34)

# kennzeichnen
wiktionary_merkmal.loc[mask,'data_id_bak'] = wiktionary_merkmal[mask].data_id  # sichern
wiktionary_merkmal.loc[mask,'data_id']     = ''                                # leeren

# data_id schreiben 
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='score')
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',  col_score='score', )

# Kontrolle
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_merkmal,mask, 0)

# anschauen
mask = wiktionary_merkmal.data_id_bak != ''
grid(wiktionary_merkmal,mask)

check_mask:  31 rows
update_col: col_score score applied, 3747 records less!
update_col: 28 cells written into existing column
update_col: col_score score applied, 386278 records less!
update_col: 3 cells written into existing column
check_mask:  0 rows
31 rows out of 2480102


time: 14 s


In [471]:
mask = wiktionary_merkmal.data_id == 'Petersen~Peterson'
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,score,hunspell,data_id_bak
816898,Petersen,alt_3,True,Petersen~Peterson,Peterson,NE,NAM,N,Petersen,NE,N,,,5.0,0.919367,0,


time: 448 ms


In [472]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 530 ms


In [473]:
# Verlinkung auf data prüfen 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, on='data_id', col='data', col_rename='data_check', col_score='score')
mask_kaputt = wiktionary_merkmal.data != wiktionary_merkmal.data_check
check_mask(wiktionary_merkmal,mask_kaputt,0)
#grid(wiktionary_merkmal,mask_kaputt)

update_col: col_score score applied, 402987 records less!
update_col: 2005906 cells written into new column
check_mask:  0 rows
time: 17.7 s


In [474]:
# meta == lemma
mask = wiktionary_merkmal.meta == wiktionary_merkmal.lemma
check_mask(wiktionary_merkmal,mask,57940)

wiktionary_merkmal.loc[mask,'meta'] = ''

check_mask:  56185 rows
time: 650 ms


In [475]:
# meta == lex 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, left_on='meta', right_on='data', col='lemma', col_rename='löschmich_lemma', col_score='score')

mask = wiktionary_merkmal.lemma == wiktionary_merkmal.löschmich_lemma
check_mask(wiktionary_merkmal,mask,13845)
wiktionary_merkmal.loc[mask,'meta'] = ''

wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'löschmich_lemma')

update_col: col_score score applied, 422127 records less!
update_col: 146621 cells written into new column
check_mask:  14152 rows
time: 17 s


In [476]:
# Hilfsdaten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['data_id_bak','notiz','data_check'])

time: 1.63 s


In [477]:
grid(wiktionary_merkmal,pak.sample)

21 rows out of 2480102


time: 22.7 s


In [478]:
#mask = translate_lex.data.isin(['besonnte','überlegte'])
#grid(translate_lex,mask)

time: 39.5 ms


In [484]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

time: 230 ms


## translate_lemma ==========
* Datensätze aus translate_lex, deren IDs von Lemma zu Lemma zeigen
* Aber keine Lemmas (also keine Verweise auf sich selbst)
* Datensätze aus translate_abk

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [490]:
# translate_lemma erzeugen
keine_lemmas = ['einen','beigen','Montage',]

mask1 =  translate_lex.data_home == 'B'             # beides sind lemmas
mask2 =  translate_lex.data != translate_lex.lemma  # sie sind verschieden
mask3 = ~translate_lex.data.isin(keine_lemmas)      # gehören nicht zu den Ausnahmen
mask4 = ~translate_lex.member.str.contains('Ptz1')  # kein Partizip1
mask5 = ~translate_lex.member.str.contains('Ptz2')  # kein Partizip2
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lex,mask,57729)
#grid(translate_lex,mask)
translate_lex, translate_lemma = pak.move_rows(translate_lex,mask, msg=None)

check_mask:  57729 rows
Move 57729 rows from 1221295
time: 1.82 s


In [ ]:
pak.sample(translate_lemma)

### translate_abk

In [492]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0,50)

msg = 'kurz_id inzwischen verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg )
grid(translate_abk_trash,msg)

check_mask:  3 rows
Move 3 rows from 5658
3 rows out of 2305


time: 201 ms


In [493]:
assert not pak.any_nan(translate_abk)

time: 42.1 ms


In [494]:
# Umbenennungen. data und lemma sind nicht passende Bezeichnungen, aber pflege_translate_tabelle wünscht diese. 
# Wird später durch passendere ersetzt.
spalten = ['kurz_id','lang_id','todo']
df = translate_abk[spalten].copy()
df = pak.rename_col(df, 'kurz_id', 'data_id')
df = pak.rename_col(df, 'lang_id', 'lemma_id') 
df = pak.rename_col(df, 'todo', 'member') 
df['member'] = ('abk ' + df.member).str.strip()
df.head(1)

,data_id,lemma_id,member
1102,Bd.,Band,abk lemma


time: 60.5 ms


In [495]:
# lang_id war kein Lemma >> translate_lex nachschlagen
df['lemma_id_new'] = ''
mask = ~df.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(df,mask,30)

df.loc[mask] = pak.update_col( df[mask],          
                               translate_lex,
                               col_score='score', 
                               left_on='lemma_id',
                               right_on='data_id',
                               col='lemma_id',
                               col_rename='lemma_id_new',
                               
                       )   

mask = df.lemma_id_new != ''
df.loc[mask,'lemma_id'] = df[mask].lemma_id_new

df = pak.drop_cols(df, 'lemma_id_new')
#df[mask]

check_mask:  30 rows
update_col: col_score score applied, 345578 records less!
update_col: 30 cells written into existing column
time: 3.42 s


In [496]:
# Müll löschen
löschmich = ['PLURV','PIMS','MRSA','EBRG','BodSchätzDB','ZÄPrO','SJZ','Mmes.','RÄ','GDPdU','E_570','E_163']
mask = df.data_id.isin(löschmich)
df = pak.drop_rows(df,mask)

Delete 12 rows from 5655
time: 43.7 ms


In [497]:
# dups löschen
mask = df.duplicated(subset=['data_id','lemma_id'])
df = pak.drop_rows(df,mask)

Delete 208 rows from 5643
time: 51.5 ms


In [498]:
assert not pak.any_nan(df)

time: 39.8 ms


In [499]:
pak.nnan(df)

Series([], dtype: int64)

time: 48.5 ms


In [500]:
# score wird später zu data_score
df = pak.update_col(df,          
                    wiktionary_lemma,
                    left_on='data_id',
                    right_on='lemma_id',
                    col='score',
                       )   

update_col: 5435 cells written into new column
time: 155 ms


In [501]:
# pflege_translate_tabelle
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 345578 rows less, now 817988 rows
update_col: 5414 cells written into new column
update_col: 5435 cells written into new column
update_col: 5435 cells written into existing column
update_col: 5435 cells written into new column
update_col: 5435 cells written into new column
update_col: 5435 cells written into new column
renaming score --> data_score
time: 5.62 s


In [502]:
# data_swaped: dächtest um >> umdächtest
df['data_swaped'] = df.data.str.split()
mask = df.data_swaped.str.len() == 2
check_mask(df,mask)
df.loc[ mask, 'data_swaped'] = df[mask].data_swaped.str[-1] + df[mask].data_swaped.str[0]
df.loc[~mask, 'data_swaped'] = ''

check_mask:  396 rows
time: 63.8 ms


In [503]:
#pak.analyse_cols(df)

time: 50.1 ms


In [504]:
# apply analyse_stringdiff 
df['same_lexlemma']   = 0.0
df = df.apply( analyse_stringdiff, axis=1 )

time: 654 ms


In [505]:
# data_swaped wieder löschen
df = pak.drop_cols(df,'data_swaped')

time: 46.5 ms


In [506]:
# set_translate_lex_score
df['score'] = 0
df = set_translate_lex_score(df, lexeme_manuell)

5435 Datensätze werden neu bewertet
time: 187 ms


In [507]:
# data_tag
df['data_tag'] = df.lemma_tag.copy()

time: 33 ms


In [508]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lemma.columns, format='diff')

[]

time: 43.1 ms


In [509]:
# add_rows
translate_lemma = pak.add_rows(translate_lemma, df)

5435 rows added, now a total of 63164
time: 137 ms


In [510]:
# rename_col
translate_lemma = pak.rename_col(translate_lemma, 'data_id',     'lem_id')
translate_lemma = pak.rename_col(translate_lemma, 'data_home',   'lem_home')
translate_lemma = pak.rename_col(translate_lemma, 'data',        'lem')
translate_lemma = pak.rename_col(translate_lemma, 'data_score',  'lem_score')

time: 178 ms


In [511]:
translate_lemma = pak.rename_col(translate_lemma, 'data_tag',  'lem_tag')
translate_lemma = pak.rename_col(translate_lemma, 'data_tagZZ','lem_tagZZ')

time: 98.2 ms


In [512]:
grid(translate_lemma, pak.sample)

19 rows out of 63164


time: 1.09 s


In [513]:
# lem_tag
translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tag',
                                  col_rename = 'lem_tag',
                       )   

translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tagZZ',
                                  col_rename = 'lem_tagZZ',
                       )   

translate_lemma['lem_tag']   = translate_lemma.lem_tag.fillna('')
translate_lemma['lem_tagZZ'] = translate_lemma.lem_tagZZ.fillna('')
#translate_lemma = pak.move_cols( translate_lemma, ['lem_tag','lem_tagZZ'],'lem')      

update_col: 63164 cells written into existing column
update_col: 63164 cells written into existing column
time: 1.66 s


### belöschen

In [514]:
# translate_lemma_trash
translate_lemma_trash = translate_lemma.head(0)
translate_lemma_trash['msg'] = ''

time: 47.1 ms


In [515]:
a = pak.analyse_freqs(translate_lemma,['lemma','lem'])
grid(a)

33817 rows


time: 2.08 s


In [516]:
#translate_lemma

time: 29.3 ms


In [517]:
# aussortiert1
erlaubt = ['H2O','CAD','CH','GAU','DIN','UN','UNO','LCD','IGS','2D','3D','4D','ISP','IR','DRK','MPU','OOP','ÖPNV','° C','°C','HDTV','° F','°F']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 =  translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask3 =  translate_lemma.lem.str.len() <= 4
mask4 =  translate_lemma.lem.str.isupper()
mask5 = ~translate_lemma.lem.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lemma,mask,0,215)

msg = 'aussortiert1'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  212 rows
Move 212 rows from 63164
212 rows


time: 351 ms


In [518]:
# aussortiert2
# 

löschmich = ['-','Hag','DrIng','Deik','Fritsch','Frerk', 'Fricke','nomen nominandum',
             'wullacken','Diastema mediale','BM', 'JO', 'NB',
             'Rosch ha-Schana','Wiktionary','Fuzo', 'Fuzo.',
             'Intern', 'Intern.', 'iNet', 'iNet.', 'ZUM', 'ut supra','Art', 'GSSD', 
             'Met','DE', 'DFV', 'DHB', 'DSV','AN', 'AS', 'AUR','GVBl', 'GWB', 'GenG', 'Ges',
             'BKG', 'BVT', 'BfN', 'BfS','ritenuto','Folio','Aserbaidschan-Manat',
             'sensu lato','nes.','ceterum censeo',
             'pro rata','GGVS','IRPA','Tau','ABC',
             'ISA','DOS','ABF','ÖFB','VOC','KPÖ','IN','ZIF','PVP','BDO','g. e.','AUT','EUV',
            ]
behalten  = ['nen','ne','nes', 'geeint','einend', ]
mask1A =  translate_lemma.lem.isin(löschmich)
mask1B =  translate_lemma.lemma.isin(löschmich)
mask2  = ~translate_lemma.lem.isin(behalten)
mask = (mask1A | mask1B)  &  mask2
check_mask(translate_lemma,mask,150)

msg = 'aussortiert2'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  88 rows
Move 88 rows from 62952
88 rows out of 300


time: 290 ms


In [519]:
# unabhängige_lexeme

unabhängige_lexeme  = ['Alu','ALU','Arbeitslosenunterstützung','CO2', 'CO₂','chemisch','USK','vergl', 'verglasen','vergl.','24/7','Stunde','Schriftwerk','Buch','und','E 962','Verbindung',
                       'km/h','Kilometer','weiblich','männlich','Elefant','frühneuhochdeutsch','sein','Disko','modern','tun','Handlung',
                      ]
mask1 = translate_lemma.lem.isin(unabhängige_lexeme)
mask2 = translate_lemma.lemma.isin(unabhängige_lexeme)
mask = mask1  &  mask2
check_mask(translate_lemma,mask,17)

msg = 'unabhängige_lexeme'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  16 rows
Move 16 rows from 62864
16 rows out of 316


time: 219 ms


In [520]:
# Mögliche Probleme
# 
spalten = ['lem','lemma','member','lemma_score']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 = translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask = mask1  &  mask2
a = translate_lemma[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

536 rows


time: 198 ms


In [521]:
# lem_tag übertragen, falls nötig
mask = translate_lemma.lem_tag.isnull()
check_mask(translate_lemma,mask,0,10)

translate_lemma.loc[mask,'lem_tag']   = translate_lemma[mask].lemma_tag
translate_lemma.loc[mask,'lem_tagZZ'] = translate_lemma[mask].lemma_tagZZ

check_mask:  0 rows
time: 73.9 ms


In [522]:
# alle getaggt
maskA = translate_lemma.lem_tag == ''
maskB = translate_lemma.lem_tagZZ == ''
mask = maskA | maskB
check_mask(translate_lemma,mask,0)

check_mask:  0 rows
time: 85.7 ms


In [523]:
mask = translate_lemma.isna().any(axis=1)
translate_lemma[mask]

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower


time: 194 ms


In [524]:
#assert not pak.any_nan(translate_lemma)
pak.nnan(translate_lemma)

Series([], dtype: int64)

time: 168 ms


In [525]:
pak.sample(translate_lemma)

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
0,Eis,B,Eis,NN,N,0.434412,lma alt_1 meist,eislaufen,B,eislaufen,VVFIN,V,14.099391,0.333333,74.685255,eislaufen
283,Azubi,B,Azubi,NN,N,0.286338,merkmal alt_3 alt_weib,Lehrmädchen,B,Lehrmädchen,NN,N,9.880322,0.000000,12.784067,lehrmädchen
463,Friederike,B,Friederike,NE,N,1.906078,merkmal alt_3 alt_männ,Friedrich,B,Friedrich,NE,N,28.056476,0.700000,102.283973,friedrich
528,Hanke_N,B,Hanke,NE,N,4.593802,merkmal alt_3 alt_klein,Johannes,B,Johannes,NE,N,66.813616,0.500000,105.804106,johannes
722,Maria,B,Maria,NE,N,6.594947,merkmal alt_3 alt_männ,Mariechen,B,Mariechen,NE,N,10.093301,0.444444,36.384912,mariechen
749,Mendelsch,B,Mendelsch,ADJA,A,0.775684,merkmal alt alt_2,mendelsch,B,mendelsch,ADJA,A,9.040926,1.000000,38.704497,mendelsch
953,Schnecken,B,Schnecken,ITJ,X,1110.974491,merkmal Nom Pl Gen Dat Akk,Schnecke,B,Schnecke,NN,N,559.753421,0.888889,3337.514452,schnecke
1104,Wacken,B,Wacken,NE,N,0.505992,merkmal Nom Pl Gen Dat Akk,Wacke_4,L,Wacke,NN,N,0.327345,0.833333,0.177097,wacke
3432,zusammengeritten_1,B,zusammengeritten,ADJA,A,1.888391,gm,zusammenreiten_M,B,zusammenreiten,VVFIN,V,0.975000,0.812500,550.346504,zusammenreiten
3592,schwimmend_1,B,schwimmend,ADJA,A,0.463265,gm,schwimmen,B,schwimmen,VVFIN,V,594.083452,0.900000,119249.850277,schwimmen


time: 1.48 s


## wiktionary_merkmal_text ==========

In [526]:
# suche_debug

mask = wiktionary_merkmal_text.lemma_id.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

11 rows out of 621260


time: 285 ms


In [527]:
# Striche entfernen, auch in wiktionary_merkmal_text
mask1 = wiktionary_merkmal_text.data.str.count("\'") > 1
mask2 = wiktionary_merkmal_text.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal_text,mask,2290)
wiktionary_merkmal_text.loc[mask,'data'] = wiktionary_merkmal_text[mask].data.str.replace("'", '')
#grid( wiktionary_merkmal_text, mask )

check_mask:  2288 rows
time: 1.28 s


In [528]:
# not any_nan
#wiktionary_merkmal_text['merkmal_order2'] = wiktionary_merkmal_text.merkmal_order2.fillna(0) 
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.analyse_cols(wiktionary_merkmal_text)

time: 597 ms


In [529]:
# Verwaiste löschen
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,8263)
#grid(wiktionary_merkmal_text,mask)
wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

check_mask:  8206 rows
Delete 8206 rows from 621260
time: 634 ms


In [530]:
# lemma übertragen
wiktionary_merkmal_text = pak.update_col( wiktionary_merkmal_text, wiktionary_lemma, on='lemma_id', col='lemma'  )

update_col: 613054 cells written into existing column
time: 3.07 s


In [531]:
# meta == lemma
mask = wiktionary_merkmal_text.meta == wiktionary_merkmal_text.lemma
check_mask(wiktionary_merkmal_text,mask,204089)

wiktionary_merkmal_text.loc[mask,'meta'] = ''

check_mask:  204078 rows
time: 416 ms


In [532]:
# data leer
mask = wiktionary_merkmal_text.data == ''
check_mask(wiktionary_merkmal_text,mask,27)

wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

check_mask:  27 rows
Delete 27 rows from 613054
time: 550 ms


In [533]:
# suche_debug
mask = wiktionary_merkmal_text.lemma.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

11 rows out of 613027


time: 160 ms


In [534]:
#a = pak.analyse_cols(wiktionary_merkmal_text)
#(a)

time: 47.7 ms


In [535]:
# Was ist drin?
a = pak.analyse_freqs(wiktionary_merkmal_text,['merkmal','data'] )
grid(a)

6 rows


time: 2.47 s


In [536]:
# TODO
mask1 = wiktionary_merkmal_text.merkmal == 'bsp_re'
mask2 = wiktionary_merkmal_text.data.str.contains('-')
mask  = mask1  &  mask2
grid( wiktionary_merkmal_text, mask )

551 rows out of 613027


time: 489 ms


## wiktionary_lemma =====

In [537]:
# Anzahl der Lexeme untersuchen

mask = wiktionary_merkmal.is_lex == True
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'lex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['lex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='lex_anz')

973960 rows less, now 165873 rows
update_col: 165873 cells written into existing column
time: 3.49 s


In [538]:
# Anzahl der Nichtlexeme untersuchen

mask = wiktionary_merkmal.is_lex == False
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'nolex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['nolex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='nolex_anz')

1180500 rows less, now 159769 rows
update_col: 159769 cells written into existing column
time: 2.91 s


In [539]:
# Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
grid(wiktionary_lemma,mask, color='green')

No rows, mask filters them all away
time: 81.3 ms


## wiktionary_merkmal ==========

In [540]:
# verwaiste wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask, 0)

check_mask:  0 rows
time: 1.9 s


In [541]:
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])
#grid(a)

time: 35.2 ms


In [542]:
# Fehlende data_id?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2 
check_mask(wiktionary_merkmal,mask,0, 10)

check_mask:  0 rows
time: 521 ms


### Überschneidungen
* Lexeme, die sich in translate_lex finden aber auch direkt in wiktionary_lemma >> entweder oder!

In [547]:
# mask_LL erstellen
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1550)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

check_mask:  1513 rows
1513 rows


time: 4.5 s


In [548]:
# wiktionary_lemma belöschen: Abkürzungen wieder löschen
mask1 =  wiktionary_lemma.lemma.isin(translate_lex[mask_LL].data)
mask2 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask3 = ~wiktionary_lemma.member.str.contains('manuell')
mask4 =  wiktionary_lemma.page_part == -1
#mask5 = ~wiktionary_lemma.lemma.isin(['pos',])
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_lemma, mask, 0, 150)

msg = 'Überschneidungen_1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  136 rows
Move 136 rows from 165873
136 rows out of 1641


time: 461 ms


In [549]:
# wiktionary_merkmal: verwaiste_Lemmas löschen
mask1 =  wiktionary_merkmal.merkmal == 'lemma'
mask2 =  wiktionary_merkmal.data_id != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3 &  mask4
check_mask(wiktionary_merkmal,mask,0,150)

msg = 'verwaiste_Lemmas'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 5.72 s


In [550]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,150)
# translate_lemma[mask]

msg = 'verwaist_lem_id'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  136 rows
Move 136 rows from 62848
136 rows out of 452


time: 269 ms


In [551]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

#msg = 'verwaist_lemma_id'
#translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
#grid(translate_lemma_trash,msg)

check_mask:  0 rows
time: 136 ms


In [552]:
# Löschen: Einige sonstige Verwaisungen
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,30)
#grid(wiktionary_merkmal,mask)

msg = 'verwaist_data_id_24'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 4.78 s


In [553]:
# TODO Ungelöste Probleme 
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1400)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

check_mask:  1341 rows
1341 rows


time: 5.51 s


## translate_lex.data_tag 

In [554]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask1]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
784848,ausgerutscht,B,ausgerutscht,VVPP,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.992630,ausrutschen
1132788,ausgerutscht,B,ausgerutscht,ADJA,A,2.653274,merkmal lemma,ausgerutscht,B,ausgerutscht,ADJA,A,2.529797,1.00,13.266368,ausgerutscht
1191918,ausgerutscht,B,ausgerutscht,ADJA,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.992630,ausrutschen


time: 274 ms


In [556]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
1021390,Ackermann,B,Ackermann,NE,N,8.860208,merkmal lemma,Ackermann_1,L,Ackermann,NE,N,8.447877,1.0,11.075260,ackermann
1025867,Ackermann,B,Ackermann,NN,N,31.981134,merkmal lemma,Ackermann,B,Ackermann,NN,N,30.492814,1.0,159.905671,ackermann


time: 139 ms


In [557]:
# Beispiel anzeigen
mask = translate_lemma.lem == 'Ackermann'
translate_lemma[mask]

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
38486,Ackermann,B,Ackermann,NN,N,5.316687,merkmal alt_1,Ackersmann,B,Ackersmann,NN,N,11.561946,0.9,51.564471,ackersmann


time: 56.8 ms


In [558]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex   = pflege_translate_tabelle(translate_lex,   translate_tagZ, wiktionary_lemma)
translate_lemma = pflege_translate_tabelle(translate_lemma, translate_tagZ, wiktionary_lemma, translate_lex )

update_col: 1163429 cells written into existing column
update_col: 219960 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1163429 cells written into new column
update_col: 1163429 cells written into existing column
rank: 345578 rows less, now 817988 rows
update_col: 62712 cells written into existing column
update_col: 62712 cells written into new column
update_col: 62712 cells written into existing column
renaming score --> data_score
time: 1min 7s


In [559]:
# Nachschlagetabelle vorbereiten
spalten = ['data_id','data','data_tag','lemma_id','lemma','lemma_tag','lemma_tagZZ']
mask = wiktionary_merkmal.is_lex == True
df = wiktionary_merkmal[mask][spalten].drop_duplicates().sort_values(spalten)
#df

time: 6.73 s


In [560]:
# Prüfe Anzahl data_tag
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

check_mask:  531668 rows
time: 273 ms


In [561]:
# Vorhandene data_tag übertragen
# sonst leeren
translate_lex['data_tag_0'] = translate_lex.data_tag
translate_lex['data_tag_1'] = translate_lex.data_tag
translate_lex = pak.move_cols(translate_lex, ['data_tag','data_tag_0','data_tag_1'],'data')

time: 890 ms


In [562]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
964346,Ackermann,B,Ackermann,NE,NE,NE,N,8.860208,merkmal lemma,Ackermann_1,L,Ackermann,NE,N,8.447877,1.0,11.075260,ackermann
968823,Ackermann,B,Ackermann,NN,NN,NN,N,31.981134,merkmal lemma,Ackermann,B,Ackermann,NN,N,30.492814,1.0,159.905671,ackermann


time: 156 ms


In [563]:
# data_tag übertragen 
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
           
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
  
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

print()
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
   
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

update_col: func first applied, 56718 records less!
update_col: 258640 cells written into existing column
update_col: func last applied, 56718 records less!
update_col: 258640 cells written into existing column
update_col: func first applied, 58559 records less!
update_col: 10478 cells written into existing column
update_col: func last applied, 58559 records less!
update_col: 10478 cells written into existing column
update_col: func first applied, 60332 records less!
update_col: 1229 cells written into existing column
update_col: func last applied, 60332 records less!
update_col: 1229 cells written into existing column
update_col: func first applied, 60676 records less!
update_col: 132 cells written into existing column
update_col: func last applied, 60676 records less!
update_col: 132 cells written into existing column

update_col: func first applied, 64041 records less!
update_col: 85 cells written into existing column
update_col: func last applied, 64041 records less!
update_col: 85

In [564]:
# data_tag fehlt
maskA = translate_lex.data_tag_0 == ''
maskB = translate_lex.data_tag_1 == ''
mask = maskA | maskB
check_mask(translate_lex,mask,356699)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

check_mask:  360947 rows
time: 527 ms


In [565]:
# data_tag eindeutig
mask = translate_lex.data_tag_0 == translate_lex.data_tag_1
check_mask(translate_lex,mask,391328)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

check_mask:  776762 rows
time: 1.19 s


In [566]:
# data_tag uneindeutig
mask1 = translate_lex.data_tag_0 != translate_lex.data_tag_1
mask2 = translate_lex.data_tag_0 != ''
mask3 = translate_lex.data_tag_1 != ''
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask, 22364)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0 + ' ' + translate_lex[mask].data_tag_1
#grid(translate_lex,mask)

check_mask:  25857 rows
time: 325 ms


In [567]:
# Sicherstellen: data_tag fehlt nirgens komplett
mask = translate_lex.data_tag == ''
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask)

check_mask:  0 rows
time: 111 ms


In [568]:
# nach Priorität sortieren (laut Steuertabellen)
translate_lex['data_tag'] = tag_sortieren(translate_lex.data_tag)

time: 2.39 s


In [569]:
# translate_lex vorbereiten
translate_lex = pak.drop_cols(translate_lex,['data_tag_0','data_tag_1','data_tagZZ'])    
translate_lex = pak.rename_col(translate_lex,'data_tag','tag')    

time: 2.1 s


In [570]:
# pflege_tag_felder
translate_lex = pflege_tag_felder(translate_lex, translate_tagZ)

pflege_tag_felder Error: 368 Datensätze haben kein eindeutiges tagZZ
time: 33.3 s


In [571]:
# # translate_lex nachbereiten
translate_lex = pak.rename_col(translate_lex,'tag',   'data_tag')    
translate_lex = pak.rename_col(translate_lex,'tag_0', 'data_tag_0')   
translate_lex = pak.rename_col(translate_lex,'tag_1', 'data_tag_1')   
translate_lex = pak.rename_col(translate_lex,'tagZZ', 'data_tagZZ')   
translate_lex = pak.drop_cols(translate_lex,['tagZ'])    
grid(translate_lex,pak.sample)

20 rows out of 1163566


time: 12.3 s


In [572]:
mask = translate_lex.data == 'vervollständigt'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
663826,vervollständigt,T,vervollständigt,ADJA VVPP,ADJA,VVPP,A,1.974842,wiki Konjugierte_Form Verb gm,vervollständigen,B,vervollständigen,VVFIN,V,16.998781,0.875,14232.929589,vervollständigen
936494,vervollständigt_MADJ,B,vervollständigt,VVPP,VVPP,uzCMiQkm48,A,7.356947,merkmal Ptz2 manuell,vervollständigen,B,vervollständigen,VVFIN,V,16.998781,0.875,7859.675310,vervollständigen
1119201,vervollständigt_MADJ,B,vervollständigt,ADJA,ADJA,VgKW80xtop,A,2.868000,merkmal lemma,vervollständigt_MADJ,B,vervollständigt,ADJA,A,2.734531,1.000,3.585000,vervollständigt
1156664,vervollständigt_MADJ,B,vervollständigt,ADJA,ADJA,KcLjCYswfS,A,7.356947,merkmal Ptz2 manuell,vervollständigen,B,vervollständigen,VVFIN,V,16.998781,0.875,7859.675310,vervollständigen


time: 182 ms


In [573]:
#mask = wiktionary_lemma.tag.str.contains(' ')
#wiktionary_lemma[mask]

time: 76 ms


In [574]:
# Beispiel anzeigen
mask = translate_lex.data == 'lachen'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
966453,lachen,B,lachen,VVINF,VVINF,7iY9lDDv2P,V,45.436035,merkmal lemma,lachen,B,lachen,VVFIN,V,43.321558,1.0,2281.801767,lachen


time: 169 ms


In [575]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
736670,ausgerutscht,B,ausgerutscht,VVPP,VVPP,QqSkKqzWHB,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.99263,ausrutschen
1134874,ausgerutscht,B,ausgerutscht,ADJA,ADJA,KDW0ckbVOS,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.99263,ausrutschen


time: 293 ms


## Prüfen ==========

### Unique

In [577]:
# wiktionary_lemma: lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows
time: 109 ms


In [578]:
# wiktionary_merkmal: data_id vorhanden und nicht widersprüchlich?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

# Vergebene data_id nicht widersprüchlich?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 4.97 s


In [579]:
# translate_lex: data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows
time: 1.97 s


### Mögliche Probleme

In [580]:
# Mögliche Probleme
spalten = ['data','lemma','lemma_score']
mask =  translate_lex['same_lexlemma'] < 0.3
a = translate_lex[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

834 rows


time: 88.9 ms


In [581]:
mask = translate_lex.same_lexlemma < 0.5
a = pak.analyse_freqs(translate_lex[mask],['lemma','data'])
grid(a)

1301 rows


time: 380 ms


### Verwaisungen

In [582]:
# translate_lex lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,140)

msg = 'verwaist_lemma_id_endkontrolle'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  137 rows
Move 137 rows from 1163566
137 rows out of 350


time: 2.07 s


In [583]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,32)
# translate_lemma[mask]

msg = 'verwaist_lem_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 142 ms


In [584]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

msg = 'verwaist_lemma_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 99.3 ms


In [585]:
# wiktionary_lemma.data_id (nur leeren)
mask1 =  wiktionary_lemma.data_id != ''
mask2 = ~wiktionary_lemma.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_lemma.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,1)

wiktionary_lemma.loc[mask,'data_id'] = ''
grid(wiktionary_lemma,mask)

check_mask:  1 rows
1 rows out of 165737


time: 459 ms


In [586]:
# wiktionary_merkmal lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0,140)
# wiktionary_merkmal[mask]

msg = 'verwaist_lemma_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  137 rows
Move 137 rows from 2480102
137 rows out of 1577470


time: 11.9 s


In [587]:
# verwaist wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask4 = wiktionary_merkmal.data_id.isin(translate_lex_trash.data_id)
mask = mask1  &  mask2  &  mask3 # &  mask4
check_mask(wiktionary_merkmal,mask)

#grid(wiktionary_merkmal,mask)

check_mask:  311 rows
time: 2.76 s


In [588]:
# wiktionary_merkmal data_id
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,40)

msg = 'verwaist_data_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away
time: 4.55 s


In [589]:
#grid(wiktionary_merkmal,mask)

time: 57.5 ms


In [590]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])

time: 62.7 ms


In [591]:
# wiktionary_merkmal_text lemma_id
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)
# wiktionary_merkmal_text[mask]

#msg = 'verwaist_lemma_id_endkontrolle'
#wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg )
#grid(df)

check_mask:  0 rows
time: 267 ms


### NaN

In [592]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

time: 337 ms


In [593]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

time: 2.03 s


In [594]:
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.nnan(wiktionary_merkmal_text)

time: 251 ms


In [595]:
assert not pak.any_nan(translate_lemma)
#pak.nnan(translate_lemma)

time: 151 ms


In [596]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

time: 1.68 s


### Div

In [597]:
# lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 1.23 s


In [598]:
# lemma und data müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma != wiktionary_merkmal.data
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
time: 1.74 s


### Partizipien prüfen

In [599]:
#suche_L = 'missraten' 
#suche_P = 'missraten'  

suche_L = 'gefriertrocknen' 
suche_P = 'gefriergetrocknet'  

time: 74.1 ms


In [600]:
# wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data  == suche_P
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,score,hunspell
1931614,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,VVPP,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,1.0,1.442266,0
2296317,gefriergetrocknet,lemma,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriergetrocknet,ADJA,A,,,0.0,1.034178,0
2486944,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,11.0,1.442266,0


time: 712 ms


In [601]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag,lex_anz,nolex_anz
2049,gefriertrocknen,1054310,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,XkCBLk3Wge,VER,V,manuell,,31.451888,1,gefriertrockn,VVPP,18,18


time: 208 ms


In [602]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag,lex_anz,nolex_anz
2048,gefriergetrocknet,273380,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,PQFyiOn9vW,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,10.341779,1,friergetrocknen,VVPP,1,4


time: 314 ms


In [603]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,score,hunspell
1931614,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,VVPP,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,1.0,1.442266,0
2486944,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,11.0,1.442266,0


time: 3.21 s


### Testlexeme

In [604]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3 )
#grid(lemma_test)

79 rows loaded
Delete 3 rows from 79
time: 160 ms


In [605]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

time: 4.44 s


In [606]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
maskC = lemma_test.check_member == ''
mask = maskA | maskB | maskC
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

check_mask:  0 rows


,level,lemma_id,lemma,tag_soll,tag_ist,member_soll,check_tag,member_ist,check_member


time: 64 ms


In [607]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

check_mask:  0 rows
time: 64.5 ms


### lex_test

In [608]:
# lex_test
from x32_Funktionen import *
lex_test = check_lex_test(lex_test, translate_lex)
#lex_test

time: 17.1 s


In [609]:
# Fehler
maskA = lex_test.lex_id == ''
maskB = lex_test.check_tag == ''
mask = maskA | maskB 
check_mask(lex_test,mask)

probleme = list(lex_test[mask].lemma)
lex_test[mask]

check_mask:  0 rows


,level,lex_id,lex,lex_tag_soll,lex_tag_ist,lemma,lemma_ist,notiz,check_tag


time: 48.3 ms


In [610]:
# Sicherstellen: Keine Fehler
check_mask(lex_test,mask,0)

check_mask:  0 rows
time: 41.7 ms


In [611]:














bgc('green')

time: 43.4 ms


### Fehler analysieren

In [612]:
sucheL = ['ausrutschen']
suche  = ['ausgerutscht']

time: 35.2 ms


In [613]:
mask = lex_test.lex.isin(suche)
lex_test[mask]

,level,lex_id,lex,lex_tag_soll,lex_tag_ist,lemma,lemma_ist,notiz,check_tag
77,3,ausgerutscht,ausgerutscht,ADJA,VVPP,ausgerutscht,ausrutschen,Partizip,OK
78,3,ausgerutscht,ausgerutscht,VVPP,VVPP,ausrutschen,ausrutschen,Partizip,OK


time: 48.1 ms


In [614]:
mask = translate_lex.data.isin(suche)
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,score,lemma_lower
736670,ausgerutscht,B,ausgerutscht,VVPP,VVPP,QqSkKqzWHB,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.992630,ausrutschen
1075744,ausgerutscht,B,ausgerutscht,ADJA,ADJA,4sluvGO5ih,A,2.653274,merkmal lemma,ausgerutscht,B,ausgerutscht,ADJA,A,2.529797,1.00,13.266368,ausgerutscht
1134874,ausgerutscht,B,ausgerutscht,ADJA,ADJA,KDW0ckbVOS,A,8.624373,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,19.894762,0.75,81.992630,ausrutschen


time: 119 ms


In [615]:
mask = translate_lex_trash.data.isin(sucheL)
translate_lex_trash[mask]

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,...,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order,data_tag,data_tag_0,data_tag_1,same_lexlemma


time: 46.7 ms


In [616]:
mask = wiktionary_lemma.lemma.isin( sucheL + suche )
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,doc_len,doc_lemma,doc_tag,lex_anz,nolex_anz
111440,ausgerutscht,460363,0,ausgerutscht,ausgerutscht,False,1,ADJA,ADJA,Ti9bfYOTwf,ADJ,A,Ptz2,ausrutschen,2.529797,1,Ausgerutscht,VVPP,1,2
111860,ausrutschen,87503,0,ausrutschen,ausrutschen,False,1,VVFIN,VVFIN,IfxmNwPWyK,VER,V,,,19.894762,1,ausrutschen,VVFIN,14,15


time: 54.9 ms


In [617]:
mask1 = wiktionary_merkmal.data.isin(suche)
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
wiktionary_merkmal[mask].sort_values(['lemma_id','merkmal','data'])

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,score,hunspell
2405708,ausgerutscht,lemma,True,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausgerutscht,ADJA,A,,,0.0,2.529797,1
904567,ausrutschen,Ptz2,True,ausgerutscht,ausgerutscht,VVPP,ADJ,A,ausrutschen,VVFIN,V,,,1.0,9.354445,1
2465006,ausrutschen,Ptz2,True,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausrutschen,VVFIN,V,,,11.0,9.354445,1


time: 143 ms


In [618]:
grid(lex_test)

76 rows


time: 79.8 ms


## Speichern ==========

In [620]:
assert gründlich

time: 70.3 ms


In [621]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, ['data_id_old','data_id_bak'])

time: 1.29 s


In [622]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,         wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_merkmal,       wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,  wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(translate_lemma,          translate_lemma_filename_save) 
pak.dump_pickle(translate_lex,            translate_lex_filename_save)
pak.dump_pickle(translate_lex_trash,      translate_lex_trash_filename_save)
os.utime(verzeichnis_save)

time: 54.6 s


In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [623]:
raise Stop

Stop Time:  21:34:30
Elapsed:    2 hours, 23 mins, 6 secs


time: 70.7 ms


## suche_error ==========

In [ ]:
suche_debug = [suche_error[1]]
suche_debug

In [ ]:
# suche_debug wiktionary_lemma
mask = wiktionary_lemma.lemma.isin(suche_debug)
(wiktionary_lemma[mask])

In [ ]:
# suche_debug wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug)
(wiktionary_lemma_trash[mask])

In [ ]:
# translate_lex
search_str(translate_lex, suche_debug)

In [ ]:
# suche_debug translate_lemma
search_str(translate_lemma, suche_debug)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma.isin(suche_debug)
grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(['besonne'])
grid(wiktionary_merkmal,mask)

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal_text
mask = wiktionary_merkmal.lemma_id.isin(suche_debug)
ids = set(wiktionary_merkmal[mask].lemma_id)
mask = wiktionary_merkmal_text.lemma_id.isin(ids)
grid(wiktionary_merkmal_text,mask)

In [ ]:
raise Stop

## Fragmente

In [ ]:
raise Stop

In [ ]:
# rename_col 
translate_lex = pak.rename_col(translate_lex, 'data_id',    'lex_id')
translate_lex = pak.rename_col(translate_lex, 'data_home',  'lex_home')
translate_lex = pak.rename_col(translate_lex, 'data',       'lex')
translate_lex = pak.rename_col(translate_lex, 'data_score', 'lex_score')

translate_lex_full = pak.rename_col(translate_lex_full, 'data_id',    'lex_id')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_home',  'lex_home')
translate_lex_full = pak.rename_col(translate_lex_full, 'data',       'lex')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_score', 'lex_score')

translate_lemma = pak.rename_col(translate_lemma, 'same_lexlemma', 'same_lemlemma')

In [ ]:
# rank!
#translate_lex = pak.rank(translate_lex, col_score='score', cols_group='data_id', on_conflict='first')

In [ ]:
# rank!
# Ein data_id pro data und lemma_tagZZ.
vorher = translate_lex.shape[0]
translate_lex = pak.rank(translate_lex, col_score='score', cols_group=['data','lemma_tagZZ'], on_conflict='first')
print(vorher - translate_lex.shape[0], 'Datensätze gelöscht')

In [ ]:
# Dups in data + lemma: Sind stets verschiedene lemma_tagZZ
mask = translate_lex.duplicated(subset=['data','lemma'], keep=False)
a = translate_lex[mask].sort_values('data_id')
#grid(a)

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)